## Dependencies

In [ ]:
!pip install contractions
!pip install numba
!pip install bitarray
!pip install numpy
!pip install ir_measures
!pip install python-terrier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.5/205.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.

## Dataset

Importing Dataset, it checks whether it's on the drive and if it isn't then it is downloaded and finally we extract it.

In [ ]:
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz

--2025-11-25 07:59:52--  https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1035009698 (987M) [application/octet-stream]
Saving to: ‘collection.tar.gz’

collection.tar.gz   100%[===================>] 987.06M  13.9MB/s    in 89s     

2025-11-25 08:01:21 (11.1 MB/s) - ‘collection.tar.gz’ saved [1035009698/1035009698]



In [ ]:
!gunzip -v collection.tar.gz

collection.tar.gz:	 66.2% -- replaced with collection.tar


## Preprocessing

This function performs comprehensive text preprocessing on an input string s to prepare it for tasks like information retrieval, natural language processing, or machine learning. It applies a series of transformations to clean, normalize, and tokenize the text, ultimately returning a list of stemmed tokens. Here's a breakdown of each step:

- Converts all characters in the string to lowercase using s.lower().
Ensures uniformity and case-insensitive matching.

- Uses a __translation table__ (map with single character keys) to replace symbols with their written forms. This improves semantic clarity and indexing.

- Replaces visually similar or typographically styled characters with standard equivalents. Ensures consistent tokenization and avoids fragmentation.

- Uses a regular expression to remove periods (.) unless they are part of acronyms (e.g., Ph.D, U.S.A).

- Uses the contractions library to expand common English contractions. Improves semantic clarity and matching.

- Builds a translation table from string.punctuation that replaces each punctuation character with a space to maintain word boundaries. Prevents punctuation from interfering with token matching.

- Strips leading/trailing spaces. Replaces multiple consecutive spaces with a single space. Ensures clean token separation.

- Splits the cleaned string into a list of tokens using s.split(). Basic whitespace-based tokenization.

- Removes common English stopwords using NLTK's stopword list. Reduces noise and improves relevance in retrieval tasks.

- Applies the Porter Stemmer from NLTK to each token. Reduces words to their root form (e.g., "running"  "run"). Improves generalization and matching.

In [ ]:
import re
import string
import nltk
import contractions


nltk.download("stopwords", quiet=True)

def preprocess(
  s: str
) -> str:
  """
    Preprocessing Function

    Arguments:
      s: String to be preprocessed

    Returns:
      Preprocessed string
  """

  # Case Folding into lowercase using python built-in method
  # of the string class
  s = s.lower()

  # Replacing some special characters with their written form
  # using a translation table object for speed purposes
  spec_chars_to_text = {
    "&": " and ",
    "|": " or ",
    "°": " degree",
  }
  trans_table = str.maketrans(spec_chars_to_text)
  s = s.translate(trans_table)

  # Replacing some special character with a standard equivalent
  # using a translation table for speed purposes
  spec_chars_to_standard = {
      "‘": "'",
      "’": "'",
      "´": "'",
      "“": "\"",
      "”": "\"",
      "–": "-",
      "-": "-"
  }
  trans_table = str.maketrans(spec_chars_to_standard)
  s = s.translate(trans_table)

  # Remove periods "." in acronyms (e.g., Ph.D, U.S.A) but not in decimals (e.g., 3.14)
  # Regex breakdown:
  # \.           -> match a literal period
  # (?!)         -> negative lookahead: only match if the following pattern does NOT occur
  # (\S[^. ])    -> a non-whitespace character followed by something that's NOT a period or space
  # | \d         -> OR a digit (preserve decimal points like "3.14")
  s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s)

  # Expanding contractions using a reliable python library
  s = contractions.fix(s)

  # Removing punctuation, done with the usual translation
  # tables but starting from a built-in string with every symbol
  table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # the two strings must have the same length
  s = s.translate(table)

  # removes any leading, and trailing whitespaces
  s = s.strip()
  # removes any double whitespaces inside the string
  while "  " in s:
    s = s.replace("  ", " ")

  # Tokenizing using python string built-in method split
  s_list = s.split()

  # Removing Stopwords using the nltk library and list comprehension
  stopwords = nltk.corpus.stopwords.words('english')
  s_list = [token for token in s_list if token not in stopwords]

  # Stemming using Porter
  stemmer = nltk.stem.PorterStemmer().stem
  s_list = [stemmer(t) for t in s_list]

  return s

The preprocess() function is slow due to several inefficiencies that compound during repeated execution:
1. it reloads the NLTK stopwords list every time the function runs, which is unnecessary and costly.
2. it re-instantiates the Porter stemmer on each call instead of initializing it once globally.
3. the function rebuilds translation tables for character replacements repeatedly, even though they remain constant.
4. the regex are re-compiled every time the function is called
5. while checking for double spaces "  "

Thus we proceed improving those points:

In [ ]:
import re
import string
import nltk
import contractions

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Load once
nltk.download("stopwords", quiet=True)
STOPWORDS = set(stopwords.words("english"))
STEM = PorterStemmer().stem

# Precompile regex
MULTISPACE_RE = re.compile(r"\s+")
ACRONYM_RE = re.compile(r"\.(?!(\S[^. ])|\d)")

# Translation tables
TEXT_TRANS = str.maketrans({"&": " and ", "|": " or ", "°": " degree"})
STANDARD_TRANS = str.maketrans({
    "‘": "'", "’": "'", "´": "'", "“": "\"", "”": "\"", "–": "-", "-": "-"
})
PUNCT_TRANS = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

def preprocess(s: str) -> list:
    s = s.lower()
    s = s.translate(TEXT_TRANS)
    s = s.translate(STANDARD_TRANS)
    s = ACRONYM_RE.sub("", s)
    s = contractions.fix(s)
    s = s.translate(PUNCT_TRANS)
    s = MULTISPACE_RE.sub(" ", s).strip()
    tokens = s.split()
    tokens = [STEM(t) for t in tokens if t not in STOPWORDS]
    return tokens

## Inverted Index

### Profiling

Defining a decorator to measure and print the execution time of a function

In [ ]:
import time
from typing import Callable, Any

def profile(
  f: Callable[..., Any]
) -> Callable[..., Any]:
  """
    Decorator to measure and print the execution time of a function.

    Args:
        func (callable): The function to be timed.

    Returns:
        callable: Wrapped function with timing logic.
    """

  # A general signature for a python function
  def f_timer(*args, **kwargs):

    # Measuring start and end time
    start = time.time()
    result = f(*args, **kwargs)
    end = time.time()

    # Actually computing and printing time execution in ms
    ms = (end - start) * 1000
    print(f"{f.__name__} ({ms:.3f} ms)")
    return result

  return f_timer

### Loading and Preprocessing

This function preprocesses a dataset and returns a list of tuples, where each tuple corresponds to a single document and contains two elements:
- the document identifier (docno)
- a list of preprocessed tokens extracted from the document's content.

The output list preserves the original order of the dataset. This structure is useful for downstream tasks.

In [ ]:
@profile
def load_and_preprocess_dataset(
    filename: str,
    max_docs: int = -1,
    granularity: int = 10000
) -> list:
  """
    Loads and preprocesses a dataset from a tab-separated file.
    Each line is expected to contain: docno, document_text.
    Returns a list of tuples: (docno, list_of_tokens).

    Arguments:
      filename: filename of the dataset
      maxdocs: maximum number of documents to process

    Returns:
      Preprocessed Dataset
  """

  dataset = []

  # Iterating over file line by line
  with open(filename, 'r') as file:
    for doc_ind, line in enumerate(file):

      # First line is not a document, but an header (docno  document_text)
      if doc_ind == 0:
        continue

      # Skipping malformed lines (not in the format docno<\tab>document_text)
      try:
        docno, doc = line.strip().split('\t')
      except ValueError:
        print(f"Skipping malformed line {doc_ind}: {line.strip()}")
        continue

      # Print checkpoints every granularity times
      if doc_ind % granularity == 0:
        print(f"Processed {doc_ind} documents")

      # Preprocess document text and append structured entity
      preprocessed_doc = preprocess(doc)
      dataset.append((docno, preprocessed_doc))

      # Early stopping for debugging (never actually used)
      if max_docs > 0 and doc_ind + 1 >= max_docs:
          break

    return dataset

### Index builder

This function processes a tokenized document dataset to construct the core indexing structures required for an information retrieval system. For each document, it computes term frequencies and updates a lexicon that maps each unique term to a tuple containing its term ID, document frequency, and global term frequency. It also builds an inverted index composed of two dictionaries: one mapping each term ID to the list of document IDs where the term appears, and another mapping each term ID to the corresponding term frequencies in those documents. Additionally, it maintains a document index that records metadata for each document, including its identifier and length. Throughout the process, the function tracks collection-wide statistics such as the total number of documents, unique terms, and overall token count, which are essential for retrieval models and evaluation.

In [ ]:
from collections import Counter

@profile
def build_index(
  dataset: list
) -> tuple:
  """
    Processes a tokenized document dataset to build core indexing structures
    for an information retrieval system.

    Returns four components:
    1. lexicon: a dictionary mapping each unique term to a tuple (term_id,
      document_frequency, collection frequency).
    2. inverted index: two dictionaries mapping each term_id to:
      - a list of document IDs where the term appears (`docids`)
      - a list of corresponding term frequencies (`freqs`)
    3. document index: a list mapping each document ID to its metadata (docno,
      document length, URL, title).
    4. collection statistics: a dictionary containing total number of documents,
      unique terms, and total tokens.

    This function iterates through each document, computes term frequencies,
    updates the lexicon and posting lists, and tracks collection-wide
    statistics for use in retrieval models or evaluation.

    Arguments:
      dataset: preprocessed dataset list(docno, list_tokens)

    Returns:
      lexicon, inverted_index, document_index, collection_statistics
  """

  # Declaring some useful Data Structures:
  lexicon = {}            # term : (termid, df, cf)
  doc_index = []          # docid = (docno, doc_len)
  inv_d, inv_f = {}, {}   # termid : docid | termid : tf

  # Declaring some counter that will be used to compute Collection Statistics
  # (num_tokens, num_docs, num_terms)
  max_term_id = 0  # used as current id of the last added term in the vocabulary creation
  num_docs = 0
  num_tokens = 0
  stats = {}

  for docid, doc_info in enumerate(dataset):

    docno, tokens = doc_info
    token_tf = Counter(tokens) # iterable with (term, tf)

    # For each (token, tf) checks if the token is not in the lexicon
    # and then either add a new entry to the lexicon and a posting list
    # or update the entry of the lexicon and append to the posting list
    for token, tf in token_tf.items():

      if token not in lexicon:
        lexicon[token] = [max_term_id, 0, 0]  # [termid, df, cf]
        inv_d[max_term_id], inv_f[max_term_id] = [], []
        max_term_id += 1

      lexicon[token][1] += 1    # df
      lexicon[token][2] += tf   # cf
      term_id = lexicon[token][0]
      inv_d[term_id].append(docid)
      inv_f[term_id].append(tf)

    # Updating Document Index
    doc_index.append((docno, len(tokens)))

    # Updating collection statistics
    num_docs += 1
    num_tokens += len(tokens)

  # Computing Statistics Collection
  stats = {
      'num_docs': num_docs,       # used in IDF
      'num_terms': len(lexicon),  # never used
      'num_tokens': num_tokens,   # never used
      'average_document_length': num_tokens / num_docs # used in BM25 / BM11
  }

  return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

### Computation

Both the preprocessing and index building phases are computationally intensive, resulting in long execution times. If an error or disconnection occurs during execution, the entire process would normally need to be repeated from the beginning.

To address this, we will designed the preprocessing phase to operate in chunks as well as the indexing.

In [ ]:
preprocessed_dataset = load_and_preprocess_dataset(
  '/content/collection.tar'
)

lexicon, inverted_index, document_index, collection_statistics = build_index(preprocessed_dataset)

Processed 10000 documents
Processed 20000 documents
Processed 30000 documents
Processed 40000 documents
Processed 50000 documents
Processed 60000 documents
Processed 70000 documents
Processed 80000 documents
Processed 90000 documents
Processed 100000 documents
Processed 110000 documents
Processed 120000 documents
Processed 130000 documents
Processed 140000 documents
Processed 150000 documents
Processed 160000 documents
Processed 170000 documents
Processed 180000 documents
Processed 190000 documents
Processed 200000 documents
Processed 210000 documents
Processed 220000 documents
Processed 230000 documents
Processed 240000 documents
Processed 250000 documents
Processed 260000 documents
Processed 270000 documents
Processed 280000 documents
Processed 290000 documents
Processed 300000 documents
Processed 310000 documents
Processed 320000 documents
Processed 330000 documents
Processed 340000 documents
Processed 350000 documents
Processed 360000 documents
Processed 370000 documents
Processed 

KeyboardInterrupt: 

### Loading and Preprocessing 2

We define a function that loads and preprocesses a specific subset of the dataset. This approach allows us to handle the dataset in manageable chunks: each subset is independently preprocessed and saved to disk. By doing so, we can later load each preprocessed subset individually and incrementally construct the inverted index, optimizing both memory usage and processing efficiency.

In [ ]:
@profile
def load_and_preprocess_dataset_batch(
  filename: str,
  line_start: int,
  line_end: int
):
  """
  Loads and preprocesses a dataset from a tab-separated file.
  Each line is expected to contain: docno, document_text.
  Returns a list of tuples: (docno, list_of_tokens).

  Arguments:
    filename: filename of the dataset
    line_start: first line to load
    line_end: last line to load

  Returns:
    Preprocessed Dataset
  """
  assert line_end >= line_start
  dataset = []

  # Iterating over file line by line
  with open(filename, 'r') as file:
    for line_ind, line in enumerate(file):

      # Finding the right batch
      # This operation could be optimized but when operating with chunks
      # of 10^5 lines this cost is negligible
      if line_ind < line_start:
        continue
      if line_ind >= line_end:
        break

      # First line is not a document, but the header (docno  document_text)
      if line_ind == 0:
        continue

      # Skipping malformed lines
      try:
        docno, doc = line.strip().split('\t')
      except ValueError:
        print(f"Skipping malformed line {line_ind}: {line.strip()}")
        continue

      # Preprocess document text and append structured entity
      preprocessed_doc = preprocess(doc)
      dataset.append((docno, preprocessed_doc))

  return dataset

We use helper functions to store and retrieve Python objects from disk. This is achieved using the pickle library, which handles serialization (converting a Python object into a byte stream for storage) and deserialization (reconstructing the object from the stored byte stream). This allows us to efficiently save intermediate data and reload it when needed.

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

def load_object(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

This loop processes a large dataset in fixed-size chunks to optimize memory usage and enable incremental indexing. Each iteration loads and preprocesses a subset of the dataset defined by the granularity parameter, starting from a calculated line range based on chunk_id

In [3]:

# Define the number of lines per chunk, the chunk from which we should
# start and the path to the dataset
granularity = 1000000
chunk_id = 0
dataset_path = '/content/collection.tar'

# Counting the number of total lines to estimate the time needed for the work
with open(dataset_path, 'r') as file:
  num_lines = sum(1 for _line in file)
print(num_lines)


while True:

  # Compute chunk delimiters and output filename
  start_line = chunk_id * granularity
  end_line = (chunk_id + 1) * granularity
  filename = f"n_preprocessed_chunks_{chunk_id}"

  # reach the end of file?
  if start_line >= num_lines:
    print("All chunks processed.")
    break

  # Load and preprocess the current chunk
  data_chunk = load_and_preprocess_dataset_batch(
    dataset_path,
    start_line,
    end_line
  )

  # Save the preprocessed chunk to disk
  save_object(data_chunk, filename)
  print(f"Chunk {chunk_id} stored as '{filename}'")

  # Move to the next chunk
  chunk_id += 1

8841824
load_and_preprocess_dataset_batch (290531.407 ms)
Chunk 0 stored as 'n_preprocessed_chunks_0'
load_and_preprocess_dataset_batch (300421.321 ms)
Chunk 1 stored as 'n_preprocessed_chunks_1'
load_and_preprocess_dataset_batch (284955.432 ms)
Chunk 2 stored as 'n_preprocessed_chunks_2'
load_and_preprocess_dataset_batch (290458.543 ms)
Chunk 3 stored as 'n_preprocessed_chunks_3'
load_and_preprocess_dataset_batch (278484.198 ms)
Chunk 4 stored as 'n_preprocessed_chunks_4'
load_and_preprocess_dataset_batch (310484.432 ms)
Chunk 5 stored as 'n_preprocessed_chunks_5'
load_and_preprocess_dataset_batch (299484.111 ms)
Chunk 6 stored as 'n_preprocessed_chunks_6'
load_and_preprocess_dataset_batch (265330.990 ms)
Chunk 7 stored as 'n_preprocessed_chunks_7'
load_and_preprocess_dataset_batch (193740.198 ms)
Chunk 8 stored as 'n_preprocessed_chunks_8'
All chunks processed.


### Index Builder 2

Processes a tokenized batched document dataset to incrementally update core indexing structures (that are passed as parameters to the function) for an information retrieval system.
An additional parameter is needed to allow this incremental update:
- 'max_term_id'  that represent the current max term id of the data structures

In [ ]:
from collections import Counter

@profile
def build_index_batch(
  dataset: list,   # batch dataset list(docno, list(tokens))
  # -- partially built data structures --
  lexicon: dict,
  doc_index: list,
  inv_d: dict,
  inv_f: dict,
  max_term_id: int,  # current max token id
  num_docs: int,
  num_tokens: int,
  docid: int
) -> tuple[int, int, int, int]:
  """
    Processes a tokenized document dataset to build core indexing structures
    for an information retrieval system.

    Returns four components:
    1. lexicon: a dictionary mapping each unique term to a tuple (term_id,
      document_frequency, global_term_frequency).
    2. inverted index: two dictionaries mapping each term_id to:
      - a list of document IDs where the term appears (`docids`)
      - a list of corresponding term frequencies (`freqs`)
    3. document index: a list mapping each document ID to its metadata (docno,
      document length, URL, title).
    4. collection statistics: a dictionary containing total number of documents,
      unique terms, and total tokens.

    This function iterates through each document, computes term frequencies,
    updates the lexicon and posting lists, and tracks collection-wide
    statistics for use in retrieval models or evaluation.

    Arguments:
      dataset: preprocessed dataset

    Returns:
      lexicon, inverted_index, document_index, collection_statistics
  """

  for doc_info in dataset:

    # Extracting doc_info and building an iterable with (term, tf)
    docid += 1
    docno, tokens = doc_info
    token_tf = Counter(tokens)

    # For each (token, tf) checks if the token is not in the lexicon
    # and then either add a new entry to the lexicon and a posting list
    # or update the entry of the lexicon and append to the posting list
    for token, tf in token_tf.items():

      if token not in lexicon:
        lexicon[token] = [max_term_id, 0, 0]  # [term_id, dc, cf]
        inv_d[max_term_id], inv_f[max_term_id] = [], []
        max_term_id += 1

      lexicon[token][1] += 1
      lexicon[token][2] += tf
      term_id = lexicon[token][0]
      inv_d[term_id].append(docid)
      inv_f[term_id].append(tf)

    # Updating Document Index
    doc_index.append((docno, len(tokens)))
    num_docs += 1
    num_tokens += len(tokens)

  return (
    max_term_id,
    num_docs,
    num_tokens,
    docid
  )

Build the index structures in batches

In [ ]:
lexicon = {}
doc_index = []
inv_d = {}
inv_f = {}
max_term_id = 0
num_docs = 0
num_tokens = 0
docid = 0

In [4]:
chunk_ids = [i for i in range(9)]

for chunk_id in chunk_ids:

  # Load the preprocessed data chunk from disk
  filename = f"n_preprocessed_chunks_{chunk_id}"
  chunk = load_object(filename)

  # Update the inverted index and related metadata using the current chunk
  max_term_id, num_docs, num_tokens, docid = build_index_batch(
    chunk,
    lexicon,
    doc_index,
    inv_d,
    inv_f,
    max_term_id,
    num_docs,
    num_tokens,
    docid
  )

build_index_batch (52174.121 ms)
build_index_batch (56044.636 ms)
build_index_batch (62034.245 ms)
build_index_batch (64838.223 ms)
build_index_batch (67483.323 ms)
build_index_batch (70138.324 ms)
build_index_batch (74204.421 ms)
build_index_batch (90362.424 ms)
build_index_batch (50494.332 ms)



We are now generating statistics based on predefined variables, along with additional information computed dynamically.

In [ ]:
stats = {
    'num_docs': num_docs,
    'num_terms': len(lexicon),
    'num_tokens': num_tokens,
    'average_document_length': num_tokens / num_docs
}

Save data structures to .pkl file

In [ ]:
save_object(lexicon, 'lexicon')
save_object(doc_index, 'doc_index')
save_object(inv_d, 'inv_d')
save_object(inv_f, 'inv_f')
save_object(stats, 'stats')

The data are stored as list but loaded as np.arrays to take advantage of their computational efficiency and reduced memory footprint compared to standard Python structures.

In [ ]:
import numpy as np

inv_f = load_object('inv_f')
for term_id in inv_f:
  inv_f[term_id] = np.array(inv_f[term_id], dtype=np.int32)

inv_d = load_object('inv_d')
for term_id in inv_d:
  inv_d[term_id] = np.array(inv_d[term_id], dtype=np.int32)


lexicon = load_object('lexicon')
doc_index = load_object('doc_index')
stats = load_object('stats')

## Disk Compression

We now examine compression algorithms for storing posting lists on disk, focusing on both memory savings and the overhead introduced by compression and decompression times. This constitutes a preliminary study; in later sections of the notebook, we will extend the analysis to posting lists held in memory.


### Delta Encoding

In [ ]:
def delta_encode(numbers):
    """
    Applies delta compression to a sorted list of integers.
    """
    if len(numbers) == 0:
        return []

    deltas = [numbers[0]]
    for i in range(1, len(numbers)):
        deltas.append(numbers[i] - numbers[i - 1])
    return deltas

def delta_decode(deltas):
    """
    Reconstructs original list from delta-compressed integers.
    """
    if len(deltas) == 0:
        return []
    numbers = [deltas[0]]
    for i in range(1, len(deltas)):
        numbers.append(numbers[-1] + deltas[i])
    return numbers

### Variable Byte Encoding

We implement a set of helper functions to handle variable-byte (VB) encoding, which are responsible for efficiently converting integer values into a compact byte representation. In addition to these utilities, we develop a main function that takes as input the posting lists along with their docids and performs the actual encoding process. This function applies delta compression followed by VB encoding to reduce storage requirements and improve data retrieval efficiency.

Once the encoding logic is implemented, we conduct a series of experiments to evaluate its performance. Specifically, we measure both the execution time and the memory usage during the encoding process. These tests aim to assess how well the approach optimizes resource consumption while maintaining correctness and scalability for larger datasets.

#### Helper

In [ ]:
def vb_encode_number(n):
    """
    Encodes a single integer using variable-byte encoding.
    """
    assert n >= 0  # negative numbers not supported

    bytes_ = []
    while True:
        bytes_.insert(0, n % 128)  # prepend
        if n < 128: # fits entirely in this byte (7 bits)
            break
        n = n // 128
    bytes_[-1] += 128  # set continuation (end) bit on last byte to 1 (MSB = 1)
    return bytes_

def vb_encode_list(numbers):
    """
    Encodes a list of integers using VB encoding.
    Returns a bytearray.
    """
    encoded_bytes = []
    for number in numbers:
        byte_list = vb_encode_number(number)
        encoded_bytes.extend(byte_list) # extend the current list adding the byte list (VB is self-delimiting)
    return bytearray(encoded_bytes) # effectively uses one byte for element, contrary to a python list

def vb_decode(byte_stream):
    """
    Decodes a byte stream using variable-byte decoding.
    Returns a list of integers.
    """
    numbers = []
    n = 0
    for byte in byte_stream:
        if byte < 128:
            n = 128 * n + byte
        else:   # last byte
            n = 128 * n + (byte - 128)
            numbers.append(n)
            n = 0  # reset local aggregator
    return numbers

Delta encoding for inv_d is used to reduce the magnitude of the numbers by storing differences between consecutive values. This results in smaller, still positive integers, which improves the efficiency of Variable Byte (VB) encoding.

In [ ]:
@profile
def delta_vb_encode_inv_d(
  inv_d: dict,
) -> None:

  """
    It takes the posting lists along with their document ids
    and directly applies delta compression and variable-byte
    (VB) encoding to reduce RAM usage.

    Arguments:
      inv_d: inverted index with growing document ids

    Returns:
      None
  """

  for term_id in inv_d:
    inv_d[term_id] = delta_encode(inv_d[term_id])
    inv_d[term_id] = vb_encode_list(inv_d[term_id])

@profile
def delta_vb_decode_inv_d(
  inv_d: dict
) -> None:

  """
    It takes the posting lists along with their document ids
    and directly applies delta compression and variable-byte
    (VB) encoding to reduce RAM usage.

    Arguments:
      inv_d: inverted index with growing document ids

    Returns:
      None
  """

  for term_id in inv_d:
    inv_d[term_id] = vb_decode(inv_d[term_id])
    inv_d[term_id] = delta_decode(inv_d[term_id])


In [ ]:
@profile
def vb_encode_inv_f(
  inv_f: dict,
) -> None:
  """
    It takes the posting lists along with their term
    frequencies and directly applies variable-byte (VB)
    encoding to reduce RAM usage.

    Arguments:
      inv_f: inverted index with term frequencies

    Returns:
      None
  """

  for term_id in inv_f:
    inv_f[term_id] = vb_encode_list(inv_f[term_id])

@profile
def vb_decode_inv_f(
  inv_f: dict
) -> None:
  """
    It takes the posting lists along with their term
    frequencies and directly applies variable-byte (VB)
    decoding to reduce RAM usage.

    Arguments:
      inv_f: inverted index with term frequencies

    Returns:
      None
  """

  for term_id in inv_f:
    inv_f[term_id] = vb_decode(inv_f[term_id])


#### Test

In evaluating the performance and memory efficiency of posting list encoding and decoding, we report the overhead introduced by compression and decompression, quantify the memory savings achieved, and compare these results against a general-purpose ZIP algorithm.

In [ ]:
# Loading inv_d from disk
inv_d = load_object('inv_d')

# Encoding using Delta + Variabile-Byte Encoding
delta_vb_encode_inv_d(inv_d)

# Saving the encoded inv_d on disk
save_object(inv_d, 'comp_inv_d')

# Decoding using Variable-Byte + Delta Decoding
delta_vb_decode_inv_d(inv_d)

delta_vb_encode_inv_d (110736.371 ms)
delta_vb_decode_inv_d (98064.552 ms)


These are the compression results obtained with a general purpose compression algorithm

In [ ]:
!du -sh inv_d
!zip test_inv_d.zip inv_d
!du -sh test_inv_d.zip

1.1G	inv_d
  adding: inv_d (deflated 58%)
471M	test_inv_d.zip


These are the compression results obtained using Delta + Variable-Byte Encoding, which are slightly better

In [ ]:
!du -sh inv_d
!du -sh comp_inv_d

1.1G	inv_d
332M	comp_inv_d


Same procedure applied to inv_f (of course without Delta Encoding)


In [ ]:
# Loading inv_f from disk
inv_f = load_object('inv_f')

# Encoding inv_f using Variably-Byte Encoding
vb_encode_inv_f(inv_f)

# Storing compressed inv_f on disk
save_object(inv_f, 'comp_inv_f')

# Decoding inv_f using Variably-Byte Decoding
vb_decode_inv_f(inv_f)

vb_encode_inv_f (69061.333 ms)
vb_decode_inv_f (36405.070 ms)


In [19]:
!zip inv_f.zip inv_f
!du -sh inv_f.zip

  adding: inv_f (deflated 89%)
52M	inv_f.zip


In [ ]:
!du -sh inv_f
!du -sh comp_inv_f

455M	inv_f
244M	comp_inv_f


### Gamma Encoding

#### Helper Functions

In [ ]:
import math

def gamma_encode_number(n):
    """
    Encodes a single positive integer using Elias gamma encoding.
    """
    if n <= 0:
        raise ValueError("Gamma encoding only supports positive integers.")

    binary = bin(n)[2:]  # get the str binary representation of n and remove '0b' prefix
    offset = binary[1:]  # remove leading '1' always present in bin() representation
    length = len(offset)
    unary = '0' * length + '1' # write the unary representation of bin(n)
    return unary + offset  # concatenate the two strings

def gamma_encode_list(numbers):
    """
    Encodes a list of positive integers using Elias gamma encoding.
    Returns a string of bits.
    Padding is made of 0s because when decoding the padding won't
    make a decodable number
    """
    encoded_list = ''.join(gamma_encode_number(n) for n in numbers)
    return encoded_list

def gamma_decode(bitstream):
    """
    Decodes a bitstream encoded with Elias gamma encoding.
    Returns a list of integers.
    """
    numbers = []
    i = 0
    while i < len(bitstream):
        # Read unary prefix (length of offset)
        length = 0
        decodable = False
        while i < len(bitstream) and bitstream[i] == '0':
            decodable = True
            length += 1
            i += 1
        i += 1  # add the skipped '1' delimiter at the end of the length

        if not decodable:  # TODO: a che serve?
          continue

        # Read offset bits
        offset = bitstream[i:i + length]
        i += length

        binary = '1' + offset  # add the 1 that was redundant in encoding
        numbers.append(int(binary, 2))
    return numbers

In [ ]:
@profile
def delta_gamma_encode_inv_d(
  inv_d: dict
) -> None:

  """
    It takes the posting lists along with their document ids
    and directly applies delta compression and gamma encoding
    to reduce RAM usage.

    Arguments:
      inv_d: inverted index with growing document ids

    Returns:
      None
  """

  for term_id in inv_d:
    inv_d[term_id] = delta_encode(inv_d[term_id])
    inv_d[term_id] = gamma_encode_list(inv_d[term_id])

@profile
def delta_gamma_decode_inv_d(
  inv_d: dict
) -> None:

  """
    It takes the posting lists along with their document ids
    and directly applies delta compression and gamma decoding
    to reduce RAM usage.

    Arguments:
      inv_d: inverted index with growing document ids

    Returns:
      None
  """

  for term_id in inv_d:
    inv_d[term_id] = gamma_decode(inv_d[term_id])
    inv_d[term_id] = delta_decode(inv_d[term_id])

#### Testing

We are also experimenting with Gamma Encoding.
However, because Python lacks efficient native methods for bit manipulation, the algorithm is expected to run slowly and is unlikely to deliver significant memory improvements compared to variable-byte encoding.


In [ ]:
# Load the inverted index object from disk
inv_d = load_object('inv_d')

# Apply delta + gamma encoding to the inverted index
# This will compress the posting lists associated with each term
delta_gamma_encode_inv_d(inv_d)

# Initialize a counter for the total number of bits used
bits_used = 0

# Iterate over each term in the inverted index
for term_id in inv_d:
    # Add the length (in bits) of the encoded posting list for this term (characters as bits)
    bits_used += len(inv_d[term_id])

# Convert the total bit count into bytes (divide by 8) and print the result
print(bits_used / 8)

# Decode the inverted index back to its original form
# This ensures that the encoding/decoding process is reversible and correct
decoded_inv_d = delta_gamma_decode_inv_d(inv_d)

delta_gamma_encode_inv_d (172095.841 ms)
321223005.0
delta_gamma_decode_inv_d (381073.575 ms)


VB Encoding

| Dataset | Original Size | Compressed Size | Encode Time (ms) | Decode Time (ms) |
| ------- | ------------- | --------------- | ---------------- | ---------------- |
| inv_d   | 1.1 G         | 332 M           | 110,736.371      | 98,064.552       |
| inv_f   | 455 M         | 244 M           | 69,061.333       | 36,405.070       |


Gamma Encoding

| Dataset | Original Size | Compressed Size | Encode Time (ms) | Decode Time (ms) |
| ------- | ------------- | --------------- | ---------------- | ---------------- |
| inv_d   | 1.1 G         | 321 M           | 172,095.841      | 381,073.575      |


The compression for gamma encoding is slightly better than byte-variable encoding, but the improvement doesn't justify the additional encoding/decoding cost. I also won't apply it to the posting lists with frequencies, since it would not compress efficiently as we cannot use of delta compression.

## Boolean Retrieval

To validate the implementation, we develop a set of simple Boolean retrieval algorithms (AND & OR)

### Inverted Index Class

A simple class for the Inverted Index

In [ ]:
import math
import numpy as np
from typing import List, Dict, Tuple, Optional


class InvertedIndex:

    class PostingListIterator:
        """
        Description:
            Iterator over a posting list for a single term in the inverted index.
        Arguments:
            docids (np.ndarray): Array of document IDs containing the term.
            freqs (np.ndarray): Array of term frequencies corresponding to docids.
        """

        def __init__(self, docids: np.ndarray, freqs: np.ndarray):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0

        def docid(self) -> int | float:
            """
            Description:
                Returns the current document ID.
            Returns:
                int: Current document ID, or math.inf if iterator is at the end.
            """
            return math.inf if self._is_end_list() else self.docids[self.pos]

        def score(self):
            pass

        def next(self, target: Optional[int] = None):
            """
            Description:
                Advances the iterator. If target is provided, jump to the first document
                with ID >= target.
            Arguments:
                target (Optional[int]): Target document ID to skip to. Default is None.
            """
            if target is None:
                if not self._is_end_list():
                    self.pos += 1
            elif target > self.docid():
                # first document with ID >= target
                self.pos = np.searchsorted(self.docids, target, side='left')

        def len(self) -> int:
            """
            Description:
                Returns the number of documents in the posting list.
            Returns:
                int: Length of the posting list.
            """
            return len(self.docids)

        def _is_end_list(self) -> bool:
            """
            Description:
                Checks whether the iterator has reached the end of the posting list.
            Returns:
                bool: True if at end, False otherwise.
            """
            return self.pos >= len(self.docids)

    # ----------------- InvertedIndex main ----------------- #

    """
    Description:
        Inverted index storing the mapping from terms to posting lists, along with
        document statistics.
    Arguments:
        lexicon (Dict[str, Tuple[int, ...]]): Maps token to (termid, df, cf) .
        inv_d (List[np.ndarray]): List of arrays of document IDs per term.
        inv_f (List[np.ndarray]): List of arrays of term frequencies per term.
        stats (Dict[str, int]): Global statistics (num_docs, num_terms, num_tokens, average_document_length)
    """
    def __init__(
        self,
        lexicon: Dict[str, Tuple[int, ...]],
        inv_d: List[np.ndarray],
        inv_f: List[np.ndarray],
        stats: Dict[str, int],
    ):
        self.lexicon = lexicon
        self.inv_d = inv_d
        self.inv_f = inv_f
        self.stats = stats

    def num_docs(self) -> int:
        """
        Description:
            Returns the total number of documents in the index.
        Returns:
            int: Number of documents.
        """
        return self.stats['num_docs']

    def get_posting(self, termid: int) -> PostingListIterator:
        """
        Description:
            Returns a PostingListIterator for a given term ID.
        Arguments:
            termid (int): The term ID to retrieve the posting list for.
        Returns:
            PostingListIterator: Iterator over the posting list.
        """
        return InvertedIndex.PostingListIterator(
            self.inv_d[termid],
            self.inv_f[termid],
        )

    def get_termids(self, tokens: List[str]) -> List[int]:
        """
        Description:
            Returns a list of term IDs for the tokens that exist in the lexicon.
        Arguments:
            tokens (List[str]): List of tokens to look up.
        Returns:
            List[int]: List of corresponding term IDs.
        """
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids: List[int]) -> List['PostingListIterator']:
        """
        Description:
            Returns a list of PostingListIterators for a list of term IDs.
        Arguments:
            termids (List[int]): List of term IDs.
        Returns:
            List[PostingListIterator]: List of posting list iterators.
        """
        return [self.get_posting(termid) for termid in termids]

### Boolean And

Utilities functions for boolean AND retrieval

In [ ]:
import math
from typing import Set

@profile
def boolean_and(
    postings: List['InvertedIndex.PostingListIterator']
) -> Set[int]:
    """
    Description:
        Computes the intersection of multiple posting lists (Boolean AND).
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
    Returns:
        Set[int]: Set of document IDs present in all postings.
    """

    if not postings:
        return set()

    # Sort postings by length (shortest first for efficiency)
    # and creating result set (set of document ids)
    postings = sorted(postings, key=lambda p: p.len())
    results = set()

    # 1) Initialize results with the first posting (docids that contains the query term with shortest posting)
    #    keeping min and max docids to search faster subsequent postings
    min_docid, max_docid = postings[0].docid(), postings[0].docid()
    first_posting = postings[0]
    while first_posting.docid() != math.inf:  # math.inf returned as last docid
        results.add(first_posting.docid())
        max_docid = first_posting.docid()
        first_posting.next()

    # 2) Intersect with remaining postings
    for posting in postings[1:]:

        # Jumping to the first possible match
        posting.next(target=min_docid)
        current_results = set()

        # Iterating over posting while finding new min and max
        # docids and also creating the new result set
        n_min_docid, n_max_docid = -1, -1
        while posting.docid() != math.inf:
            doc_id = posting.docid()
            if doc_id in results:
                current_results.add(doc_id)   # implements the AND logic, adding only terms present in the prev postings and in the current
                if n_min_docid == -1:
                    n_min_docid = doc_id
                n_max_docid = doc_id
            elif doc_id > max_docid:  # previous founded max doc id
                break
            posting.next()

        # Updating result set and max min indices and if result
        # set is empty we can stop the execution
        results = current_results
        min_docid = n_min_docid
        max_docid = n_max_docid
        if not results:
            return set()

    return results

In [ ]:
from typing import Set

def query_process_and(
    query: str,
    index: InvertedIndex
) -> Set[int]:
    """
    Description:
        Processes a query using Boolean AND retrieval on an inverted index.
    Arguments:
        query (str): Input query string.
        index (InvertedIndex): Inverted index containing terms and posting lists.
    Returns:
        Set[int]: Set of document IDs that contain all query terms.
    """

    # Preprocess query (e.g., tokenization, lowercasing, stemming)
    qtokens: List[str] = preprocess(query)

    # Map tokens to term IDs present in the index
    qtermids: list[int] = index.get_termids(qtokens)

    # Retrieve posting lists for the term IDs
    postings: list['InvertedIndex.PostingListIterator'] = index.get_postings(qtermids)

    # Perform Boolean AND on posting lists
    return boolean_and(postings)

We tested the code with the query: "USA". The result retrieved is correct (checked using Ctrl+F) in a reasonable time interval

In [ ]:
# Create an instance of the InvertedIndex using the provided components:
# - lexicon: mapping of terms to IDs
# - inv_d: dictionary of posting lists (documents per term)
# - inv_f: frequency information for terms
# - stats: global statistics about the collection
inv_ind = InvertedIndex(
    lexicon,
    inv_d,
    inv_f,
    stats
)

# Define the query string we want to search for
query = "USA mouse cat"

# Process the query using the AND operator.
# This means we retrieve all documents that contain at least one of the query terms.
result_docs = query_process_and(query, inv_ind)

# Print the number of documents returned by the query
print(result_docs)

boolean_and (397.352 ms)
{np.int32(8363778), np.int32(8363779), np.int32(1339852), np.int32(1297), np.int32(707704), np.int32(4184601)}


### Boolean Or

Set of helper functions for boolean OR retrieval

In [ ]:
import math
from typing import List, Set

@profile
def boolean_or(
    postings: List['InvertedIndex.PostingListIterator']
) -> Set[int]:
    """
    Description:
        Computes the union of multiple posting lists (Boolean OR).
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
    Returns:
        Set[int]: Set of document IDs present in any of the postings.
    """

    # Creating result sets and adding docids of each posting to the set
    result = set()
    for posting in postings:
        while posting.docid() != math.inf:
            result.add(posting.docid())
            posting.next()
    return result

In [ ]:
from typing import Set

def query_process_or(
    query: str,
    index: InvertedIndex
) -> Set[int]:
    """
    Description:
        Processes a query using Boolean OR retrieval on an inverted index.
    Arguments:
        query (str): Input query string.
        index (InvertedIndex): Inverted index containing terms and posting lists.
    Returns:
        Set[int]: Set of document IDs that contain all query terms.
    """

    # Preprocess query (e.g., tokenization, lowercasing, stemming)
    qtokens: List[str] = preprocess(query)

    # Map tokens to term IDs present in the index
    qtermids: list[int] = index.get_termids(qtokens)

    # Retrieve posting lists for the term IDs
    postings: list['InvertedIndex.PostingListIterator'] = index.get_postings(qtermids)

    # Perform Boolean AND on posting lists
    return boolean_or(postings)

We tested the code with the query: "USA". The result is retrieved in a reasonable time interval

In [ ]:
# Create an instance of the InvertedIndex using the provided components:
# - lexicon: mapping of terms to IDs
# - inv_d: dictionary of posting lists (docids per term)
# - inv_f: dictionary of posting lists (term frequencies per term)
# - stats: global statistics about the collection
inv_ind = InvertedIndex(
    lexicon,
    inv_d,
    inv_f,
    stats
)

# Define the query string we want to search for
query = "USA cat"

# Process the query using the OR operator.
# This means we retrieve all documents that contain at least one of the query terms.
result_docs = query_process_or(query, inv_ind)

# Print the number of documents returned by the query
print(len(result_docs))

boolean_or (392.299 ms)
77855


## Ranked-Retrieval

We implemented a range of scoring algorithms to explore different approaches to document ranking and retrieval effectiveness. We started with the simplest models, namely Term Frequency (TF) and Inverse Document Frequency (IDF), to establish a baseline understanding of how individual term occurrence and term rarity influence relevance scoring. Building upon these foundations, we combined the two into the well-known TF-IDF weighting scheme, which balances term importance within a document against its commonness across the entire collection. Finally, we implemented several members of the BM family of models, including BM11, BM15, and BM25, to examine how variations in term saturation and document length normalization affected retrieval performance and ranking accuracy.

### Inverted Index

We extended the InvertedIndex class implemented above, by passing a scorer Callable object to easily implement the different scoring function (tf, idf, tf-idf, BM11, BM15, BM25)
- **Flexible**: easy to swap scoring models
- **Extensible**: supports experimentation with ranking functions

The scorer function takes as input the PostingListIterator object in order to compute correctly the float score for the current term.


In [ ]:
import math
import numpy as np
from typing import List, Dict, Tuple, Callable, Optional


class InvertedIndex:

    class PostingListIterator:
        """
        Description:
            Iterator over a posting list for a single term in the inverted index.
        Arguments:
            docids (np.ndarray): Array of document IDs containing the term.
            freqs (np.ndarray): Array of term frequencies corresponding to docids.
            doc (List[Tuple]): Dictionary storing document statistics (docid, length) 
            stats (Dict[str, int]): Global statistics (num_docs, num_terms, num_tokens, average_document_length)
            scorer (Callable[['InvertedIndex.PostingListIterator'], float]): Callback scoring function 
        """

        def __init__(
            self,
            docids: np.ndarray,
            freqs: np.ndarray,
            termid: int,
            token: str,
            lexicon: Dict[str, Tuple],
            doc: List[Tuple],
            stats: Dict[str, int],
            scorer: Callable[['InvertedIndex.PostingListIterator'], float]
        ):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.termid = termid
            self.token = token
            self.lexicon = lexicon
            self.doc = doc
            self.stats = stats
            self._scorer = scorer

        def docid(self) -> int | float:
            """
            Description:
                Returns the current document ID.
            Returns:
                int: Current document ID, or math.inf if iterator is at the end.
            """
            return math.inf if self._is_end_list() else self.docids[self.pos]

        def score(self) -> float:
            """
            Description:
                Computes the score for the current document, which is constant
                for every document in the same posting
            Returns:
                float: Score for the current document, or math.inf if iterator is at the end.
            """
            if self._is_end_list():
                return math.inf
            return self._scorer(self)

        def next(self, target: Optional[int] = None):
            """
            Description:
                Advances the iterator. If target is provided, jump to the first document
                with ID >= target.
            Arguments:
                target (Optional[int]): Target document ID to skip to. Default is None.
            """
            if target is None:
                if not self._is_end_list():
                    self.pos += 1
            elif target > self.docid():
                self.pos = np.searchsorted(self.docids, target, side='left')

        def len(self) -> int:
            """
            Description:
                Returns the number of documents in the posting list.
            Returns:
                int: Length of the posting list.
            """
            return len(self.docids)

        def _is_end_list(self) -> bool:
            """
            Description:
                Checks whether the iterator has reached the end of the posting list.
            Returns:
                bool: True if at end, False otherwise.
            """
            return self.pos >= len(self.docids)

    # ----------------- InvertedIndex main ----------------- #

    """
    Description:
        Inverted index storing the mapping from terms to posting lists, along with
        document statistics.
    Arguments:
        lexicon (Dict[str, Tuple[int, ...]]): Maps token to (termid, df, cf)
        inv_d (List[np.ndarray]): List of arrays of document IDs per term.
        inv_f (List[np.ndarray]): List of arrays of term frequencies per term.
        doc (List[Tuple]): Document statistics list(docid, length)
        stats (Dict[str, int]): Global statistics (num_docs, num_terms, num_tokens, average_document_length)
        scorer (Callable[['InvertedIndex.PostingListIterator'], float]): Callback scoring function 
    """

    def __init__(
        self,
        lexicon: Dict[str, Tuple[int, ...]],
        inv_d: List[np.ndarray],
        inv_f: List[np.ndarray],
        doc: List[Tuple],
        stats: Dict[str, int],
        scorer: Callable[['InvertedIndex.PostingListIterator'], float]
    ):
        self.lexicon = lexicon
        self.inv_d = inv_d
        self.inv_f = inv_f
        self.doc = doc
        self.stats = stats
        self.scorer = scorer

    def num_docs(self) -> int:
        """
        Description:
            Returns the total number of documents in the index.
        Returns:
            int: Number of documents.
        """
        return self.stats['num_docs']

    def get_posting(self, termid: int, term: str) -> 'PostingListIterator':
        """
        Description:
            Returns a PostingListIterator for a given term ID.
        Arguments:
            termid (int): The term ID to retrieve the posting list for.
        Returns:
            PostingListIterator: Iterator over the posting list.
        """
        return InvertedIndex.PostingListIterator(
            self.inv_d[termid],
            self.inv_f[termid],
            termid,
            term,
            self.lexicon,
            self.doc,
            self.stats,
            self.scorer
        )

    def get_termids(self, tokens: List[str]) -> List[int]:
        """
        Description:
            Returns a list of term IDs for the tokens that exist in the lexicon.
        Arguments:
            tokens (List[str]): List of tokens to look up.
        Returns:
            List[int]: List of corresponding term IDs.
        """
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(
        self,
        termids: List[int],
        terms: List[str]
    ) -> List['PostingListIterator']:
        """
        Description:
            Returns a list of PostingListIterators for a list of term IDs.
        Arguments:
            termids (List[int]): List of term IDs.
        Returns:
            List[PostingListIterator]: List of posting list iterators.
        """
        return [self.get_posting(termid, term) for termid, term in zip(termids, terms)]

### Helper Functions

The functions and classes in this code collectively implement the core logic for ranked retrieval in an information retrieval (IR) system using two fundamental query evaluation strategies Term-at-a-Time (TAAT) and Document-at-a-Time (DAAT) along with a priority queue mechanism (TopQueue) for efficient ranking and top-k result management.

At the foundation of this setup is the TopQueue class, which maintains the top-k highest-scoring documents encountered during retrieval. It uses a min-heap data structure (via Python's heapq module) to store document-score pairs, ensuring that insertion and removal are efficient (O(log k)). The queue dynamically updates its threshold the minimum score required for a document to remain in the top k as new results are inserted. This allows the system to discard low-scoring documents early, optimizing both time and memory.

In [ ]:
import heapq
from typing import List, Tuple


class TopQueue:
    """
    Description:
        Maintains the top-k scored items in a min-heap with a dynamic threshold.
    Arguments:
        k (int): Maximum number of items to keep. Default is 10.
        threshold (float): Minimum score required for an item to enter. Default is 0.0.
    """
    def __init__(self, k: int = 10, threshold: float = 0.0):
        self.queue: List[Tuple[float, int]] = []  # heap of (score, docid)
        self.k: int = k
        self.threshold: float = threshold

    def size(self) -> int:
        """Returns the number of items currently in the queue."""
        return len(self.queue)

    def would_enter(self, score: float) -> bool:
        """
        Checks if a given score is high enough to enter the queue.
        Arguments:
            score (float): Score to test.
        Returns:
            bool: True if score exceeds current threshold.
        """
        return score > self.threshold

    def clear(self, new_threshold: float = -1):  # NOTE changed to -1 for types
        """
        Clears the queue.
        Arguments:
            new_threshold (float, optional): If provided, updates the threshold.
        """
        self.queue = []
        if new_threshold >= 0.0:
            self.threshold = new_threshold

    def __repr__(self) -> str:
        return f'<TopQueue: {self.size()} items, th={self.threshold}, {self.queue}>'

    def insert(self, docid: int, score: float) -> bool:
        """
        Inserts a document with its score into the top-k queue if it exceeds the threshold.
        Arguments:
            docid (int): Document ID.
            score (float): Score of the document.
        Returns:
            bool: True if the item was inserted, False otherwise.
        """
        if score > self.threshold:
            if self.size() >= self.k:
                # Replace the smallest element if heap is full
                heapq.heapreplace(self.queue, (score, docid))   # log(k)
            else:
                heapq.heappush(self.queue, (score, docid))      # log(k)

            # Update threshold to the smallest score in the heap if full
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True

        return False


The taat() function implements the Term-at-a-Time retrieval model. In this approach, the system processes one posting list (i.e., one term's document list) at a time. For each term, it iterates through all documents containing that term and accumulates their scores in a dictionary (doc_scores). Once all terms have been processed, the TopQueue selects and maintains the top-k highest-scoring documents. 

In [ ]:
import math
from typing import List, Set, Tuple
from collections import defaultdict

@profile
def taat(
    postings: List['InvertedIndex.PostingListIterator'],
    k: int = 1000
) -> List[Tuple[float, int]]:
    """
    Description:
        Computes term-at-a-time (TAAT) scoring for a list of posting lists.
        Accumulates scores for each document across all postings and returns
        the top-k documents.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
        k (int): Number of top-scoring documents to return. Default is 1000.
    Returns:
        List[Tuple[float, int]]: List of (score, docid) tuples, sorted in descending order.
    """

    # Accumulate scores for each document
    # Default dict is used since it allows automatic score initialization to 0.0 and then the += posting.score()
    doc_scores: defaultdict[int, float] = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            doc_scores[current_docid] += posting.score() # type: ignore since current_docid is not math.inf, thus int
            posting.next()
            current_docid = posting.docid()

    # Use TopQueue to maintain top-k scores
    top = TopQueue(k)
    for docid, score in doc_scores.items():
        top.insert(docid, score)

    # Return top-k results sorted by score descending
    return sorted(top.queue, reverse=True)


The daat() function implements the Document-at-a-Time retrieval model. Instead of iterating term by term, DAAT traverses all posting lists simultaneously, aligning documents across terms by their IDs. The helper function min_docid() identifies the smallest document ID currently being pointed to across all posting lists, effectively synchronizing the iteration. For each document ID, the algorithm aggregates contributions (term scores) from all posting lists that include that document, computes a total score, and then inserts it into the TopQueue. Once processed, the posting lists advance to the next relevant document, and the loop continues until all lists are exhausted.

In [ ]:
import math
from typing import List, Set, Tuple


def min_docid(
    postings: List['InvertedIndex.PostingListIterator']
) -> int | float:
    """
    Description:
        Returns the smallest current docid among all posting list iterators.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
    Returns:
        int: Minimum current docid, or math.inf if all postings are exhausted.
    """
    # iterating over the heads of the postings and returning the min
    min_docid_value = math.inf
    for p in postings:
        if not p._is_end_list():
            min_docid_value = min(p.docid(), min_docid_value)
    return min_docid_value



@profile
def daat(
    postings: List['InvertedIndex.PostingListIterator'],
    k: int = 1000
) -> List[Tuple[float, int]]:
    """
    Description:
        Computes document-at-a-time (DAAT) scoring for a list of posting lists.
        Scores each document across all postings simultaneously and returns
        the top-k documents.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
        k (int): Number of top-scoring documents to return. Default is 1000.
    Returns:
        List[Tuple[float, int]]: List of (score, docid) tuples, sorted in descending order.
    """

    top = TopQueue(k)

    # Initialize current_docid as the smallest docid among all postings
    current_docid = min_docid(postings)

    while current_docid != math.inf:
        score = 0.0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting._is_end_list():
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score) # type: ignore since current_docid != math.inf
        current_docid = next_docid

    # Return top-k results sorted by score descending
    return sorted(top.queue, reverse=True)

The query_process() function takes a raw query string, preprocesses it into tokens (using a preprocessing function such as stemming or stop-word removal), retrieves the corresponding term IDs from the inverted index, and obtains their posting list iterators. Depending on the specified retrieval mode (method parameter), it then calls either the TAAT or DAAT function to compute and return the top-k ranked results.

In [ ]:
def query_process(
    query: str,
    index: InvertedIndex,
    k: int = 1000,
    method: str = "D"
) -> List[Tuple[float, int]]:
    """
    Description:
        Processes a query using TAAT or DAAT retrieval on an inverted index.
    Arguments:
        query (str): Input query string.
        index (InvertedIndex): Inverted index containing terms and postings.
        k (int): Number of top-scoring documents to return. Default is 1000.
        method (str): Scoring method to use ('D' for DAAT, 'T' for TAAT). Default is 'D'.
    Returns:
        List[Tuple[float, int]]: List of top-k (score, docid) results.
    """

    # Preprocess query into tokens
    qtokens: List[str] = preprocess(query)

    # Map tokens to term IDs
    qtermids: List[int] = index.get_termids(qtokens)

    # Retrieve posting lists for the term IDs
    postings: List['InvertedIndex.PostingListIterator'] = index.get_postings(
        qtermids,
        qtokens
    )

    # Compute TAAT or DAAT top-k results
    if method == "D":
        return daat(postings, k)
    else:
        return taat(postings, k)

### TF

In [ ]:
import numpy as np

class TFScorer:
    """
    TF scoring function.
    Arguments:
    """

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:

        # Extracting term frequency and computing
        tf: int = posting.freqs[posting.pos]
        tf_w: float = np.log(1 + tf)

        return tf_w

The execution time is acceptable, and the test runs successfully.



In [ ]:
scorer = TFScorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer = scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (534.696 ms)
[(np.float64(3.6635616461296463), np.int32(1484875)), (np.float64(3.5553480614894135), np.int32(5733950)), (np.float64(3.401197381662155), np.int32(8363778)), (np.float64(3.401197381662155), np.int32(3874864)), (np.float64(3.295836866004329), np.int32(5733949)), (np.float64(3.295836866004329), np.int32(527509)), (np.float64(3.2188758248682006), np.int32(611370)), (np.float64(3.1780538303479453), np.int32(8363779)), (np.float64(3.1780538303479453), np.int32(3368134)), (np.float64(3.1780538303479453), np.int32(2397556))]
daat (880.180 ms)
[(np.float64(3.6635616461296463), np.int32(1484875)), (np.float64(3.5553480614894135), np.int32(5733950)), (np.float64(3.401197381662155), np.int32(8363778)), (np.float64(3.401197381662155), np.int32(3874864)), (np.float64(3.295836866004329), np.int32(5733949)), (np.float64(3.295836866004329), np.int32(527509)), (np.float64(3.2188758248682006), np.int32(611370)), (np.float64(3.1780538303479453), np.int32(8363779)), (np.float64(3.178053

### IDF

In [ ]:
import numpy as np

class IDFScorer:
    """
    IDF scoring function.
    Arguments:
    """

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:
        # Extracting doc frequency, number of docs and computing idf weight
        df: int = posting.lexicon[posting.token][1]
        N: int = posting.stats['num_docs']
        idf_w: float = np.log(N / df)

        return idf_w

Time is acceptable and tests run successfully

In [ ]:
scorer = IDFScorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (435.251 ms)
[(np.float64(17.47011581576931), np.int32(8363779)), (np.float64(17.47011581576931), np.int32(8363778)), (np.float64(17.47011581576931), np.int32(4184601)), (np.float64(17.47011581576931), np.int32(1339852)), (np.float64(17.47011581576931), np.int32(707704)), (np.float64(17.47011581576931), np.int32(1297)), (np.float64(12.083379111121442), np.int32(232930)), (np.float64(12.083379111121442), np.int32(158694)), (np.float64(12.083379111121442), np.int32(130458)), (np.float64(12.083379111121442), np.int32(1298))]
daat (796.148 ms)
[(np.float64(17.47011581576931), np.int32(8363779)), (np.float64(17.47011581576931), np.int32(8363778)), (np.float64(17.47011581576931), np.int32(4184601)), (np.float64(17.47011581576931), np.int32(1339852)), (np.float64(17.47011581576931), np.int32(707704)), (np.float64(17.47011581576931), np.int32(1297)), (np.float64(12.083379111121442), np.int32(387050)), (np.float64(12.083379111121442), np.int32(343339)), (np.float64(12.083379111121442), np.

### TF IDF

In [ ]:
import numpy as np

class TFIDFScorer:
    """
    TF-IDF scoring function.
    Arguments:
    """
    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:
        """
        Description:
            Computes the TF-IDF score for a given posting.
        """
        # Extracting term frequency, doc frequency and number of docs
        tf = posting.freqs[posting.pos]
        df = posting.lexicon[posting.token][1]
        N = posting.stats['num_docs']

        # Computing tf-idf weight
        tf_w = np.log(1 + tf)
        idf_w = np.log(N / df)
        tf_idf_w = tf_w * idf_w

        return tf_idf_w

Time is acceptable and tests run successfully

In [ ]:
scorer = TFIDFScorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (638.263 ms)
[(np.float64(22.900067434274966), np.int32(1484875)), (np.float64(21.675178999018136), np.int32(5733950)), (np.float64(20.54869052963537), np.int32(5733949)), (np.float64(20.44338616312423), np.int32(3874864)), (np.float64(20.39084744942713), np.int32(8363778)), (np.float64(19.447448451753104), np.int32(611370)), (np.float64(18.913448568973624), np.int32(8363779)), (np.float64(18.35688035827288), np.int32(4430168)), (np.float64(18.003621091775226), np.int32(527509)), (np.float64(17.970049571299597), np.int32(7150429))]
daat (1006.136 ms)
[(np.float64(22.900067434274966), np.int32(1484875)), (np.float64(21.675178999018136), np.int32(5733950)), (np.float64(20.54869052963537), np.int32(5733949)), (np.float64(20.44338616312423), np.int32(3874864)), (np.float64(20.39084744942713), np.int32(8363778)), (np.float64(19.447448451753104), np.int32(611370)), (np.float64(18.913448568973624), np.int32(8363779)), (np.float64(18.35688035827288), np.int32(4430168)), (np.float64(18.003

### BM15

In [ ]:
class BM15Scorer:
    """
    BM15 scoring function.
    Arguments:
        k1 (float): Term frequency saturation parameter.
    """
    def __init__(self, k1: float = 1.2):
        self.k1: float = k1

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:
        # Extracting term frequency, number of documents and doc frequency
        tf: int = posting.freqs[posting.pos]
        N: int = posting.stats['num_docs']
        df: int = posting.lexicon[posting.token][1]

        # Computing bm1 weight with the actual formula
        bm1: float = np.log((N - df + 0.5) / (df + 0.5))

        # Computing the Bounded TF weight approximation
        # and BM15 = BM1 * (tf) / (k_1 + tf)
        tf_approx: float = tf / (self.k1 + tf)
        bm15: float = bm1 * tf_approx

        return bm15

Time is almost acceptable and tests run successfully (we will worry about making it faster later)

In [ ]:
scorer = BM15Scorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (672.266 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.926860918228257), np.int32(7150429)), (np.float64(8.926860918228257), np.int32(7150428))]
daat (1043.604 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.9268609182282

### BM11

In [ ]:
class BM11Scorer:
    """
    BM11 scoring function.
    Arguments:
        k1 (float): Term frequency saturation parameter.
    """
    def __init__(self, k1: float = 1.2):
        self.k1: float = k1

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:
        # Extracting term frequency, number of documents and doc frequency
        tf: int = posting.freqs[posting.pos]
        N: int = posting.stats['num_docs']
        df: int = posting.lexicon[posting.token][1]

        # Computing bm1 weight with the actual formula
        bm1: float = np.log((N - df + 0.5) / (df + 0.5))

        # Computing tilde tf, then the Bounded TF weight approximation
        # and finally BM11 = BM1 * (tf') / (k_1 + tf')  
        dl: int = posting.doc[posting.docid() - 1][1] # type: ignore
        avg_dl: int = posting.stats['average_document_length']
        tf_tilde: float = tf * avg_dl / dl
        tf_approx: float = tf_tilde / (self.k1 + tf_tilde)
        bm11: float = bm1 * tf_approx  

        return bm11


Time is almost acceptable and tests run successfully (we will worry about making it faster later)

In [ ]:
scorer = BM11Scorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (999.479 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.926860918228257), np.int32(7150429)), (np.float64(8.926860918228257), np.int32(7150428))]
daat (1386.923 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.9268609182282

### BM25

In [ ]:
class BM25Scorer:
    """
    BM25 scoring function.
    Arguments:
        k1 (float): Term frequency saturation parameter.
        b (float): Document length normalization parameter.
    """
    def __init__(self, k1: float = 1.2, b: float = 0.75):
        self.k1: float = k1
        self.b: float = b

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> float:
        # Extracting term frequency, number of documents and doc frequency
        tf: int = posting.freqs[posting.pos]
        N: int = posting.stats['num_docs']
        df: int = posting.lexicon[posting.token][1]

        # Computing bm1 weight with the actual formula
        bm1: float = np.log((N - df + 0.5) / (df + 0.5))

        # Computing the tilde tf, then the Bounded TF weight approximation
        # and finally BM25 = BM1 * (tf) / (k_1 + tf_tilde)  
        # tf_tilde = tf / (1 - b) + b * (dl / avg_dl)
        dl: int = posting.doc[posting.docid() - 1][1] # type: ignore
        avg_dl: int = posting.stats['average_document_length']
        norm_factor: float = (1 - self.b) + self.b * (dl / avg_dl)  # convex combination
        tf_tilde: float = tf / norm_factor
        tf_approx: float = tf_tilde / (self.k1 + tf_tilde)
        bm25: float = bm1 * tf_approx

        return bm25

Time is not acceptable and tests run successfully (we will worry about making it faster later)

In [ ]:
scorer = BM25Scorer()
inv_ind = InvertedIndex(
  lexicon,
  inv_d,
  inv_f,
  doc_index,
  stats,
  scorer
)

query = "mouse cat USA"
print(query_process(query, inv_ind, 10, "T"))
print(query_process(query, inv_ind, 10, "D"))

taat (1138.629 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.926860918228257), np.int32(7150429)), (np.float64(8.926860918228257), np.int32(7150428))]
daat (1629.016 ms)
[(np.float64(10.949743618358667), np.int32(8363778)), (np.float64(10.586036503821358), np.int32(8363779)), (np.float64(9.71489299972576), np.int32(5733950)), (np.float64(9.65394411103718), np.int32(1339852)), (np.float64(9.493729408374108), np.int32(3874864)), (np.float64(9.381535166040274), np.int32(1484875)), (np.float64(9.290568032765567), np.int32(611370)), (np.float64(9.167478520133201), np.int32(5733949)), (np.float64(8.926860918228

## Experimental Evaluation

An experimental evaluation framework in information retrieval (IR) provides a systematic and reproducible approach for assessing the performance of retrieval models, ranking algorithms, and indexing techniques. Its primary goal is to measure how effectively a retrieval system satisfies users' information needs, by comparing the system's ranked outputs against a set of relevance judgments (qrels) provided by human assessors. Such a framework generally consists of several key components: a document collection (e.g., web pages, news articles, or academic papers), a set of queries or topics, corresponding ground-truth relevance assessments, and a suite of evaluation metrics such as Mean Average Precision (MAP), Normalized Discounted Cumulative Gain (nDCG), or Mean Reciprocal Rank (MRR).

### Download Queries and QRels

Downloading queries, qrels and unzipping queries

In [ ]:
# Download queries for MS MARCO
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz

# Downlaod relevance judgements for MS MARCO queries
# example:   23849 0 1034183 3  (topic_id, fix, docno, relevance)
!wget https://trec.nist.gov/data/deep/2019qrels-pass.txt
!wget https://trec.nist.gov/data/deep/2020qrels-pass.txt

# Uncompress the queries
!gunzip msmarco-test2019-queries.tsv.gz
!gunzip msmarco-test2020-queries.tsv.gz

--2025-11-25 07:30:15--  https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4276 (4.2K) [application/x-gzip]
Saving to: ‘msmarco-test2019-queries.tsv.gz’

msmarco-test2019-qu 100%[===================>]   4.18K  --.-KB/s    in 0s      

2025-11-25 07:30:16 (1.03 GB/s) - ‘msmarco-test2019-queries.tsv.gz’ saved [4276/4276]

--2025-11-25 07:30:16--  https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4131 

### Evaluation-Test Split

We have chosen MSMARCO 2019 as our evaluation set and MSMARCO 2020 as our test set. The evaluation set will be used to tune the parameters of various scoring functions and to identify the best-performing function. The test set will then serve to compare our selected scoring function implementation against the PyTerrier implementation and to compute performance statistics.

However, it is clear that our initial plan for the evaluation set is not entirely valid: we cannot both tune a model and compare it with other models using the same evaluation data. To address this, we split the evaluation set into two distinct subsets with a 70-30 ratio. The larger portion is reserved for model comparison, while the smaller portion is used for parameter tuning.



To divide MSMARCO 2019 into two evaluation sets, we split the dataset by query IDs. Out of the 200 available queries, the first 140 are assigned to the primary evaluation set, while the remaining 60 form the secondary set. Splitting the qrels is straightforward: for each entry, we simply check the query ID and assign the relevance judgment to the corresponding evaluation subset.


Query example:    1030303 who is aziz hashim  (topicid, text)

In [ ]:
# Filenames for queries and qrels
query_file = "msmarco-test2019-queries.tsv"
qrel_file = "2019qrels-pass.txt"

# Output filenames for the two evaluation splits
eval_filename_1 = "eval-set-1.tsv"
eval_filename_2 = "eval-set-2.tsv"
eval_qrels_filename_1 = "eval-qrels-1.txt"
eval_qrels_filename_2 = "eval-qrels-2.txt"

# Sets to store query IDs belonging to each evaluation split
query_ids_eval_1 = set()
query_ids_eval_2 = set()

# Ratio for splitting queries (70% in set 1, 30% in set 2)
eval_split = 0.7

In [ ]:
# -------------------------------
# Step 1: Read queries and split them
# -------------------------------
with open(query_file, "r") as f:
    lines = f.readlines()              # Read all query lines
    num_lines = len(lines)             # Total number of queries
    split_index = int(num_lines * eval_split)  # Index at which to split

# Assign query IDs to the appropriate sets
for line in lines[:split_index]:
    query_id = line.split("\t")[0]     # Extract query ID (first column)
    query_ids_eval_1.add(query_id)

for line in lines[split_index:]:
    query_id = line.split("\t")[0]
    query_ids_eval_2.add(query_id)

# Save the split queries into separate files
with open(eval_filename_1, "w") as f:
    f.writelines(lines[:split_index])

with open(eval_filename_2, "w") as f:
    f.writelines(lines[split_index:])

In [ ]:
# -------------------------------
# Step 2: Read qrels and split them according to query IDs
# -------------------------------
with open(qrel_file, "r") as f:
    qrel_lines = f.readlines()

# Write qrels into the corresponding evaluation files
with open(eval_qrels_filename_1, "w") as f1, open(eval_qrels_filename_2, "w") as f2:
    for line in qrel_lines:
        query_id = line.split()[0]     # Extract query ID (first column in qrels)
        if query_id in query_ids_eval_1:
            f1.write(line)
        elif query_id in query_ids_eval_2:
            f2.write(line)
        else:
            # This should not happen if queries and qrels are consistent
            print(f"Warning: Query ID {query_id} not found in either evaluation set")

### BM25 Tuning

I'm conducting a grid search over various combinations of the parameters (b,k). For each configuration, I compute performance scores using TREC evaluation metrics. To identify the best parameter pair, I apply a statistical test to determine whether the observed differences in performance are statistically significant.

#### Helper Functions


The two functions, extract_queries() and write_run_file(), together form a crucial part of the retrieval and evaluation pipeline in an information retrieval (IR) system — specifically handling the preparation of input queries and the generation of output run files for evaluation.

The extract_queries() function is responsible for reading a tab-separated values (TSV) file that contains pairs of query identifiers and their corresponding query texts. Each line in the file is expected to follow the format query_id<TAB>query_text. The function parses this file line by line, splitting each entry into a tuple of (query_id, query) and appending it to a list. It also includes a safeguard to skip malformed lines that don’t match the expected format, ensuring robustness against minor formatting errors. The resulting list of tuples provides a structured and standardized way to store multiple queries, which can then be processed in bulk by the retrieval system.

The write_run_file() function, on the other hand, takes these extracted queries and runs them through the retrieval engine using a provided inverted index. For each query, it calls the query_process() function to obtain a ranked list of documents, represented as pairs of (score, docid) values. These results are then formatted according to the TREC standard, a widely used evaluation format in IR research, which includes columns for the query ID, document number, rank position, score, and run name (an identifier for the retrieval experiment). Before writing to the file, document identifiers are cleaned and standardized by removing prefixes (e.g., “DOC”) to match the TREC conventions. The final run file, saved to the specified output path, serves as an input for evaluation tools like trec_eval or ir_measures, enabling performance comparison across different retrieval models. Together, these functions automate the process of preparing queries, retrieving ranked results, and producing evaluation-ready outputs — ensuring reproducibility, consistency, and compatibility with standard IR benchmarking frameworks.

In [ ]:
from typing import List, Tuple

def extract_queries(
    filename: str
) -> List[Tuple[str, str]]:
    """
    Description:
        Reads a TSV file containing query IDs and query texts,
        and returns them as a list of (query_id, query) tuples.

    Arguments:
        filename (str): Path to the TSV file.
          Each line should contain 'query_id<TAB>query_text'.

    Returns:
        List[Tuple[str, str]]: A list where each element is a tuple
          containing (query_id, query_text).
    """
    queries: List[Tuple[str, str]] = []

    with open(filename, "r", encoding="utf-8") as f:
        for line in f:

            # Skipping malformed lines
            try:
                query_id, query = line.strip().split("\t")
            except ValueError:
                continue

            # Appending query_id and query text
            queries.append((query_id, query))

    return queries

In [ ]:
def write_run_file(
    queries: list[tuple[str, str]],
    index: InvertedIndex,
    output_path: str,
    run_name: str,
):
    """
    Description:
        Generates a TREC-formatted run file from a set of queries and the inverted index.

    Arguments:
        queries (Dict[int, str]): Mapping from query IDs to query strings.
        index (InvertedIndex): The inverted index used for retrieval.
        output_path (str): Path to save the output run file.
        run_name (str): Identifier for the run (appears in the last column).
    """

    with open(output_path, "w") as f:
        for qid, query in queries:

            # List of scores and docids ordered by score using DAAT
            results: list[tuple[np.float64, np.int32]] = query_process(query, index)
            # TODO: problema: il tipo np.float64 / int32 non è quello che ritora il query_process. E altra cosa, prima venivano definite float int immagino

            # Iterating over results, converting docid into docno while
            # removing DOC and storing the line in the run file
            for rank, (score, docid) in enumerate(results, start=1):
                docno: str = doc_index[docid - 1][0]  #  NOTE: -1 since docid starts from 1
                docno: str = docno.replace("DOC", "")
                f.write(f"{qid} Q0 {docno} {rank} {score:.6f} {run_name}\n")

    print(f"Run file saved to: {output_path}")

#### Creating Run Files

This script automates the generation of TREC-formatted run files for a series of BM25 retrieval configurations. It begins by specifying the input query file, containing query IDs and text and associates them with unique suffixes to distinguish the resulting run files. A grid of BM25 parameter combinations is defined using different values of b and k1, and for each pair, a corresponding BM25Scorer object is created. These scorers are paired with descriptive run names that encode their parameter settings. For each query file, the script extracts the queries and iterates over all scoring functions. It initializes an InvertedIndex using the current scorer and document statistics, then generates a run file by applying the scorer to the queries. The output files are named using the run identifier and the appropriate suffix, ensuring clarity and traceability across different configurations and query sets. This setup enables systematic evaluation of retrieval performance across a range of BM25 parameter settings.

In [ ]:
# ---------------------- Query Files ---------------------- #
# Path to MS MARCO 2019 extracted evaluation set
# File contains query IDs and query text in TSV format.
queries_file = "eval-set-2.tsv"

# ---------------------- Scoring Functions ---------------------- #
# List of scorer objects that implement different retrieval models.
# Each scorer defines how to compute document scores during retrieval.
b_l = [0.65, 0.75, 0.85]
k1_l = [1, 1.2, 1.4]
scorers = [
    BM25Scorer(k1=k1, b=b)
    for k1 in k1_l
    for b in b_l
]

# ---------------------- Run Names ---------------------- #
# List of run identifiers corresponding to the scorers above.
# These names appear in the run file and are used by evaluation tools.
run_names = [
    "bm25_run_b={}_k1={}".format(b, k1)
    for k1 in k1_l
    for b in b_l
]

In [ ]:
# Extract queries from the current file
# queries: List of tuples (query_id, query_text)
queries = extract_queries(queries_file)

# Iterate over each scoring function and its run name
for scorer, run_name in zip(scorers, run_names):

    # Initialize the inverted index with the current scoring function
    # This allows different retrieval models (e.g., TF, TF-IDF, BM25)
    inv_ind = InvertedIndex(
        lexicon,       # term -> (termid, ...) mapping   # TODO exact pls
        inv_d,         # list of numpy arrays: doc IDs per term
        inv_f,         # list of numpy arrays: term frequencies per term
        doc_index,     # document statistics
        stats,         # global statistics (e.g., total # of documents)   # TODO exact pls
        scorer=scorer  # scoring function to use for this run
    )

    # Generate and write a TREC-formatted run file for the current query set
    # The filename combines the run name and the run suffix
    write_run_file(
        queries,              # queries to process
        inv_ind,              # inverted index with current scoring function
        run_name,             # output filename for this run
        run_name              # run identifier (appears in run file)
    )

daat (1655.801 ms)
daat (1421.533 ms)
daat (7185.995 ms)
daat (18658.461 ms)
daat (3576.058 ms)
daat (4631.403 ms)
daat (11786.030 ms)
daat (3661.095 ms)
daat (7459.922 ms)
daat (9209.614 ms)
daat (7090.973 ms)
daat (15366.650 ms)
daat (4601.224 ms)
daat (7223.803 ms)
daat (4583.142 ms)
daat (26391.746 ms)
daat (5347.172 ms)
daat (349.129 ms)
daat (5274.575 ms)
daat (1.253 ms)
daat (4376.082 ms)
daat (26702.857 ms)
daat (2941.654 ms)
daat (3177.396 ms)
daat (3692.624 ms)
daat (1231.449 ms)
daat (3056.164 ms)
daat (5607.698 ms)
daat (3196.943 ms)
daat (5174.020 ms)
daat (22869.164 ms)
daat (24605.258 ms)
daat (4899.773 ms)
daat (7667.347 ms)
daat (1490.896 ms)
daat (9001.701 ms)
daat (12526.452 ms)
daat (64460.077 ms)
daat (13249.014 ms)
daat (2259.607 ms)
daat (3921.624 ms)
daat (8711.583 ms)
daat (4394.598 ms)
daat (189.254 ms)
daat (13919.914 ms)
daat (0.635 ms)
daat (3305.546 ms)
daat (311.719 ms)
daat (8923.694 ms)
daat (3884.089 ms)
daat (6585.105 ms)
daat (22232.395 ms)
daat (588

#### Configuring Score Evaluation

I'm initializing key variables to streamline the evaluation of performance scores, ensuring the process runs efficiently and consistently.
This code sets up the evaluation framework for a series of BM25 retrieval runs, each defined by a unique combination of the parameters b and k1. It constructs a list of method identifiers based on these parameter pairs, appending a consistent suffix to match the naming convention of the corresponding run files. The evaluation focuses on three standard information retrieval metrics: Average Precision (AP), normalized Discounted Cumulative Gain at rank 10 (nDCG@10), and Mean Reciprocal Rank at rank 10 (MRR@10). Relevance judgments are loaded from a TREC-formatted qrels file, which serves as the ground truth for scoring. For each method, the corresponding run file is read and parsed using ir_measures, and the resulting run data is stored in a list for subsequent evaluation. This setup ensures that all parameter configurations are systematically prepared for performance comparison.

In [ ]:
import ir_measures
from ir_measures import * # import natural measure name
from scipy.stats import ttest_rel

# List of retrieval/scoring methods
# TODO praticamente una ridefinizione di run_names?
methods = [
    f"bm25_run_b={b}_k1={k1}"
    for k1 in k1_l
    for b in b_l
]

# Metrics to evaluate
metrics = [
    ("AP", AP),
    ("nDCG@10", nDCG@10),
    ("MRR@10", MRR@10)
]

In [ ]:
###### Step 1: Load relevance judgments (qrels)

# qrels_file contains the ground-truth relevance labels for queries
qrels_file = 'eval-qrels-2.txt'

# Read the qrels file using ir_measures utility
# Each entry represents a (query_id, doc_id, relevance) tuple
qrel = list(ir_measures.read_trec_qrels(qrels_file))


###### Step 2: Load runs for each retrieval method

# List of runs
runs = []

# 'methods' is the list of run filenames,
# each corresponding to the output of a retrieval method
for method in methods:

    # The run file for this method
    run_file = method  # TODO WTF? perchè ridefinirlo

    # Read the run file using ir_measures utility
    # Each entry represents a (query_id, doc_id, score) tuple
    run_data = list(ir_measures.read_trec_run(run_file))

    # Append the run data to the list of runs
    runs.append(run_data)

#### Computing Scores

This code segment constructs a structured evaluation of retrieval performance across multiple methods and metrics. For each metric defined in the metrics list, such as Average Precision or nDCG@10, it initializes a table where each row corresponds to a retrieval method and each column holds the per-query scores for that method. Using the ir_measures library, it computes the metric values for each query in the run associated with a given method. These per-query scores are stored as tuples and deep-copied to ensure data integrity and prevent unintended modifications. Once all methods have been evaluated for a particular metric, the resulting table is appended to the scores list, which ultimately holds a complete set of evaluation results for all metrics across all retrieval configurations. This setup enables systematic comparison and statistical analysis of retrieval performance.

In [ ]:
from copy import deepcopy

# This will store per-metric tables of per-query scores
scores = []

for metric_name, metric_func in metrics:

    # Table: rows = methods (BM25 combinations), columns = per-query scores
    table = []

    for method_idx, method in enumerate(methods):
        run_data = runs[method_idx]

        # Compute the metric for each query in the run
        per_query_scores = tuple(
            m.value for m in ir_measures.iter_calc([metric_func], qrel, run_data)
        )

        # Append a deep copy to avoid accidental modifications
        table.append(deepcopy(per_query_scores))  # TODO WTF? qui sicuro non serve

    # Append the table for this metric to the overall scores list
    scores.append(deepcopy(table)) # TODO pure qua non serve immagino

#### Statistical Tests

In [ ]:
from statsmodels.stats.multitest import multipletests
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

def p_values_correction(
    metric_idx: int,
    method: str
):
    """
    Performs pairwise t-tests between scoring methods for a given metric,
    applies multiple testing correction, and returns a DataFrame of signed corrected p-values.

    Parameters:
        metric_idx (int): Index of the metric to evaluate.
        method (str): Correction method to use ('bonferroni', 'holm', 'fdr_bh', etc.).

    Returns:
        pd.DataFrame: Matrix of corrected p-values with direction indicators.
    """
    # TODO questa si ritrova i methods dal global senza renderlo esplicito, not troppo carino, anche scores

    # Initialize p-value matrix
    p_values = pd.DataFrame(index=methods, columns=methods, dtype=object)  # NOTE: row / columns labels are methods names (BM25 combinations)

    # Collect raw p-values from pairwise comparisons
    raw_pvals = []
    comparisons = []

    # Make pair-wise tests
    for row_method in methods:
        for col_method in methods:

            # Skipping diagonal because it's the same method
            if row_method == col_method:
                p_values.loc[row_method, col_method] = np.nan
                continue

            # Get per-query scores for both methods, and the metric_idx metric
            scores_a = scores[metric_idx][methods.index(row_method)]
            scores_b = scores[metric_idx][methods.index(col_method)]

            # Run paired t-test
            t_stat, p_val = ttest_rel(scores_a, scores_b)
            raw_pvals.append(p_val)
            comparisons.append((row_method, col_method, t_stat >= 0))  # NOTE: Store direction (True = the row method tends to perform better or equal than the column method)

    # Apply multiple testing correction
    _, corrected_pvals, _, _ = multipletests(raw_pvals, method=method)

    # Fill in the corrected p-values with direction indicators
    for i, (row_method, col_method, is_greater) in enumerate(comparisons):
        sign = '+' if is_greater else '-'
        p_values.loc[row_method, col_method] = f"({sign}){corrected_pvals[i]:.6f}"

    return p_values

This segment of code applies the Benjamini-Hochberg (BH) correction to the p-values obtained from pairwise statistical comparisons across different retrieval methods for three evaluation metrics: MAP, nDCG@10, and MRR@10. For each metric, the p_values_correction function is called with the corresponding index and the "fdr_bh" method to adjust for multiple comparisons while controlling the false discovery rate. The corrected p-values are stored in separate DataFrames—p_values_MAP, p_values_nDCG, and p_values_MRR—each representing the significance of performance differences between methods. These tables are then converted to markdown format for easy visualization or reporting, enabling a clear and interpretable summary of which method comparisons remain statistically significant after correction.

In [ ]:
p_values_MAP = p_values_correction(0, "fdr_bh")
p_values_MAP.to_markdown()


'|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |\n|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|\n| bm25_run_b=0.65_k1=1   | nan                    | (+)0.122513            | (+)0.122513            | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              |\n| bm25_run_b=0.75_k1=1   | (-)0.122513            | nan                    | (+)0.122513            | (-)0.933471              | (+)0.122513              | (+)0.122513              | (+)0.261466          

In [ ]:
p_values_nDCG = p_values_correction(1, "fdr_bh")
p_values_nDCG.to_markdown()


'|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |\n|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|\n| bm25_run_b=0.65_k1=1   | nan                    | (+)0.249449            | (+)0.095179            | (+)0.140184              | (+)0.043462              | (+)0.043462              | (+)0.140184              | (+)0.043462              | (+)0.043462              |\n| bm25_run_b=0.75_k1=1   | (-)0.249449            | nan                    | (+)0.283433            | (-)0.709305              | (+)0.108797              | (+)0.108797              | (-)0.894233          

In [ ]:
p_values_MRR = p_values_correction(2, "fdr_bh")
p_values_MRR.to_markdown()

'|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |\n|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|\n| bm25_run_b=0.65_k1=1   | nan                    | (+)nan                 | (-)nan                 | (-)nan                   | (+)nan                   | (+)nan                   | (-)nan                   | (+)nan                   | (+)nan                   |\n| bm25_run_b=0.75_k1=1   | (-)nan                 | nan                    | (-)nan                 | (-)nan                   | (+)nan                   | (+)nan                   | (-)nan               

#### MAP

|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |
|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|
| bm25_run_b=0.65_k1=1   | nan                    | (+)0.122513            | (+)0.122513            | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              |
| bm25_run_b=0.75_k1=1   | (-)0.122513            | nan                    | (+)0.122513            | (-)0.933471              | (+)0.122513              | (+)0.122513              | (+)0.261466              | (+)0.122513              | (+)0.122513              |
| bm25_run_b=0.85_k1=1   | (-)0.122513            | (-)0.122513            | nan                    | (-)0.216359              | (-)0.933471              | (+)0.209427              | (-)0.943246              | (+)0.365486              | (+)0.122513              |
| bm25_run_b=0.65_k1=1.2 | (-)0.122513            | (+)0.933471            | (+)0.216359            | nan                      | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              | (+)0.122513              |
| bm25_run_b=0.75_k1=1.2 | (-)0.122513            | (-)0.122513            | (+)0.933471            | (-)0.122513              | nan                      | (+)0.122513              | (+)0.943246              | (+)0.122513              | (+)0.122513              |
| bm25_run_b=0.85_k1=1.2 | (-)0.122513            | (-)0.122513            | (-)0.209427            | (-)0.122513              | (-)0.122513              | nan                      | (-)0.231197              | (+)0.873522              | (+)0.122513              |
| bm25_run_b=0.65_k1=1.4 | (-)0.122513            | (-)0.261466            | (+)0.943246            | (-)0.122513              | (-)0.943246              | (+)0.231197              | nan                      | (+)0.122513              | (+)0.122513              |
| bm25_run_b=0.75_k1=1.4 | (-)0.122513            | (-)0.122513            | (-)0.365486            | (-)0.122513              | (-)0.122513              | (-)0.873522              | (-)0.122513              | nan                      | (+)0.122513              |
| bm25_run_b=0.85_k1=1.4 | (-)0.122513            | (-)0.122513            | (-)0.122513            | (-)0.122513              | (-)0.122513              | (-)0.122513              | (-)0.122513              | (-)0.122513              | nan                      |

#### nDCG

|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |
|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|
| bm25_run_b=0.65_k1=1   | nan                    | (+)0.249449            | (+)0.095179            | (+)0.140184              | (+)0.043462              | (+)0.043462              | (+)0.140184              | (+)0.043462              | (+)0.043462              |
| bm25_run_b=0.75_k1=1   | (-)0.249449            | nan                    | (+)0.283433            | (-)0.709305              | (+)0.108797              | (+)0.108797              | (-)0.894233              | (+)0.108797              | (+)0.108797              |
| bm25_run_b=0.85_k1=1   | (-)0.095179            | (-)0.283433            | nan                    | (-)0.169155              | (+)0.817367              | (+)0.189504              | (-)0.266548              | (+)0.800955              | (+)0.155886              |
| bm25_run_b=0.65_k1=1.2 | (-)0.140184            | (+)0.709305            | (+)0.169155            | nan                      | (+)0.108797              | (+)0.095179              | (+)0.643535              | (+)0.108797              | (+)0.095041              |
| bm25_run_b=0.75_k1=1.2 | (-)0.043462            | (-)0.108797            | (-)0.817367            | (-)0.108797              | nan                      | (+)0.653847              | (-)0.108797              | (+)0.870052              | (+)0.568091              |
| bm25_run_b=0.85_k1=1.2 | (-)0.043462            | (-)0.108797            | (-)0.189504            | (-)0.095179              | (-)0.653847              | nan                      | (-)0.081059              | (-)0.659967              | (+)0.220320              |
| bm25_run_b=0.65_k1=1.4 | (-)0.140184            | (+)0.894233            | (+)0.266548            | (-)0.643535              | (+)0.108797              | (+)0.081059              | nan                      | (+)0.108797              | (+)0.052287              |
| bm25_run_b=0.75_k1=1.4 | (-)0.043462            | (-)0.108797            | (-)0.800955            | (-)0.108797              | (-)0.870052              | (+)0.659967              | (-)0.108797              | nan                      | (+)0.582671              |
| bm25_run_b=0.85_k1=1.4 | (-)0.043462            | (-)0.108797            | (-)0.155886            | (-)0.095041              | (-)0.568091              | (-)0.220320              | (-)0.052287              | (-)0.582671              | nan                      |

#### MRR

Although it may seem unusual to encounter many NaN values, this outcome is not unexpected.
In most cases, the highest score among relevant documents remains largely unaffected across different combinations.
As a result, the position of the first relevant document does not change, which explains why NaN values appear in the evaluation.


|                        | bm25_run_b=0.65_k1=1   | bm25_run_b=0.75_k1=1   | bm25_run_b=0.85_k1=1   | bm25_run_b=0.65_k1=1.2   | bm25_run_b=0.75_k1=1.2   | bm25_run_b=0.85_k1=1.2   | bm25_run_b=0.65_k1=1.4   | bm25_run_b=0.75_k1=1.4   | bm25_run_b=0.85_k1=1.4   |
|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|:-------------------------|
| bm25_run_b=0.65_k1=1   | nan                    | (+)nan                 | (-)nan                 | (-)nan                   | (+)nan                   | (+)nan                   | (-)nan                   | (+)nan                   | (+)nan                   |
| bm25_run_b=0.75_k1=1   | (-)nan                 | nan                    | (-)nan                 | (-)nan                   | (+)nan                   | (+)nan                   | (-)nan                   | (+)nan                   | (+)nan                   |
| bm25_run_b=0.85_k1=1   | (-)nan                 | (+)nan                 | nan                    | (-)nan                   | (+)nan                   | (+)nan                   | (-)nan                   | (+)nan                   | (+)nan                   |
| bm25_run_b=0.65_k1=1.2 | (-)nan                 | (+)nan                 | (-)nan                 | nan                      | (+)nan                   | (+)nan                   | (-)nan                   | (+)nan                   | (+)nan                   |
| bm25_run_b=0.75_k1=1.2 | (-)nan                 | (-)nan                 | (-)nan                 | (-)nan                   | nan                      | (-)nan                   | (-)nan                   | (-)nan                   | (-)nan                   |
| bm25_run_b=0.85_k1=1.2 | (-)nan                 | (-)nan                 | (-)nan                 | (-)nan                   | (+)nan                   | nan                      | (-)nan                   | (+)nan                   | (-)nan                   |
| bm25_run_b=0.65_k1=1.4 | (+)nan                 | (+)nan                 | (+)nan                 | (+)nan                   | (+)nan                   | (+)nan                   | nan                      | (+)nan                   | (+)nan                   |
| bm25_run_b=0.75_k1=1.4 | (-)nan                 | (-)nan                 | (-)nan                 | (-)nan                   | (-)nan                   | (-)nan                   | (-)nan                   | nan                      | (-)nan                   |
| bm25_run_b=0.85_k1=1.4 | (-)nan                 | (-)nan                 | (-)nan                 | (-)nan                   | (+)nan                   | (-)nan                   | (-)nan                   | (+)nan                   | nan                      |

The best model is: b=0.65, k1=1 according both nDCG and MAP

In [ ]:
# TODO: qui bisogna giustificarla meglio, non sempre è statisticamente significativa la differenza (anzi quasi mai)

### Compute Run Files

#### Helper Functions


The two functions, extract_queries() and write_run_file(), together form a crucial part of the retrieval and evaluation pipeline in an information retrieval (IR) system, specifically handling the preparation of input queries and the generation of output run files for evaluation.

The extract_queries() function is responsible for reading a tab-separated values (TSV) file that contains pairs of query identifiers and their corresponding query texts. Each line in the file is expected to follow the format query_id<TAB>query_text. The function parses this file line by line, splitting each entry into a tuple of (query_id, query) and appending it to a list. It also includes a safeguard to skip malformed lines that don't match the expected format, ensuring robustness against minor formatting errors. The resulting list of tuples provides a structured and standardized way to store multiple queries, which can then be processed in bulk by the retrieval system.

The write_run_file() function, on the other hand, takes these extracted queries and runs them through the retrieval engine using a provided inverted index. For each query, it calls the query_process() function to obtain a ranked list of documents, represented as pairs of (score, docid) values. These results are then formatted according to the TREC standard, a widely used evaluation format in IR research, which includes columns for the query ID, document number, rank position, score, and run name (an identifier for the retrieval experiment). Before writing to the file, document identifiers are cleaned and standardized by removing prefixes (e.g., “DOC”) to match the TREC conventions. The final run file, saved to the specified output path, serves as an input for evaluation tools like trec_eval or ir_measures, enabling performance comparison across different retrieval models. Together, these functions automate the process of preparing queries, retrieving ranked results, and producing evaluation-ready outputs — ensuring reproducibility, consistency, and compatibility with standard IR benchmarking frameworks.

In [ ]:
# TODO: questi sono metodi sono identici a prima? in caso toglierei, no?

In [ ]:
def extract_queries(
    filename: str
) -> List[Tuple[str, str]]:
    """
    Description:
        Reads a TSV file containing query IDs and query texts,
        and returns them as a list of (query_id, query) tuples.

    Arguments:
        filename (str): Path to the TSV file.
          Each line should contain 'query_id<TAB>query_text'.

    Returns:
        List[Tuple[str, str]]: A list where each element is a tuple
          containing (query_id, query_text).
    """
    queries: List[Tuple[str, str]] = []

    with open(filename, "r", encoding="utf-8") as f:
        for line in f:

            # Skipping malformed lines
            try:
                query_id, query = line.strip().split("\t")
            except ValueError:
                continue

            # Appending query_id and query text
            queries.append((query_id, query))

    return queries

In [ ]:
def write_run_file(
    queries: list[tuple[str, str]],
    index: InvertedIndex,
    output_path: str,
    run_name: str = "my_run",
):
    """
    Description:
        Generates a TREC-formatted run file from a set of queries and the inverted index.

    Arguments:
        queries (Dict[int, str]): Mapping from query IDs to query strings.
        index (InvertedIndex): The inverted index used for retrieval.
        output_path (str): Path to save the output run file.
        run_name (str): Identifier for the run (appears in the last column).
    """

    with open(output_path, "w") as f:
        for qid, query in queries:

            # List of scores and docids ordered by score
            results: list[tuple[np.float64, np.int32]] = query_process(query, index)

            # Iterating over results, converting docid into docno while
            # removing DOC and storing the line in the run file
            for rank, (score, docid) in enumerate(results, start=1):
                docno: str = doc_index[docid - 1][0]
                docno: str = docno.replace("DOC", "")
                f.write(f"{qid} Q0 {docno} {rank} {score:.6f} {run_name}\n")

    print(f"Run file saved to: {output_path}")

#### Creating Run Files

This section of the code defines and executes the experimental pipeline used to evaluate multiple retrieval models across different query datasets a common setup in information retrieval research. It is organized into four main parts: query file configuration, run suffix definition, scoring model specification, and run generation.

First, the code defines query_files, a path to query files from the MS MARCO test sets for 2019 first evaluation set. Each file consists of query IDs and their corresponding text in TSV format, enabling the system to process queries from different years independently.

Next, the code defines a list of scoring function objects under scorers, representing various retrieval models ranging from simple ones like TF (Term Frequency) and IDF (Inverse Document Frequency) to more sophisticated ones such as TF-IDF and the BM series (BM11, BM15, BM25). Each scorer encapsulates a mathematical formula used to assign relevance scores to documents based on their relationship with the query terms. The run_names list mirrors the scorers, assigning a readable identifier (e.g., tf_run, bm25_run) that appears in the output run files and evaluation logs.

Finally, the main loop iterates through each combination of query file and run suffix, calling the helper function extract_queries() to read and parse all queries. For every scorer in the list, a new InvertedIndex object is instantiated this allows each scoring function to be applied independently while keeping the underlying data structures (lexicon, posting lists, document stats) the same. The results are then passed to the write_run_file() function, which retrieves the top-ranked documents for each query and writes them in TREC format to an output file named according to the scoring model and year (e.g., bm25_run_2019).

In essence, this code automates a systematic retrieval experiment where multiple scoring models are evaluated on multiple query datasets. By separating configurations (queries, scorers, and filenames), it ensures modularity, reproducibility, and scalability allowing researchers to efficiently compare retrieval performance across different models and datasets in a controlled and well-documented framework.

In [ ]:
# ---------------------- Query Files ---------------------- #
# Path to MS MARCO first evaluation set query file.
# File contains query IDs and query text in TSV format.
queries_file = "eval-set-1.tsv"


# ---------------------- Scoring Functions ---------------------- #
# List of scorer objects that implement different retrieval models.
# Each scorer defines how to compute document scores during retrieval.
scorers = [
    TFScorer(),                   # simple term frequency scoring
    IDFScorer(),                  # inverse document frequency scoring
    TFIDFScorer(),                # combined TF-IDF scoring
    BM11Scorer(),                 # BM11 variant
    BM15Scorer(),                 # BM15 variant
    BM25Scorer(k1=1.0, b=0.65)    # standard BM25 scoring
]

# ---------------------- Run Names ---------------------- #
# List of run identifiers corresponding to the scorers above.
# These names appear in the run file and are used by evaluation tools.
run_names = [
    "tf_run",     # run using TF scoring
    "idf_run",    # run using IDF scoring
    "tfidf_run",  # run using TF-IDF scoring
    "bm11_run",   # run using BM11 scoring
    "bm15_run",   # run using BM15 scoring
    "bm25_run"    # run using BM25 scoring
]

In [ ]:
# Extract queries from the current file
# queries: List of tuples (query_id, query_text)
queries = extract_queries(queries_file)

# Iterate over each scoring function and its run name
for scorer, run_name in zip(scorers, run_names):

    # Initialize the inverted index with the current scoring function
    # This allows different retrieval models (e.g., TF, TF-IDF, BM25)
    inv_ind = InvertedIndex(
        lexicon,       # term -> (termid, ...) mapping
        inv_d,         # list of numpy arrays: doc IDs per term
        inv_f,         # list of numpy arrays: term frequencies per term
        doc_index,     # document statistics
        stats,         # global statistics (e.g., total # of documents)
        scorer=scorer  # scoring function to use for this run
    )

    # Generate and write a TREC-formatted run file for the current query set
    # The filename combines the run name and the run suffix
    write_run_file(
        queries,              # queries to process
        inv_ind,              # inverted index with current scoring function
        run_name,             # output filename for this run
        run_name              # run identifier (appears in run file)
    )

daat (3369.512 ms)
daat (2413.840 ms)
daat (0.427 ms)
daat (305.093 ms)
daat (9204.375 ms)
daat (3810.138 ms)
daat (2050.464 ms)
daat (7559.655 ms)
daat (1640.114 ms)
daat (1007.523 ms)
daat (4176.325 ms)
daat (7079.831 ms)
daat (632.310 ms)
daat (10328.781 ms)
daat (5875.533 ms)
daat (6849.895 ms)
daat (4421.413 ms)
daat (28512.858 ms)
daat (1766.466 ms)
daat (516.128 ms)
daat (1155.939 ms)
daat (3631.293 ms)
daat (382.923 ms)
daat (11000.242 ms)
daat (24289.334 ms)
daat (2051.642 ms)
daat (5133.271 ms)
daat (24727.500 ms)
daat (979.397 ms)
daat (10214.860 ms)
daat (2285.490 ms)
daat (1798.054 ms)
daat (332.321 ms)
daat (10536.668 ms)
daat (2293.660 ms)
daat (1661.789 ms)
daat (6393.504 ms)
daat (8357.380 ms)
daat (1432.653 ms)
daat (3025.335 ms)
daat (5967.372 ms)
daat (5656.662 ms)
daat (10190.184 ms)
daat (6652.640 ms)
daat (5685.286 ms)
daat (21590.929 ms)
daat (3258.810 ms)
daat (5142.411 ms)
daat (11924.356 ms)
daat (1965.310 ms)
daat (1408.371 ms)
daat (100.683 ms)
daat (2627.6

### Compute Scores

This section of the code defines the evaluation phase of the information retrieval experiment where multiple scoring methods are systematically compared using standard performance metrics. It begins by listing the retrieval models (methods) being evaluated, such as TF, IDF, TF-IDF, and the BM family variants (BM11, BM15, BM25). Each method corresponds to a previously generated run file, named according to a consistent naming convention.

The metrics list specifies the evaluation measures used to assess retrieval effectiveness in this case, Average Precision (AP), Normalized Discounted Cumulative Gain at rank 10 (nDCG@10), and Mean Reciprocal Rank at rank 10 (MRR@10). These metrics capture different aspects of retrieval performance: AP measures overall precision across recall levels, nDCG focuses on ranking quality and relevance distribution, while MRR emphasizes how early the first relevant document appears in the ranking.

The code then loads the relevance judgments (qrels_file), which provide ground-truth information about which documents are relevant for each query. Using ir_measures, each run file (containing the ranked retrieval results for a method) is read and stored in the runs list. This ensures that for every retrieval method, the corresponding set of query-document scores is available for evaluation.

In the main loop, the code iterates over each metric defined earlier. For each metric, it initializes a table that will hold the per-query scores of all methods. Then, for every method in methods, it retrieves the corresponding run data and computes the metric's value for each query using ir_measures.iter_calc(). The results a sequence of scores (one per query) are stored in table. A deep copy of the scores is appended to avoid unintended data overwriting across iterations.

Finally, each metric's full table (containing all methods' per-query results) is stored in the scores list. This structure effectively creates a 3D matrix of evaluation results: one dimension for metrics, one for retrieval methods, and one for queries. This organized data structure is essential for later statistical analysis (e.g., paired t-tests, p-value computation) to compare the significance of performance differences between scoring methods across queries.

#### Configuration

In [ ]:
import ir_measures
from ir_measures import * # import natural measure name
from scipy.stats import ttest_rel


# List of retrieval/scoring methods
methods = ["tf", "idf", "tfidf", "bm11", "bm15", "bm25"]

# Metrics to evaluate
metrics = [
    ("AP", AP),
    ("nDCG@10", nDCG@10),
    ("MRR@10", MRR@10)
]

#### Loading Qrels and Runs

In [ ]:
##### Step 1: Load relevance judgments (qrels)

# qrels_file contains the ground-truth relevance labels for queries
qrels_file = 'eval-qrels-1.txt'

# Read the qrels file using ir_measures utility
# Each entry represents a (query_id, doc_id, relevance) tuple
qrel = list(ir_measures.read_trec_qrels(qrels_file))



##### Step 2: Load runs for each retrieval method

# List of runs
runs = []

# 'methods' is the list of run filenames,
# each corresponding to the output of a retrieval method
for method in methods:

    # The run file for this method
    run_file = method + "_run"

    # Read the run file using ir_measures utility
    # Each entry represents a (query_id, doc_id, score) tuple
    run_data = list(ir_measures.read_trec_run(run_file))

    # Append the run data to the list of runs
    runs.append(run_data)

#### Scores

In [ ]:
# TODO: penso codice uguale a sopra, no? fare metodo

In [ ]:

from copy import deepcopy

# This will store per-metric tables of per-query scores
scores = []

for metric_name, metric_func in metrics:

    # Table: rows = methods, columns = per-query scores
    table = []

    for method_idx, method in enumerate(methods):
        run_data = runs[method_idx]

        # Compute the metric for each query in the run
        per_query_scores = tuple(
            m.value for m in ir_measures.iter_calc([metric_func], qrel, run_data)
        )

        # Append a deep copy to avoid accidental modifications
        table.append(deepcopy(per_query_scores))

    # Append the table for this metric to the overall scores list
    scores.append(deepcopy(table))

### Statistical Tests

This section of the code performs a pairwise statistical significance analysis between different retrieval methods using paired t-tests and summarizes the results in a structured table of p-values. The goal is to determine whether the differences in retrieval performance between methods are statistically meaningful, based on the per-query scores obtained for a specific evaluation metric (e.g., MRR@10).

The process begins by creating a pandas DataFrame called p_values, where both the rows and columns correspond to the list of retrieval methods (e.g., TF, IDF, BM25). Each cell in this matrix will later contain the p-value resulting from a statistical comparison between the two methods in the corresponding row and column. A dtype=object is used to allow storage of string-formatted results that combine both the p-value and the performance direction.

The code then iterates over all possible pairs of methods using nested loops. For each pair, it retrieves the per-query scores of the two methods (a and b) from the scores data structure. The diagonal entries where a method is compared to itself are set to NaN, since self-comparison is meaningless. The key statistical operation here is the paired t-test (ttest_rel(a, b)), which evaluates whether the mean difference between the two methods' per-query performances is statistically significant. The test assumes that the queries are paired, meaning each query serves as a common evaluation unit across all methods.

The t-statistic returned by the test indicates the direction of the difference: a positive value means that the method in the row tends to perform better than the one in the column. The p-value reflects the probability that the observed difference occurred by chance lower values (e.g., < 0.05) suggest a statistically significant difference. The code captures both the direction and significance by storing the result as a string, such as "(+)0.013257" (indicating that the row method outperformed the column method with p=0.013257) or "(-)0.427896" (indicating worse performance).

Finally, the resulting p_values DataFrame provides a compact visual summary of statistical relationships between all pairs of methods. By converting it to Markdown (p_values.to_markdown()).

We want *p_value < 0.05*

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

# TODO: sopra c'era la funzione per fare questo!!!! se sopra aveva il correction method e basta per me questa parte si può eliminare compleamente, mi pare inutile e rindondante

# ----------------- Prepare p-value DataFrame ----------------- #
# Rows and columns represent the scoring methods
p_values = pd.DataFrame(index=methods, columns=methods, dtype=object)

# ----------------- Compute pairwise t-tests ----------------- #
# scores[metric_idx][method_idx] gives per-query scores for that method

metric_idx = 2    # Change this to 0 for AP, 1 for nDCG@10, etc.

for row_method in methods:
    for col_method in methods:
        if row_method == col_method:
            # Diagonal: comparison with itself is meaningless
            p_values.loc[row_method, col_method] = np.nan
            continue

        # Retrieve per-query scores for the two methods
        a = scores[metric_idx][methods.index(row_method)]
        b = scores[metric_idx][methods.index(col_method)]

        # Run paired t-test
        # ttest_rel returns (t-statistic, p-value)
        t_stat, p_val = ttest_rel(a, b)

        # Determine if method in row performed better than column
        # Positive t_stat -> row_method > col_method
        greater = t_stat >= 0

        # Store p-value as a string with sign indication (+/-)
        p_values.loc[row_method, col_method] = f"({'+' if greater else '-'}){p_val:.6f}"

# ----------------- Result ----------------- #
# p_values DataFrame now contains a matrix of signed p-values
# Rows = row_method, Columns = col_method
# Example: "(+)0.023456" means row_method performed better than col_method with p=0.023456
p_values.to_markdown()


'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.037434 | (-)0.006425 | (-)0.000985 | (-)0.010726 | (-)0.002332 |\n| idf   | (+)0.037434 | nan         | (+)0.815053 | (-)0.567260 | (-)0.552977 | (-)0.265345 |\n| tfidf | (+)0.006425 | (-)0.815053 | nan         | (-)0.232571 | (-)0.451618 | (-)0.224335 |\n| bm11  | (+)0.000985 | (+)0.567260 | (+)0.232571 | nan         | (+)0.951553 | (-)0.572126 |\n| bm15  | (+)0.010726 | (+)0.552977 | (+)0.451618 | (-)0.951553 | nan         | (-)0.488398 |\n| bm25  | (+)0.002332 | (+)0.265345 | (+)0.224335 | (+)0.572126 | (+)0.488398 | nan         |'

#### MAP

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (+)0.022078 | (-)0.000639 | (-)0.000001 | (-)0.000001 | (-)0.000000 |
| idf   | (-)0.022078 | nan         | (-)0.000020 | (-)0.000000 | (-)0.000000 | (-)0.000000 |
| tfidf | (+)0.000639 | (+)0.000020 | nan         | (-)0.000000 | (-)0.000012 | (-)0.000001 |
| bm11  | (+)0.000001 | (+)0.000000 | (+)0.000000 | nan         | (-)0.604252 | (-)0.014204 |
| bm15  | (+)0.000001 | (+)0.000000 | (+)0.000012 | (+)0.604252 | nan         | (-)0.000156 |
| bm25  | (+)0.000000 | (+)0.000000 | (+)0.000001 | (+)0.014204 | (+)0.000156 | nan         |

#### nDCG

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.026656 | (-)0.025302 | (-)0.000172 | (-)0.006351 | (-)0.000448 |
| idf   | (+)0.026656 | nan         | (+)0.719015 | (-)0.063093 | (-)0.576542 | (-)0.152149 |
| tfidf | (+)0.025302 | (-)0.719015 | nan         | (-)0.003755 | (-)0.279346 | (-)0.037860 |
| bm11  | (+)0.000172 | (+)0.063093 | (+)0.003755 | nan         | (+)0.146553 | (+)0.603104 |
| bm15  | (+)0.006351 | (+)0.576542 | (+)0.279346 | (-)0.146553 | nan         | (-)0.039888 |
| bm25  | (+)0.000448 | (+)0.152149 | (+)0.037860 | (-)0.603104 | (+)0.039888 | nan         |

#### MRR

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.037434 | (-)0.006425 | (-)0.000985 | (-)0.010726 | (-)0.002332 |
| idf   | (+)0.037434 | nan         | (+)0.815053 | (-)0.567260 | (-)0.552977 | (-)0.265345 |
| tfidf | (+)0.006425 | (-)0.815053 | nan         | (-)0.232571 | (-)0.451618 | (-)0.224335 |
| bm11  | (+)0.000985 | (+)0.567260 | (+)0.232571 | nan         | (+)0.951553 | (-)0.572126 |
| bm15  | (+)0.010726 | (+)0.552977 | (+)0.451618 | (-)0.951553 | nan         | (-)0.488398 |
| bm25  | (+)0.002332 | (+)0.265345 | (+)0.224335 | (+)0.572126 | (+)0.488398 | nan         |

#### Problem

When conducting multiple statistical tests simultaneously, it's essential to apply a correction method to account for the increased risk of false positives that is, incorrectly identifying a result as significant purely by chance. This phenomenon is known as the multiple comparisons problem. To mitigate it, several correction techniques are available, each with different levels of stringency.

Among the most commonly used methods are Bonferroni, Holm, and Benjamini-Hochberg (BH) corrections. The Bonferroni correction is the most conservative of the three. It strictly controls the family-wise error rate by dividing the significance threshold (e.g., 0.05) by the number of tests. While this approach effectively minimizes false positives, it can be overly stringent, especially when dealing with a large number of comparisons, potentially leading to a high rate of false negatives (missed true effects).

The Holm correction offers a more balanced alternative. It also controls the family-wise error rate but does so using a step-down procedure that is generally less conservative than Bonferroni. This makes it a good middle ground for researchers who want to maintain rigorous standards without being excessively restrictive.

On the other end of the spectrum is the Benjamini-Hochberg (BH) procedure, which controls the false discovery rate the expected proportion of false positives among the rejected hypotheses. This method is less conservative and more powerful, making it particularly well-suited for exploratory analyses or studies involving a large number of tests, such as genomics, psychology, or machine learning model comparisons.

In [ ]:
from statsmodels.stats.multitest import multipletests
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel


# TODO Già definitaaaaa

def p_values_correction(
    metric_idx: int,
    method: str
):
    """
    Performs pairwise t-tests between scoring methods for a given metric,
    applies multiple testing correction, and returns a DataFrame of signed corrected p-values.

    Parameters:
        metric_idx (int): Index of the metric to evaluate.
        method (str): Correction method to use ('bonferroni', 'holm', 'fdr_bh', etc.).

    Returns:
        pd.DataFrame: Matrix of corrected p-values with direction indicators.
    """

    # Initialize p-value matrix
    p_values = pd.DataFrame(index=methods, columns=methods, dtype=object)

    # Collect raw p-values from pairwise comparisons
    raw_pvals = []
    comparisons = []

    # Make pair-wise tests
    for row_method in methods:
        for col_method in methods:

            # Skipping diagonal because it's the same method
            if row_method == col_method:
                p_values.loc[row_method, col_method] = np.nan
                continue

            # Get per-query scores for both methods
            scores_a = scores[metric_idx][methods.index(row_method)]
            scores_b = scores[metric_idx][methods.index(col_method)]

            # Run paired t-test
            t_stat, p_val = ttest_rel(scores_a, scores_b)
            raw_pvals.append(p_val)
            comparisons.append((row_method, col_method, t_stat >= 0))  # Store direction

    # Apply multiple testing correction
    reject, corrected_pvals, _, _ = multipletests(raw_pvals, method=method)

    # Fill in the corrected p-values with direction indicators
    for i, (row_method, col_method, is_greater) in enumerate(comparisons):
        sign = '+' if is_greater else '-'
        p_values.loc[row_method, col_method] = f"({sign}){corrected_pvals[i]:.6f}"

    return p_values

### Bonferroni Correction

In [ ]:
p_values_MAP = p_values_correction(0, "bonferroni")
p_values_MAP.to_markdown()

'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (+)0.662326 | (-)0.019180 | (-)0.000018 | (-)0.000035 | (-)0.000008 |\n| idf   | (-)0.662326 | nan         | (-)0.000614 | (-)0.000005 | (-)0.000008 | (-)0.000003 |\n| tfidf | (+)0.019180 | (+)0.000614 | nan         | (-)0.000002 | (-)0.000360 | (-)0.000028 |\n| bm11  | (+)0.000018 | (+)0.000005 | (+)0.000002 | nan         | (-)1.000000 | (-)0.426132 |\n| bm15  | (+)0.000035 | (+)0.000008 | (+)0.000360 | (+)1.000000 | nan         | (-)0.004671 |\n| bm25  | (+)0.000008 | (+)0.000003 | (+)0.000028 | (+)0.426132 | (+)0.004671 | nan         |'

In [ ]:
p_values_nDCG = p_values_correction(1, "bonferroni")
p_values_nDCG.to_markdown()

'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.799677 | (-)0.759054 | (-)0.005171 | (-)0.190535 | (-)0.013443 |\n| idf   | (+)0.799677 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |\n| tfidf | (+)0.759054 | (-)1.000000 | nan         | (-)0.112659 | (-)1.000000 | (-)1.000000 |\n| bm11  | (+)0.005171 | (+)1.000000 | (+)0.112659 | nan         | (+)1.000000 | (+)1.000000 |\n| bm15  | (+)0.190535 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |\n| bm25  | (+)0.013443 | (+)1.000000 | (+)1.000000 | (-)1.000000 | (+)1.000000 | nan         |'

In [ ]:
p_values_MRR = p_values_correction(2, "bonferroni")
p_values_MRR.to_markdown()

'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)1.000000 | (-)0.192760 | (-)0.029555 | (-)0.321778 | (-)0.069963 |\n| idf   | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |\n| tfidf | (+)0.192760 | (-)1.000000 | nan         | (-)1.000000 | (-)1.000000 | (-)1.000000 |\n| bm11  | (+)0.029555 | (+)1.000000 | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 |\n| bm15  | (+)0.321778 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |\n| bm25  | (+)0.069963 | (+)1.000000 | (+)1.000000 | (+)1.000000 | (+)1.000000 | nan         |'

#### MAP

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (+)0.662326 | (-)0.019180 | (-)0.000018 | (-)0.000035 | (-)0.000008 |
| idf   | (-)0.662326 | nan         | (-)0.000614 | (-)0.000005 | (-)0.000008 | (-)0.000003 |
| tfidf | (+)0.019180 | (+)0.000614 | nan         | (-)0.000002 | (-)0.000360 | (-)0.000028 |
| bm11  | (+)0.000018 | (+)0.000005 | (+)0.000002 | nan         | (-)1.000000 | (-)0.426132 |
| bm15  | (+)0.000035 | (+)0.000008 | (+)0.000360 | (+)1.000000 | nan         | (-)0.004671 |
| bm25  | (+)0.000008 | (+)0.000003 | (+)0.000028 | (+)0.426132 | (+)0.004671 | nan         |

#### nDCG

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.799677 | (-)0.759054 | (-)0.005171 | (-)0.190535 | (-)0.013443 |
| idf   | (+)0.799677 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |
| tfidf | (+)0.759054 | (-)1.000000 | nan         | (-)0.112659 | (-)1.000000 | (-)1.000000 |
| bm11  | (+)0.005171 | (+)1.000000 | (+)0.112659 | nan         | (+)1.000000 | (+)1.000000 |
| bm15  | (+)0.190535 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |
| bm25  | (+)0.013443 | (+)1.000000 | (+)1.000000 | (-)1.000000 | (+)1.000000 | nan         |

#### MRR

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)1.000000 | (-)0.192760 | (-)0.029555 | (-)0.321778 | (-)0.069963 |
| idf   | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |
| tfidf | (+)0.192760 | (-)1.000000 | nan         | (-)1.000000 | (-)1.000000 | (-)1.000000 |
| bm11  | (+)0.029555 | (+)1.000000 | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 |
| bm15  | (+)0.321778 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |
| bm25  | (+)0.069963 | (+)1.000000 | (+)1.000000 | (+)1.000000 | (+)1.000000 | nan         |

### Holm Correction

In [ ]:
p_values_MAP = p_values_correction(0, "holm")
p_values_MAP.to_markdown()


'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (+)0.088310 | (-)0.005115 | (-)0.000012 | (-)0.000019 | (-)0.000006 |\n| idf   | (-)0.088310 | nan         | (-)0.000246 | (-)0.000004 | (-)0.000006 | (-)0.000003 |\n| tfidf | (+)0.005115 | (+)0.000246 | nan         | (-)0.000002 | (-)0.000168 | (-)0.000017 |\n| bm11  | (+)0.000012 | (+)0.000004 | (+)0.000002 | nan         | (-)1.000000 | (-)0.085226 |\n| bm15  | (+)0.000019 | (+)0.000006 | (+)0.000168 | (+)1.000000 | nan         | (-)0.001557 |\n| bm25  | (+)0.000006 | (+)0.000003 | (+)0.000017 | (+)0.085226 | (+)0.001557 | nan         |'

In [ ]:
p_values_nDCG = p_values_correction(1, "holm")
p_values_nDCG.to_markdown()


'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.556639 | (-)0.556639 | (-)0.005171 | (-)0.152428 | (-)0.012547 |\n| idf   | (+)0.556639 | nan         | (+)1.000000 | (-)0.883304 | (-)1.000000 | (-)1.000000 |\n| tfidf | (+)0.556639 | (-)1.000000 | nan         | (-)0.097638 | (-)1.000000 | (-)0.681482 |\n| bm11  | (+)0.005171 | (+)0.883304 | (+)0.097638 | nan         | (+)1.000000 | (+)1.000000 |\n| bm15  | (+)0.152428 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)0.681482 |\n| bm25  | (+)0.012547 | (+)1.000000 | (+)0.681482 | (-)1.000000 | (+)0.681482 | nan         |'

In [ ]:
p_values_MRR = p_values_correction(2, "holm")
p_values_MRR.to_markdown()

'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.823548 | (-)0.167059 | (-)0.029555 | (-)0.257422 | (-)0.065298 |\n| idf   | (+)0.823548 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |\n| tfidf | (+)0.167059 | (-)1.000000 | nan         | (-)1.000000 | (-)1.000000 | (-)1.000000 |\n| bm11  | (+)0.029555 | (+)1.000000 | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 |\n| bm15  | (+)0.257422 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |\n| bm25  | (+)0.065298 | (+)1.000000 | (+)1.000000 | (+)1.000000 | (+)1.000000 | nan         |'

#### MAP

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (+)0.088310 | (-)0.005115 | (-)0.000012 | (-)0.000019 | (-)0.000006 |
| idf   | (-)0.088310 | nan         | (-)0.000246 | (-)0.000004 | (-)0.000006 | (-)0.000003 |
| tfidf | (+)0.005115 | (+)0.000246 | nan         | (-)0.000002 | (-)0.000168 | (-)0.000017 |
| bm11  | (+)0.000012 | (+)0.000004 | (+)0.000002 | nan         | (-)1.000000 | (-)0.085226 |
| bm15  | (+)0.000019 | (+)0.000006 | (+)0.000168 | (+)1.000000 | nan         | (-)0.001557 |
| bm25  | (+)0.000006 | (+)0.000003 | (+)0.000017 | (+)0.085226 | (+)0.001557 | nan         |

#### nDCG

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.556639 | (-)0.556639 | (-)0.005171 | (-)0.152428 | (-)0.012547 |
| idf   | (+)0.556639 | nan         | (+)1.000000 | (-)0.883304 | (-)1.000000 | (-)1.000000 |
| tfidf | (+)0.556639 | (-)1.000000 | nan         | (-)0.097638 | (-)1.000000 | (-)0.681482 |
| bm11  | (+)0.005171 | (+)0.883304 | (+)0.097638 | nan         | (+)1.000000 | (+)1.000000 |
| bm15  | (+)0.152428 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)0.681482 |
| bm25  | (+)0.012547 | (+)1.000000 | (+)0.681482 | (-)1.000000 | (+)0.681482 | nan         |

#### MRR

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.823548 | (-)0.167059 | (-)0.029555 | (-)0.257422 | (-)0.065298 |
| idf   | (+)0.823548 | nan         | (+)1.000000 | (-)1.000000 | (-)1.000000 | (-)1.000000 |
| tfidf | (+)0.167059 | (-)1.000000 | nan         | (-)1.000000 | (-)1.000000 | (-)1.000000 |
| bm11  | (+)0.029555 | (+)1.000000 | (+)1.000000 | nan         | (+)1.000000 | (-)1.000000 |
| bm15  | (+)0.257422 | (+)1.000000 | (+)1.000000 | (-)1.000000 | nan         | (-)1.000000 |
| bm25  | (+)0.065298 | (+)1.000000 | (+)1.000000 | (+)1.000000 | (+)1.000000 | nan         |

### Benjamini-Hochberg

In [ ]:
p_values_MAP = p_values_correction(0, "fdr_bh")
p_values_MAP.to_markdown()


'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (+)0.023654 | (-)0.000799 | (-)0.000001 | (-)0.000002 | (-)0.000001 |\n| idf   | (-)0.023654 | nan         | (-)0.000031 | (-)0.000001 | (-)0.000001 | (-)0.000001 |\n| tfidf | (+)0.000799 | (+)0.000031 | nan         | (-)0.000001 | (-)0.000020 | (-)0.000002 |\n| bm11  | (+)0.000001 | (+)0.000001 | (+)0.000001 | nan         | (-)0.604252 | (-)0.016390 |\n| bm15  | (+)0.000002 | (+)0.000001 | (+)0.000020 | (+)0.604252 | nan         | (-)0.000212 |\n| bm25  | (+)0.000001 | (+)0.000001 | (+)0.000002 | (+)0.016390 | (+)0.000212 | nan         |'

In [ ]:
p_values_nDCG = p_values_correction(1, "fdr_bh")
p_values_nDCG.to_markdown()


'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.066640 | (-)0.066640 | (-)0.002585 | (-)0.023817 | (-)0.003361 |\n| idf   | (+)0.066640 | nan         | (+)0.719015 | (-)0.105155 | (-)0.646183 | (-)0.207476 |\n| tfidf | (+)0.066640 | (-)0.719015 | nan         | (-)0.018777 | (-)0.349183 | (-)0.074790 |\n| bm11  | (+)0.002585 | (+)0.105155 | (+)0.018777 | nan         | (+)0.207476 | (+)0.646183 |\n| bm15  | (+)0.023817 | (+)0.646183 | (+)0.349183 | (-)0.207476 | nan         | (-)0.074790 |\n| bm25  | (+)0.003361 | (+)0.207476 | (+)0.074790 | (-)0.646183 | (+)0.074790 | nan         |'

In [ ]:
p_values_MRR = p_values_correction(2, "fdr_bh")
p_values_MRR.to_markdown()

'|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |\n|:------|:------------|:------------|:------------|:------------|:------------|:------------|\n| tf    | nan         | (-)0.112302 | (-)0.032127 | (-)0.014777 | (-)0.040222 | (-)0.017491 |\n| idf   | (+)0.112302 | nan         | (+)0.873271 | (-)0.660146 | (-)0.660146 | (-)0.497523 |\n| tfidf | (+)0.032127 | (-)0.873271 | nan         | (-)0.497523 | (-)0.660146 | (-)0.497523 |\n| bm11  | (+)0.014777 | (+)0.660146 | (+)0.497523 | nan         | (+)0.951553 | (-)0.660146 |\n| bm15  | (+)0.040222 | (+)0.660146 | (+)0.660146 | (-)0.951553 | nan         | (-)0.660146 |\n| bm25  | (+)0.017491 | (+)0.497523 | (+)0.497523 | (+)0.660146 | (+)0.660146 | nan         |'

#### MAP

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (+)0.023654 | (-)0.000799 | (-)0.000001 | (-)0.000002 | (-)0.000001 |
| idf   | (-)0.023654 | nan         | (-)0.000031 | (-)0.000001 | (-)0.000001 | (-)0.000001 |
| tfidf | (+)0.000799 | (+)0.000031 | nan         | (-)0.000001 | (-)0.000020 | (-)0.000002 |
| bm11  | (+)0.000001 | (+)0.000001 | (+)0.000001 | nan         | (-)0.604252 | (-)0.016390 |
| bm15  | (+)0.000002 | (+)0.000001 | (+)0.000020 | (+)0.604252 | nan         | (-)0.000212 |
| bm25  | (+)0.000001 | (+)0.000001 | (+)0.000002 | (+)0.016390 | (+)0.000212 | nan         |

#### nDCG

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.066640 | (-)0.066640 | (-)0.002585 | (-)0.023817 | (-)0.003361 |
| idf   | (+)0.066640 | nan         | (+)0.719015 | (-)0.105155 | (-)0.646183 | (-)0.207476 |
| tfidf | (+)0.066640 | (-)0.719015 | nan         | (-)0.018777 | (-)0.349183 | (-)0.074790 |
| bm11  | (+)0.002585 | (+)0.105155 | (+)0.018777 | nan         | (+)0.207476 | (+)0.646183 |
| bm15  | (+)0.023817 | (+)0.646183 | (+)0.349183 | (-)0.207476 | nan         | (-)0.074790 |
| bm25  | (+)0.003361 | (+)0.207476 | (+)0.074790 | (-)0.646183 | (+)0.074790 | nan         |

#### MRR

|       | tf          | idf         | tfidf       | bm11        | bm15        | bm25        |
|:------|:------------|:------------|:------------|:------------|:------------|:------------|
| tf    | nan         | (-)0.112302 | (-)0.032127 | (-)0.014777 | (-)0.040222 | (-)0.017491 |
| idf   | (+)0.112302 | nan         | (+)0.873271 | (-)0.660146 | (-)0.660146 | (-)0.497523 |
| tfidf | (+)0.032127 | (-)0.873271 | nan         | (-)0.497523 | (-)0.660146 | (-)0.497523 |
| bm11  | (+)0.014777 | (+)0.660146 | (+)0.497523 | nan         | (+)0.951553 | (-)0.660146 |
| bm15  | (+)0.040222 | (+)0.660146 | (+)0.660146 | (-)0.951553 | nan         | (-)0.660146 |
| bm25  | (+)0.017491 | (+)0.497523 | (+)0.497523 | (+)0.660146 | (+)0.660146 | nan         |

### Conclusion

It is clear from the results that BM25 consistently outperforms the other scoring models, reaffirming its status as one of the most effective and robust retrieval functions. However, the Mean Reciprocal Rank (MRR) metric does not fully reflect this superiority a somewhat expected outcome, since MRR is a rank-sensitive measure that emphasizes the position of the first relevant document rather than overall retrieval quality. This makes MRR less aligned with models like BM25 that optimize for broader ranking effectiveness across multiple relevant documents, rather than focusing solely on the top-ranked one.

### Test Result

We import the test queries and generate a run file using our BM25 tuned model.


This code prepares and executes an information retrieval experiment on the MS MARCO 2020 test set using a tuned BM25 model. It begins by specifying the path to the query file and extracting the queries as (query_id, query_text) pairs.

A BM25 scorer is then instantiated with custom hyperparameters (k1=1, b=0.65) to control term frequency saturation and document length normalization. The inverted index is initialized with the dataset's lexicon, posting lists, term frequencies, document statistics, and global collection statistics, while associating it with the chosen scoring function.

Finally, the script generates a TREC-formatted run file, which records the retrieval results for all queries under the identifier "bm25_test_run". This run file can later be used for evaluation against relevance judgments to measure the effectiveness of the tuned BM25 implementation.


In [ ]:
# Path to MS MARCO test set query file (2020)
queries_file = "msmarco-test2020-queries.tsv"

# BM25 Scorer function with tuned hyperparameters
scorer = BM25Scorer(k1=1, b=0.65)

# Name of the test run of BM25
run_name = "bm25_test_run"

In [ ]:
# Extract queries from the current file
# queries: List of tuples (query_id, query_text)
queries = extract_queries(queries_file)

# Initialize the inverted index with the current scoring function
# This allows different retrieval models (e.g., TF, TF-IDF, BM25)
inv_ind = InvertedIndex(
    lexicon,       # term -> (termid, ...) mapping
    inv_d,         # list of numpy arrays: doc IDs per term
    inv_f,         # list of numpy arrays: term frequencies per term
    doc_index,     # document statistics
    stats,         # global statistics (e.g., total # of documents)
    scorer=scorer  # scoring function to use for this run
)

# Generate and write a TREC-formatted run file for the current query set
# The filename combines the run name and the run suffix
write_run_file(
    queries,              # queries to process
    inv_ind,              # inverted index with current scoring function
    run_name,             # output filename for this run
    run_name              # run identifier (appears in run file)
)

daat (5.908 ms)
daat (99.231 ms)
daat (1687.472 ms)
daat (600.627 ms)
daat (31482.802 ms)
daat (1869.177 ms)
daat (7115.235 ms)
daat (677.498 ms)
daat (7921.152 ms)
daat (1630.932 ms)
daat (269.542 ms)
daat (2813.742 ms)
daat (12593.451 ms)
daat (1399.809 ms)
daat (4613.180 ms)
daat (7813.129 ms)
daat (2946.128 ms)
daat (15919.087 ms)
daat (18695.359 ms)
daat (2413.388 ms)
daat (2314.335 ms)
daat (14942.277 ms)
daat (21152.215 ms)
daat (5793.670 ms)
daat (1059.728 ms)
daat (3074.444 ms)
daat (2205.235 ms)
daat (124.034 ms)
daat (2854.110 ms)
daat (8239.803 ms)
daat (1447.088 ms)
daat (14727.180 ms)
daat (3270.055 ms)
daat (4103.326 ms)
daat (128.123 ms)
daat (2307.709 ms)
daat (9108.725 ms)
daat (9299.859 ms)
daat (6150.089 ms)
daat (7239.929 ms)
daat (5182.303 ms)
daat (5805.129 ms)
daat (5305.539 ms)
daat (4654.574 ms)
daat (14663.310 ms)
daat (3674.541 ms)
daat (3049.235 ms)
daat (17106.726 ms)
daat (8503.889 ms)
daat (4615.597 ms)
daat (12761.919 ms)
daat (2809.077 ms)
daat (3560.3

We define the evaluation metrics of interest, load the relevance judgments (qrels), and compute the results using the TREC evaluation framework.

This code evaluates the performance of a BM25 retrieval run on the MS MARCO 2020 test set using several standard information retrieval metrics. It begins by defining the metrics of interest: Average Precision (AP), normalized Discounted Cumulative Gain at rank 10 (nDCG@10), and Mean Reciprocal Rank at rank 10 (MRR@10).

The ground-truth relevance judgments (qrels) are loaded from the file 2020qrels-pass.txt, while the BM25 run results are read from the corresponding run file. For each metric, the script computes per-query scores using the ir_measures library, ensuring that the results are stored by appending to a table structure. Finally, the per-query scores are printed, providing a detailed view of how the BM25 model performs across different evaluation measures.

In [ ]:
# List of retrieval/scoring methods
method = "bm25"

# Metrics to evaluate
metrics = [
    ("AP", AP),
    ("nDCG@10", nDCG@10),
    ("MRR@10", MRR@10)
]

In [ ]:
# qrels_file contains the ground-truth relevance labels for queries
qrels_file = '2020qrels-pass.txt'

# Read the qrels file using ir_measures utility
# Each entry represents a (query_id, doc_id, relevance) tuple
qrel = list(ir_measures.read_trec_qrels(qrels_file))

# The run file for this method
run_file = method + "_test_run"

# Read the run file using ir_measures utility
# Each entry represents a (query_id, doc_id, score) tuple
run_data = list(ir_measures.read_trec_run(run_file))

In [ ]:
# This will store score per-metric, averaged among all queries
scores = {}

for metric_name, metric_func in metrics:

    # Compute the metric for each query in the run
    per_query_scores = tuple(
        m.value for m in ir_measures.iter_calc([metric_func], qrel, run_data)
    )

    # Compute the average metric for each query
    metric_score = sum(per_query_scores) / len(per_query_scores)

    # Add the score to a dictionary
    scores[metric_name] = metric_score

print(scores)

{'AP': 0.3680402948054137, 'nDCG@10': 0.5123122754755617, 'MRR@10': 0.8271604938271605}


| Metric   | Score   |
|----------|---------|
| AP       | 0.3680  |
| nDCG@10  | 0.5123  |
| MRR@10   | 0.8272  |


## PyTerrier Comparison

### Creating PyTerrier Index

This code demonstrates a complete workflow for indexing and retrieving documents using PyTerrier with the BM25 model. It begins by initializing PyTerrier and loading a tab-separated file (collection.tar) into a Pandas DataFrame, where each row contains a document identifier (docno) and its text. Since PyTerrier requires string inputs, the DataFrame is converted to string format before indexing. The DFIndexer is then used to build an index in the folder index_3docs, and the resulting index reference is loaded with IndexFactory. Collection statistics such as the number of documents and average length are printed to provide insight into the indexed dataset. Retrieval is performed using the BM25 weighting model, with the query "document", and the top-ranked result's document ID, rank, and score are displayed. Finally, the code iterates through all retrieved results, printing each document's identifier, rank, and BM25 score, thereby illustrating how to inspect and analyze search outputs in a structured way.

In [ ]:
import pyterrier as pt

# Initializing Pyterrier
pt.init()

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipython-input-2775686768.py:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [ ]:
import pandas as pd

# Read the file as a tab-separated file (TSV)
df = pd.read_csv(
    "collection.tar",  # TODO: che è collection.tar? quando è stata scaricata? aggiungere qualsiasi cosa sia, o cambiare nome
    sep="\t",
    header=None,
    names=["docno", "text"],
    skiprows=1
)

# PyTerrier wants strings and not integer
df = df.astype(str)

In [ ]:
# Initialize a Terrier DFIndexer to build an index in the folder "index_3docs".
# Setting overwrite=True ensures that any existing index in this folder is replaced.
indexer = pt.DFIndexer("./index_3docs", overwrite=True)

# Index the documents using the DataFrame columns:
# - "text": the document content
# - "docno": the unique document identifier
indexref = indexer.index(df["text"], df["docno"])

# Print a string representation of the index reference (for debugging/inspection).
indexref.toString()

/tmp/ipython-input-3638899273.py:1: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./index_3docs", overwrite=True)


08:05:05.068 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (35723) - further warnings are suppressed
09:36:09.927 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 265 empty documents


'./index_3docs/data.properties'

In [ ]:
# Load the index object from the reference.
index = pt.IndexFactory.of(indexref)

# Display collection statistics (e.g., number of documents, average length).
print(index.getCollectionStatistics())

# Initialize a Retriever with the BM25 weighting model.
br = pt.terrier.Retriever(index, wmodel="BM25")

# Perform a search for the query "document".
k = br.search("document")

# Print details of the top-ranked document:
print(k["docno"][0])   # Document ID of the first result
print(k["rank"][0])    # Rank position of the first result
print(k["score"][0])   # BM25 score of the first result


# Use itertuples to iterate over the search results DataFrame.
# Each row contains (docno, rank, score) for a retrieved document.
for row in k.itertuples(index=True, name="Result"):
    docno, rank, score = row.docno, row.rank, row.score
    print(f"docno: {docno}, rank: {rank}, score: {score}")

Number of documents: 8841822
Number of terms: 1168845
Number of postings: 215136819
Number of fields: 0
Number of tokens: 288586998
Field names: []
Positions:   false

7020668
0
13.24502440525085
docno: 7020668, rank: 0, score: 13.24502440525085
docno: 7020666, rank: 1, score: 13.24245619740485
docno: 5752808, rank: 2, score: 13.005899012907783
docno: 5707062, rank: 3, score: 12.967163280052093
docno: 5752809, rank: 4, score: 12.965962005008993
docno: 1877654, rank: 5, score: 12.906734864605493
docno: 7406563, rank: 6, score: 12.864100306050426
docno: 1015990, rank: 7, score: 12.862031500056508
docno: 7451610, rank: 8, score: 12.862031500056508
docno: 5589815, rank: 9, score: 12.84735118132147
docno: 5589809, rank: 10, score: 12.819280318777322
docno: 7551108, rank: 11, score: 12.811201574359899
docno: 2995443, rank: 12, score: 12.793665209666331
docno: 3517429, rank: 13, score: 12.793665209666331
docno: 4246251, rank: 14, score: 12.793665209666331
docno: 2205195, rank: 15, score: 12.7

### Creating PyTerrier Run File

#### Helper Functions

These are the same helper functions we used earlier, included here again to avoid having to scroll back and forth.
The second function has been slightly adapted to use a PyTerrier retriever instead of the custom InvertedIndex class.

In [ ]:
# TODO: queste funzioni sono ridefiniteeeee

In [ ]:
from typing import List, Tuple

def extract_queries(
    filename: str
) -> List[Tuple[str, str]]:
    """
    Description:
        Reads a TSV file containing query IDs and query texts,
        and returns them as a list of (query_id, query) tuples.

    Arguments:
        filename (str): Path to the TSV file.
          Each line should contain 'query_id<TAB>query_text'.

    Returns:
        List[Tuple[str, str]]: A list where each element is a tuple
          containing (query_id, query_text).
    """
    queries: List[Tuple[str, str]] = []

    with open(filename, "r", encoding="utf-8") as f:
        for line in f:

            # Skipping malformed lines
            try:
                query_id, query = line.strip().split("\t")
            except ValueError:
                continue

            # Appending query_id and query text
            queries.append((query_id, query))

    return queries


In [ ]:
def write_run_file(
    queries: list[tuple[str, str]],
    retriever: pt.terrier.Retriever,
    output_path: str,
    run_name: str = "my_run",
):
    """
    Description:
        Generates a TREC-formatted run file from a set of queries and the inverted index.

    Arguments:
        queries (Dict[int, str]): Mapping from query IDs to query strings.
        index (InvertedIndex): The inverted index used for retrieval.
        output_path (str): Path to save the output run file.
        run_name (str): Identifier for the run (appears in the last column).
    """

    with open(output_path, "w") as f:
        for qid, query in queries:

            # List of scores and docids ordered by score
            try:
                results: list[tuple[np.float64, np.int32]] = retriever.search(query)
            except:
                print(query)


            # Iterating over results and adding line
            for row in results.itertuples(index=True):
                docno, rank, score = row.docno, row.rank, row.score
                f.write(f"{qid} Q0 {docno} {rank} {score:.6f} {run_name}\n")

    print(f"Run file saved to: {output_path}")


#### Creating Run File

This code reads the MS MARCO 2020 test queries from a TSV file, then uses a PyTerrier BM25 retriever to process each query. For each query, it retrieves documents, computes their scores, and writes the results in a TREC-formatted run file. The run file includes the query ID, document ID, rank, score, and run name, making it ready for evaluation with standard IR tools.

In [ ]:
# Path to the MS MARCO 2020 test queries file (TSV format: query_id<TAB>query_text)
queries_file = "msmarco-test2020-queries.tsv"

# Name of this retrieval run (appears in the TREC run file)
run_name = "pyterrier_bm25_run"

# Output filename for the TREC-formatted run
run_file = run_name + ".txt"

# Read queries from the TSV file and store as a list of tuples: (query_id, query_text)
queries = extract_queries(queries_file)

# Using the PyTerrier BM25 retriever (`br`) to process each query,
# compute scores for retrieved documents, and write results to a run file
write_run_file(
    queries,    # list of (query_id, query_text) tuples
    br,         # PyTerrier retrieval object (BM25)
    run_file,   # output filename for the run file
    run_name    # run identifier to appear in the last column of the run file
)

print(f"Run file '{run_file}' successfully created using PyTerrier BM25 retriever.")

define: geon
Run file saved to: pyterrier_bm25_run.txt
Run file 'pyterrier_bm25_run.txt' successfully created using PyTerrier BM25 retriever.


### Statistical Test

These two code blocks perform a comparative evaluation between a PyTerrier BM25 run and a custom BM25 run. First, the TREC-formatted run files and ground truth relevance judgments are read, and per-query evaluation metrics Mean Average Precision (MAP), nDCG@10, and MRR@10 are computed for both runs. Then, paired t-tests are applied to these per-query scores to determine whether there are statistically significant differences between the two systems for each metric. The resulting t-statistics indicate the direction of the difference, while the p-values indicate the significance of the observed differences, allowing a quantitative comparison of retrieval effectiveness.

In [ ]:
# Qrels (ground truth relevance judgments)
qrels_file = '2020qrels-pass.txt'

# TREC-formatted run file generated by PyTerrier BM25
pt_run_file = 'pyterrier_bm25_run.txt'

# Custom run file generated using our own BM25 implementation
my_run_file = 'bm25_test_run'

# Read qrels into a list of relevance judgments and read the runs into
# lists of run entries
qrel = list(ir_measures.read_trec_qrels(qrels_file))
pt_run_data = list(ir_measures.read_trec_run(pt_run_file))
my_run_data = list(ir_measures.read_trec_run(my_run_file))

# Mean Average Precision (MAP), nDCG and MRR for PyTerrier Run
pt_MAP = tuple(m.value for m in ir_measures.iter_calc([AP], qrel, pt_run_data))
pt_nDCG = tuple(m.value for m in ir_measures.iter_calc([nDCG@10], qrel, pt_run_data))
pt_MRR = tuple(m.value for m in ir_measures.iter_calc([MRR@10], qrel, pt_run_data))

# Mean Average Precision (MAP), nDCG and MRR for Custom Run
my_MAP = tuple(m.value for m in ir_measures.iter_calc([AP], qrel, my_run_data))
my_nDCG = tuple(m.value for m in ir_measures.iter_calc([nDCG@10], qrel, my_run_data))
my_MRR = tuple(m.value for m in ir_measures.iter_calc([MRR@10], qrel, my_run_data))

In [ ]:
from scipy.stats import ttest_rel

# ---------------------- Paired t-tests Between PyTerrier and Custom Runs ---------------------- #

# Paired t-test for Mean Average Precision (MAP)
# Compares per-query MAP scores between the two runs
t_stat, p_val = ttest_rel(pt_MAP, my_MAP)
print(f"MAP: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")

# Paired t-test for nDCG@10
# Compares per-query nDCG@10 scores between the two runs
t_stat, p_val = ttest_rel(pt_nDCG, my_nDCG)
print(f"nDCG@10: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")

# Paired t-test for MRR@10
# Compares per-query MRR@10 scores between the two runs
t_stat, p_val = ttest_rel(pt_MRR, my_MRR)
print(f"MRR@10: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")

MAP: t_stat = -1.761199, p_value = 0.083973
nDCG@10: t_stat = -1.925950, p_value = 0.059482
MRR@10: t_stat = -0.897236, p_value = 0.373651


| Metric   | t-stat     | p-value   |
|----------|------------|-----------|
| MAP      | -1.761199  | 0.083973  |
| nDCG@10  | -1.925950  | 0.059482  |
| MRR@10   | -0.897236  | 0.373651  |


Our BM25 model shows slightly higher performance across the three evaluation metrics. However, the p-values indicate that these improvements are not statistically significant. This outcome is expected, since our BM25 model was tuned on a separate evaluation set, which naturally leads to marginally better scores. Nevertheless, the default parameters provided by PyTerrier do not yield results that are statistically inferior, highlighting that the observed differences are not meaningful in a rigorous statistical sense.

## Space and Time Optimization

### Pre-Computed Weights

#### Pre-Computing Weights

This block of code precomputes BM25 weights for every term–document pair in the inverted index, replacing raw term frequencies with their BM25-weighted equivalents. For each term, it retrieves key statistics such as document frequency, total number of documents, and document length, then applies the BM25 formula that combines an inverse document frequency (IDF) component and a term frequency (TF) component normalized by document length. The constants k₁ and b control term frequency saturation and document length normalization, respectively. By storing the computed BM25 weights directly into the posting lists (inv_f), the retrieval process becomes more efficient since scoring computations are effectively done in advance.

We multiply the scores by 1000 because their absolute values are quite small, and ranking decisions often depend on the digits immediately after the decimal point. By scaling the scores, we preserve the first three decimal places, ensuring that subtle differences between documents are retained. At the same time, multiplying by 1000 avoids the risk of numerical overflow, making the ranking process both precise and safe.

In [ ]:
# Creating a mapping from termid to term, implemented with a list
termid_to_term = [ term for term, _ in lexicon.items() ]

In [ ]:
import numpy as np

# This block modifies the posting list frequencies (inv_f) by replacing
# raw term frequencies (TF) with their precomputed BM25-weighted values.
# Doing this in advance can save computation time at query time.

# BM25 hyperparameters (standard values)
b = 0.65
k1 = 1

# Computing scaling factor on the basis of how many decimal places we
# want to keep in our precomputed weights
decimal_place = 3
scaling_factor = 10 ** decimal_place
# TODO: vuol dire che facciamo anche quantizzazione? non c'è scritto sul commento sopra

# Iterate over each term in the inverted index
for termid in inv_f:

    # For each occurrence of the term, get its term frequency (tf) and document ID (docid)
    for i, (tf, docid) in enumerate(zip(inv_f[termid], inv_d[termid])):

        #  Retrieving Required Statistics
        N = stats['num_docs']
        term = termid_to_term[termid]  # TODO: perchè non lo facciamo normalmente con il lexicon? se teniamo questo portare la lista dentro la cella
        df = lexicon[term][1]

        # Computing BM1 component
        bm1 = np.log((N - df + 0.5) / (df + 0.5))

        # Get the document length and the average document length
        dl = doc_index[docid - 1][1]
        avg_dl = stats['average_document_length']

        # Compute the normalization factor for document length
        # This adjusts TF based on how long the document is compared to the average.
        norm_factor = (1 - b) + b * (dl / avg_dl)

        # Apply document length normalization to TF
        tf_tilde = tf / norm_factor

        # Compute the BM25 TF scaling: (tf_tilde) / (k1 + tf_tilde)
        tf_approx = tf_tilde / (k1 + tf_tilde)

        # Combine IDF and TF components to get the full BM25 weight
        bm25 = bm1 * tf_approx

        # Replace the original term frequency with the BM25 weight
        # This stores the weighted value directly in the inverted index for efficiency.
        inv_f[termid][i] = scaling_factor * bm25
        # TODO: ma non è quantizzato? a che serve scalarlo? è perchè l'np.array è già int32?

#### Optimized DAAT

The main optimization made are:
1. **Efficient min_docid Computation**: The min_docid function uses a list comprehension combined with Python's built-in min() to quickly identify the smallest document ID among active posting lists. This avoids unnecessary loops and provides a modest performance boost by leveraging optimized native operations.
2. **An efficient pruning mechanism** that identifies exhausted posting lists using a boolean flag. Once flagged, it employs a Python list comprehension to swiftly remove these inactive iterators, ensuring that subsequent iterations only process active postings and thereby improving performance.

In [ ]:
import math
from typing import List, Set, Tuple
import numba
import heapq


# TODO: stai dicendo che computare il min_docid in questo modo è meglio rispetto a prima?
def min_docid(
    postings: List['InvertedIndex.PostingListIterator']
) -> int:
    """
    Description:
        Returns the smallest current docid among all posting list iterators.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
    Returns:
        int: Minimum current docid, or math.inf if all postings are exhausted.
    """

    # I am simply compressing the operation using list comprehension
    return min([p.docid() for p in postings if not p._is_end_list()])


@profile
def daat(
    postings: List['InvertedIndex.PostingListIterator'],
    k: int = 10
) -> List[Tuple[float, int]]:
    """
    Description:
        Computes document-at-a-time (DAAT) scoring for a list of posting lists.
        Scores each document across all postings simultaneously and returns
        the top-k documents.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
        k (int): Number of top-scoring documents to return. Default is 10.
    Returns:
        List[Tuple[float, int]]: List of (score, docid) tuples, sorted in descending order.
    """

    top = TopQueue(k)

    # Initialize current_docid as the smallest docid among all postings
    current_docid = min_docid(postings)

    # Until you have finished all the posting lists you compute the score
    # for the next document
    while current_docid != math.inf:

        # I have only added the updated variable as it will be used to remove
        # useless posting lists (posting lists that have been completed)
        score = 0.0                 # Score of the Document
        next_docid = math.inf       # Docid that will be evaluated on the next iter
        update = False              # It's set to true when some posting have been finished

        # Iterate over each posting head
        for posting in postings:

            # Extract docid
            docid = posting.docid()

            # If it matches current doc then update the score and advance on the
            # posting list
            if docid == current_docid:
                score += posting.score()
                posting.next()
                docid = posting.docid()  # update after advancing

            # If the posting list is not ended then consider the docid head as
            # a candidate for the next docid
            if not posting._is_end_list():
                next_docid = min(next_docid, docid)

            # Otherwise set update to true so it will be pruned
            else:
                update = True

        # Pruning useless Posting lists
        if update:
            postings = [p for p in postings if not p._is_end_list()]

        # Inserting in top queue and updating current docid
        top.insert(current_docid, score) # type: ignore
        current_docid = next_docid

    # Return top-k results sorted by score descending
    return sorted(top.queue, reverse=True)


Converting NumPy arrays to native Python lists improves the performance of the daat function in this context because the function relies heavily on control flow, dynamic indexing, and method calls on custom iterator objects. While NumPy excels at vectorized numerical operations, it introduces overhead when used for frequent element-wise access or conditional logic inside Python loops. By switching to Python lists, we reduce the cost of array conversion and gain faster iteration and indexing, making the DAAT traversal more efficient for this workload.

I attempted to optimize the daat function using NumPy vectorized operations, but the performance gains were limited.

In [ ]:
# TODO: cosa migliora esattamente? list comprehension? identificare le istruzioni esatte

for term_id in inv_f:
  inv_f[term_id] = inv_f[term_id].tolist()

for term_id in inv_d:
  inv_d[term_id] = inv_d[term_id].tolist()

I copied the TopQueue class so that all required classes are located together for easier access. In addition, I made a small adjustment to the inverted index scoring function: the check for reaching the end of the posting list has been removed. This simplification is safe because we can assume that the method will only be called when the posting list is not yet exhausted, as the daat method already performs that validation.

In [ ]:
import heapq
from typing import List, Tuple

# TODO io toglierei se è duplicato

class TopQueue:
    """
    Description:
        Maintains the top-k scored items in a min-heap with a dynamic threshold.
    Arguments:
        k (int): Maximum number of items to keep. Default is 10.
        threshold (float): Minimum score required for an item to enter. Default is 0.0.
    """

    def __init__(self, k: int = 10, threshold: float = 0.0):
        self.queue: List[Tuple[float, int]] = []  # heap of (score, docid)
        self.k: int = k
        self.threshold: float = threshold

    def size(self) -> int:
        """Returns the number of items currently in the queue."""
        return len(self.queue)

    def would_enter(self, score: float) -> bool:
        """
        Checks if a given score is high enough to enter the queue.
        Arguments:
            score (float): Score to test.
        Returns:
            bool: True if score exceeds current threshold.
        """
        return score > self.threshold

    def clear(self, new_threshold: float = None):
        """
        Clears the queue.
        Arguments:
            new_threshold (float, optional): If provided, updates the threshold.
        """
        self.queue = []
        if new_threshold is not None:
            self.threshold = new_threshold

    def __repr__(self) -> str:
        return f'<TopQueue: {self.size()} items, th={self.threshold}, {self.queue}>'

    def insert(self, docid: int, score: float) -> bool:
        """
        Inserts a document with its score into the top-k queue if it exceeds the threshold.
        Arguments:
            docid (int): Document ID.
            score (float): Score of the document.
        Returns:
            bool: True if the item was inserted, False otherwise.
        """
        if score > self.threshold:
            if self.size() >= self.k:
                # Replace the smallest element if heap is full
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))

            # Update threshold to the smallest score in the heap if full
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True

        return False


In [ ]:
import math
import numpy as np
from typing import List, Dict, Tuple, Callable, Optional

# TODO: toglierei pure questo

class InvertedIndex:

    class PostingListIterator:
        """
        Description:
            Iterator over a posting list for a single term in the inverted index.
        Arguments:
            docids (np.ndarray): Array of document IDs containing the term.
            freqs (np.ndarray): Array of term frequencies corresponding to docids.
            doc (Dict[int, Tuple]): Dictionary storing document statistics (e.g., length).
        """

        def __init__(
            self,
            docids: np.ndarray,
            freqs: np.ndarray,
            termid: int,
            token: str,
            lexicon: Dict[str, Tuple],
            doc: Dict[int, Tuple],
            stats: Dict[str, int],
            scorer: Callable[['InvertedIndex.PostingListIterator'], float]
        ):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.termid = termid
            self.token = token
            self.lexicon = lexicon
            self.doc = doc
            self.stats = stats
            self._scorer = scorer

        def docid(self) -> int:
            """
            Description:
                Returns the current document ID.
            Returns:
                int: Current document ID, or math.inf if iterator is at the end.
            """
            return math.inf if self._is_end_list() else self.docids[self.pos]

        def score(self) -> float:
            """
            Description:
                Computes the score for the current document, which is constant
                for every document in the same posting
            Returns:
                float: Score for the current document, or math.inf if iterator is at the end.
            """
            return self._scorer(self)

        def next(self, target: Optional[int] = None):
            """
            Description:
                Advances the iterator. If target is provided, jump to the first document
                with ID >= target.
            Arguments:
                target (Optional[int]): Target document ID to skip to. Default is None.
            """
            if target is None:
                if not self._is_end_list():
                    self.pos += 1
            elif target > self.docid():
                self.pos = np.searchsorted(self.docids, target, side='left')

        def len(self) -> int:
            """
            Description:
                Returns the number of documents in the posting list.
            Returns:
                int: Length of the posting list.
            """
            return len(self.docids)

        def _is_end_list(self) -> bool:
            """
            Description:
                Checks whether the iterator has reached the end of the posting list.
            Returns:
                bool: True if at end, False otherwise.
            """
            return self.pos >= len(self.docids)

    # ----------------- InvertedIndex main ----------------- #

    """
    Description:
        Inverted index storing the mapping from terms to posting lists, along with
        document statistics.
    Arguments:
        lexicon (Dict[str, Tuple[int, ...]]): Maps token to (termid, ...).
        inv_d (List[np.ndarray]): List of arrays of document IDs per term.
        inv_f (List[np.ndarray]): List of arrays of term frequencies per term.
        doc (Dict[int, Tuple]): Document statistics dictionary.
        stats (Dict[str, int]): Global statistics like number of documents.
    """

    def __init__(
        self,
        lexicon: Dict[str, Tuple[int, ...]],
        inv_d: List[np.ndarray],
        inv_f: List[np.ndarray],
        doc: Dict[int, Tuple],
        stats: Dict[str, int],
        scorer: Callable[['InvertedIndex.PostingListIterator'], float]
    ):
        self.lexicon = lexicon
        self.inv_d = inv_d
        self.inv_f = inv_f
        self.doc = doc
        self.stats = stats
        self.scorer = scorer

    def num_docs(self) -> int:
        """
        Description:
            Returns the total number of documents in the index.
        Returns:
            int: Number of documents.
        """
        return self.stats['num_docs']

    def get_posting(self, termid: int, term: str) -> 'PostingListIterator':
        """
        Description:
            Returns a PostingListIterator for a given term ID.
        Arguments:
            termid (int): The term ID to retrieve the posting list for.
        Returns:
            PostingListIterator: Iterator over the posting list.
        """
        return InvertedIndex.PostingListIterator(
            self.inv_d[termid],
            self.inv_f[termid],
            termid,
            term,
            self.lexicon,
            self.doc,
            self.stats,
            self.scorer
        )

    def get_termids(self, tokens: List[str]) -> List[int]:
        """
        Description:
            Returns a list of term IDs for the tokens that exist in the lexicon.
        Arguments:
            tokens (List[str]): List of tokens to look up.
        Returns:
            List[int]: List of corresponding term IDs.
        """
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(
        self,
        termids: List[int],
        terms: List[str]
    ) -> List['PostingListIterator']:
        """
        Description:
            Returns a list of PostingListIterators for a list of term IDs.
        Arguments:
            termids (List[int]): List of term IDs.
        Returns:
            List[PostingListIterator]: List of posting list iterators.
        """
        return [self.get_posting(termid, term) for termid, term in zip(termids, terms)]


#### Creating the Run File

This block of code runs an information retrieval experiment using precomputed BM25 weights. It first loads the test queries from the MS MARCO 2019 dataset and initializes an InvertedIndex object with a special PreComputedScorer, which retrieves document scores directly from pre-stored BM25 weights rather than recalculating them at query time. This makes retrieval faster and more efficient. Finally, it processes all queries, ranks documents using the precomputed scores, and writes the results into a TREC-formatted run file named precomputed_bm25_run.txt, which can be later used for evaluation and comparison against other retrieval runs.

In [ ]:
class PreComputedScorer:
    """
    Using Precomputed Score
    Arguments:
    """

    def __call__(
        self,
        posting: 'InvertedIndex.PostingListIterator'
    ) -> int:

        # Returning precomputed score
        return posting.freqs[posting.pos]

In [ ]:

# File where the test queries are stored
queries_file = "msmarco-test2020-queries.tsv"

# Name of the run with the precomputed bm25 weights
run_name = "precomputed_bm25_run"

# Name of the file where the run will be stored
run_file = run_name + ".txt"

# Scorer Leveraging PreComputed Weights
scorer = PreComputedScorer()

# Extracting queries from test queries file
queries = extract_queries(queries_file)

# Initialize the inverted index with the current scoring function
# This allows different retrieval models (e.g., TF, TF-IDF, BM25)
inv_ind = InvertedIndex(
    lexicon,       # term -> (termid, ...) mapping
    inv_d,         # list of numpy arrays: doc IDs per term
    inv_f,         # list of numpy arrays: term frequencies per term
    doc_index,     # document statistics
    stats,         # global statistics (e.g., total # of documents)
    scorer=scorer  # scoring function to use for this run
)

# Generate and write a TREC-formatted run file for the current query set
# The filename combines the run name and the run suffix
write_run_file(
    queries,              # queries to process
    inv_ind,              # inverted index with current scoring function
    run_file,             # output filename for this run
    run_name              # run identifier (appears in run file)
)

daat (0.867 ms)
daat (15.580 ms)
daat (151.159 ms)
daat (43.116 ms)
daat (2480.398 ms)
daat (103.583 ms)
daat (497.643 ms)
daat (47.770 ms)
daat (515.802 ms)
daat (84.577 ms)
daat (16.465 ms)
daat (164.247 ms)
daat (899.553 ms)
daat (78.706 ms)
daat (306.279 ms)
daat (520.885 ms)
daat (231.866 ms)
daat (959.585 ms)
daat (1401.083 ms)
daat (79.005 ms)
daat (119.663 ms)
daat (1130.219 ms)
daat (1664.289 ms)
daat (301.268 ms)
daat (59.358 ms)
daat (236.207 ms)
daat (148.535 ms)
daat (8.348 ms)
daat (247.914 ms)
daat (429.762 ms)
daat (106.480 ms)
daat (990.716 ms)
daat (190.345 ms)
daat (163.360 ms)
daat (4.572 ms)
daat (137.046 ms)
daat (679.724 ms)
daat (514.930 ms)
daat (307.326 ms)
daat (417.087 ms)
daat (277.322 ms)
daat (350.286 ms)
daat (363.892 ms)
daat (303.242 ms)
daat (1258.284 ms)
daat (165.884 ms)
daat (210.074 ms)
daat (1546.358 ms)
daat (508.432 ms)
daat (358.212 ms)
daat (936.248 ms)
daat (161.600 ms)
daat (154.066 ms)
daat (755.840 ms)
daat (143.595 ms)
daat (291.538 ms)


#### Measuring Time Optimization

I extracted and cleaned the execution logs from both this run and the previous BM25 run, keeping only the integer part of the execution times in milliseconds (using grep), and saved them into two separate text files. The analysis shows that the optimized version is approximately an order of magnitude faster than the standard BM25 implementation, and the performance improvement is also statistically significant.

This code compares the execution times of two retrieval methods. It begins by reading the recorded execution times (in milliseconds) from two text files, where each line represents the duration of a single query run. Using a paired t-test from the scipy.stats library, it tests whether the difference in execution times between the two methods is significant. The script then prints the t-statistic and p-value from the test, along with the average execution time for each method, providing a clear indication of both the magnitude and statistical reliability of the optimization's performance gain.

In [ ]:
from scipy.stats import ttest_rel

# Paths to files containing execution times (in ms)
daat_times_file = "daat_bm25_time.txt"        # baseline DAAT BM25 times
opt_daat_times_file = "opt_daat_bm25_time.txt"  # optimized DAAT BM25 times

# Each file contains one time measurement per line (integer milliseconds)
with open(daat_times_file, "r") as f:
    daat_times = [float(line.strip()) for line in f]

with open(opt_daat_times_file, "r") as f:
    opt_daat_times = [float(line.strip()) for line in f]

# Perform a paired t-test to determine if the optimization led to a significant difference
t_stat, p_val = ttest_rel(daat_times, opt_daat_times)

print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.6e}")

# Compute and print average execution times for both methods
avg_daat = sum(daat_times) / len(daat_times)
avg_opt_daat = sum(opt_daat_times) / len(opt_daat_times)

print(f"Average DAAT BM25 time: {avg_daat:.2f} ms")
print(f"Average Optimized DAAT BM25 time: {avg_opt_daat:.2f} ms")

#TODO: che è Average DAAT BM25 time: 3883.40 ms???? prima non stava sul secondo/secondo e mezzo?
# sei sicuro che il tempo sia migliorato anche per le due cose (min docid e pruning) e non solo per il precomputing?

T-statistic: 7.7834, P-value: 3.772940e-13
Average DAAT BM25 time: 3883.40 ms
Average Optimized DAAT BM25 time: 452.01 ms


The geometric mean and also the median are good alternatives.
In our case study, the average (arithmetic mean) response time is not a particularly insightful metric, as it is highly sensitive to outliers. For example, if we have 20 queries, one taking 20 seconds and the rest completing in just a few milliseconds, the average would misleadingly suggest a response time of over one second. In contrast, the geometric mean and the median offer more robust alternatives, as they are less influenced by extreme values and provide a more representative view of typical performance.

This code reads a file containing execution times, computes their geometric mean and median, and prints the results. It begins by opening the file opt_daat_bm25_time.txt, which stores one timing value per line, and converts each line into a floating‑point number stored in a list. The geometric mean is then calculated using SciPy’s gmean function, which is particularly useful for averaging values that vary multiplicatively, such as performance times or ratios. To compute the median, the list of times is sorted and its length determined; if the number of elements is odd, the middle value is selected, while for an even number of elements, the average of the two central values is taken. Finally, both the geometric mean and the median are printed, providing two complementary measures of central tendency for the execution time data.

In [ ]:
from scipy.stats import gmean


# Declaring path to the file containing execution times (one per line)
# and the list that will hold execution times
time_filename = "opt_daat_bm25_time.txt"
times = []

# Open the file, read all lines and convert them into a list
with open(time_filename, "r") as f:
    lines = f.readlines()                   # Reading Lines
    times = [float(line) for line in lines] # Converting using list comprehension


# Geometric mean is useful for averaging ratios or times when values
# vary multiplicatively, it's computed with scipy package to avoid overflow
# TODO: non essendo questo il caso (valori si combinano con moltiplicazione), siamo sicuri abbia senso usarlo?
geo_mean = gmean(times)

# Sort the list of times to prepare for median calculation and extract
# length as it will avoid lengthy expression
sorted_times = sorted(times)
l = len(sorted_times)

# If the number of elements is odd, take the middle element
if l % 2 == 1:
    median = sorted_times[l // 2]
# If even, take the average of the two middle elements
else:
    median = (sorted_times[l // 2] + sorted_times[(l // 2) - 1]) / 2

# Print results
print("Geometric Mean:", geo_mean)
print("Median:", median)


Geometric Mean: 196.40473532397888
Median: 283.1355


**Geometric Mean Time**: 196ms

**Median Time**: 283ms

#### Measuring Performance

Since the BM25 weights were truncated to reduce memory usage, it is important to verify that this approximation does not significantly degrade retrieval performance. Upon evaluation, the truncated-weight model shows a slight decrease in effectiveness compared to the original version; however, statistical testing confirms that this difference is not significant, meaning the optimization achieves faster or more memory-efficient retrieval without a meaningful loss in accuracy.

This block of code performs a statistical comparison between the optimized BM25 retrieval system (using precomputed and truncated weights) and the standard BM25 implementation. It first loads the ground truth relevance judgments (qrels) and the two run files containing the ranked retrieval results. Using the ir_measures library, it computes three standard information retrieval metrics Mean Average Precision (MAP), nDCG@10, and MRR@10 for both runs on a per-query basis. Finally, it applies paired t-tests to each metric, comparing the optimized and unoptimized systems to check whether performance differences are statistically significant. The printed t-statistics and p-values indicate whether the observed variations are meaningful or likely due to random chance.

In [ ]:
# TODO: da pensare assieme, stiamo facendo model selection sul test? mhh, non so se sia giusto
#si può giustificare dicendo che non è model selection ma solo performance check

# Qrels (ground truth relevance judgments)
qrels_file = '2020qrels-pass.txt'

# Custom run file generated using optimized BM25
opt_bm25_run_file = 'precomputed_bm25_run.txt'

# Custom run file generated using BM25
bm25_run_file = 'bm25_test_run'


# Read qrels into a list of relevance judgments and read the runs into
# lists of run entries
qrel = list(ir_measures.read_trec_qrels(qrels_file))
opt_bm25_run_data = list(ir_measures.read_trec_run(opt_bm25_run_file))
bm25_run_data = list(ir_measures.read_trec_run(bm25_run_file))


# Mean Average Precision (MAP), nDCG and MRR for Optimized Run
opt_MAP = tuple(m.value for m in ir_measures.iter_calc([AP], qrel, opt_bm25_run_data))
opt_nDCG = tuple(m.value for m in ir_measures.iter_calc([nDCG@10], qrel, opt_bm25_run_data))
opt_MRR = tuple(m.value for m in ir_measures.iter_calc([MRR@10], qrel, opt_bm25_run_data))


# Mean Average Precision (MAP), nDCG and MRR for Unoptimized Run
MAP = tuple(m.value for m in ir_measures.iter_calc([AP], qrel, bm25_run_data))
nDCG = tuple(m.value for m in ir_measures.iter_calc([nDCG@10], qrel, bm25_run_data))
MRR = tuple(m.value for m in ir_measures.iter_calc([MRR@10], qrel, bm25_run_data))

In [ ]:
from scipy.stats import ttest_rel

# Paired t-test for Mean Average Precision (MAP)
# Compares per-query MAP scores between the two runs
t_stat, p_val = ttest_rel(opt_MAP, MAP)
print(f"MAP: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")

# Paired t-test for nDCG@10
# Compares per-query nDCG@10 scores between the two runs
t_stat, p_val = ttest_rel(opt_nDCG, nDCG)
print(f"nDCG@10: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")

# Paired t-test for MRR@10
# Compares per-query MRR@10 scores between the two runs
t_stat, p_val = ttest_rel(opt_MRR, MRR)
print(f"MRR@10: t_stat = {t_stat:.6f}, p_value = {p_val:.6f}")


# TODO: piccolo check personale, nan vuol dire che non cambia il valore tra i due? in caso scriverlo sul commento

MAP: t_stat = -0.222298, p_value = 0.824937
nDCG@10: t_stat = nan, p_value = nan
MRR@10: t_stat = nan, p_value = nan


### Dynamic Pruning

#### Helper Functions and Data Structures

This piece of code precomputes the maximum possible contribution of each term to any document score, which is essential for implementing MaxScore or other dynamic pruning techniques. Here, N represents the total number of documents in the collection. For each term in the lexicon, the code retrieves the document frequency df (the number of documents containing that term) and computes the logarithm of the inverse document frequency, np.log(N / df). This value corresponds to the upper bound of the term's contribution to a document's score, assuming the highest possible term frequency. Storing these upper bounds in termid_to_upperbound allows the retrieval algorithm to quickly estimate whether a document could potentially enter the top-k results, enabling efficient pruning of documents that cannot surpass the current top scores.

In [ ]:
# TODO: scrivere che è l'upper bound con tf -> +inf
# Termid is mapped to the upperbound using a list
N = stats['num_docs']
termid_to_upperbound = [
    np.log(N / df) for _term, (_term_id, df, _tf) in lexicon.items()
]

The dynamic_pruning_daat function implements a document-at-a-time (DAAT) retrieval algorithm with MaxScore dynamic pruning. It iterates over multiple posting lists simultaneously, computing scores for each document across all terms. Using precomputed upper bounds for each term, the function estimates the maximum possible score a document could achieve and skips any document that cannot enter the current top-k results, significantly improving efficiency. For documents that are not skipped, the actual score is computed by summing the contributions from all relevant terms, and the document is inserted into a top-k priority queue. The function returns the top-k documents sorted by descending score, providing an optimized retrieval mechanism that improves speed while not affecting accuracy.



In [ ]:
from typing import List, Tuple


# TODO: prima hai detto che scritto come prima funge meglio!!!

def min_docid(
    postings: List['InvertedIndex.PostingListIterator']
) -> int:
    """
    Description:
        Returns the smallest current docid among all posting list iterators.
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
    Returns:
        int: Minimum current docid, or math.inf if all postings are exhausted.
    """

    # I am iterating over the heads of the postings and returning the min
    min_docid_value = math.inf
    for p in postings:
        if not p._is_end_list():
            min_docid_value = min(p.docid(), min_docid_value)
    return min_docid_value

@profile
def dynamic_pruning_daat(
    postings: List['InvertedIndex.PostingListIterator'],
    termid_to_upperbound: List[float],
    k: int = 10
) -> List[Tuple[float, int]]:
    """
    Description:
        Computes document-at-a-time (DAAT) scoring for a list of posting lists.
        Scores each document across all postings simultaneously and returns
        the top-k documents. It uses MaxScore optimization for dynamic pruning
    Arguments:
        postings (List[PostingListIterator]): List of posting list iterators.
        termid_to_upperbound (List[float]): List of upper bounds for each term.
        k (int): Number of top-scoring documents to return. Default is 10.
    Returns:
        List[Tuple[float, int]]: List of (score, docid) tuples, sorted in descending order.
    """

    top = TopQueue(k)

    # Initialize the first document ID to process
    current_docid = min_docid(postings)  # Find the smallest docid among all postings

    while current_docid != math.inf:
        # Estimate the maximum possible score for this document
        max_possible_score = sum(termid_to_upperbound[p.termid] for p in postings)

        # Skip this document if its maximum possible score cannot beat the current top-k
        if not top.would_enter(max_possible_score):
            next_docid = math.inf
            for p in postings:
                if p.docid() == current_docid:
                    p.next()
                if not p._is_end_list():
                    next_docid = min(next_docid, p.docid())
            current_docid = next_docid
            continue

        # Compute the actual score for the current document
        score = 0.0
        next_docid = math.inf
        for p in postings:
            if p.docid() == current_docid:
                score += p.score()
                p.next()
            if not p._is_end_list():
                next_docid = min(next_docid, p.docid())

        # Insert document into the top-k queue
        top.insert(current_docid, score) # type: ignore

        # Move to the next document
        current_docid = next_docid

    # Return top-k results sorted by descending score
    return sorted(top.queue, reverse=True)


I have updated the query processing function to leverage the dynamic pruning DAAT algorithm, allowing it to efficiently skip documents that cannot reach the top-k results based on term upper bounds. This modification integrates MaxScore-style optimization directly into the query evaluation, improving retrieval speed while maintaining ranking accuracy.

In [ ]:
def query_process(
    query: str,
    index: 'InvertedIndex',
    termid_to_upperbound: List[float],
    k: int = 10,
) -> List[Tuple[float, int]]:
    """
    Description:
        Processes a query using TAAT or DAAT retrieval on an inverted index.
    Arguments:
        query (str): Input query string.
        index (InvertedIndex): Inverted index containing terms and postings.
        k (int): Number of top-scoring documents to return. Default is 10.
    Returns:
        List[Tuple[float, int]]: List of top-k (score, docid) results.
    """

    # Preprocess query into tokens
    qtokens: List[str] = preprocess(query)   # NOTE: cambiato tipo

    # Map tokens to term IDs
    qtermids: List[int] = index.get_termids(qtokens)

    # Retrieve posting lists for the term IDs
    postings: List['InvertedIndex.PostingListIterator'] = index.get_postings(
        qtermids,
        qtokens
    )

    return dynamic_pruning_daat(postings, termid_to_upperbound, k)

I have updated the run file generation function to pass the precomputed term upper-bound data structure and to use the dynamic pruning DAAT algorithm. This change ensures that the retrieval process applies MaxScore-style optimizations when computing document scores, resulting in faster query evaluation while producing the same top-k results.

In [ ]:
def write_run_file(
    queries: list[tuple[str, str]],
    index: InvertedIndex,
    output_path: str,
    termid_to_upperbound: List[float],
    run_name: str = "my_run",
):
    """
    Description:
        Generates a TREC-formatted run file from a set of queries and the inverted index.

    Arguments:
        queries (Dict[int, str]): Mapping from query IDs to query strings.
        index (InvertedIndex): The inverted index used for retrieval.
        output_path (str): Path to save the output run file.
        run_name (str): Identifier for the run (appears in the last column).
    """

    with open(output_path, "w") as f:
        for qid, query in queries:

            # List of scores and docids ordered by score
            results: list[tuple[np.float64, np.int32]] = query_process(
                query,
                index,
                termid_to_upperbound
            )
            # TODO: ma questi non sono computati come listeee! perchè ora np.array

            # Iterating over results, converting docid into docno while
            # removing DOC and storing the line in the run file
            for rank, (score, docid) in enumerate(results, start=1):
                docno: str = doc_index[docid - 1][0]
                docno: str = docno.replace("DOC", "")
                f.write(f"{qid} Q0 {docno} {rank} {score:.6f} {run_name}\n")

    print(f"Run file saved to: {output_path}")

#### Creating Run File

Generating the run file while simultaneously using time profiling to measure and compare the execution speed against the baseline retrieval model.

In [ ]:

# File where the test queries are stored
queries_file = "msmarco-test2020-queries.tsv"

# Name of the run with the precomputed bm25 weights
run_name = "dp_bm25_run"

# Name of the file where the run will be stored
run_file = run_name + ".txt"

# Scorer Leveraging PreComputed Weights
scorer = BM25Scorer()
# TODO: quindi qua non uso più gli score precomputati, ma l'inv_f non contiene ora il pesi per quello fatto sopra

# Extracting queries from test queries file
queries = extract_queries(queries_file)


# Initialize the inverted index with the current scoring function
# This allows different retrieval models (e.g., TF, TF-IDF, BM25)
inv_ind = InvertedIndex(
    lexicon,       # term -> (termid, ...) mapping
    inv_d,         # list of numpy arrays: doc IDs per term
    inv_f,         # list of numpy arrays: term frequencies per term
    doc_index,     # document statistics
    stats,         # global statistics (e.g., total # of documents)
    scorer=scorer  # scoring function to use for this run
)

# Generate and write a TREC-formatted run file for the current query set
# The filename combines the run name and the run suffix
write_run_file(
    queries,              # queries to process
    inv_ind,              # inverted index with current scoring function
    run_file,             # output filename for this run
    termid_to_upperbound, # upper bounds for each term
    run_name              # run identifier (appears in run file)
)

dynamic_pruning_daat (6341.734 ms)
dynamic_pruning_daat (3366.907 ms)
dynamic_pruning_daat (0.191 ms)
dynamic_pruning_daat (454.333 ms)
dynamic_pruning_daat (9188.710 ms)
dynamic_pruning_daat (5449.454 ms)
dynamic_pruning_daat (3771.396 ms)
dynamic_pruning_daat (9771.373 ms)
dynamic_pruning_daat (2913.263 ms)
dynamic_pruning_daat (1117.760 ms)
dynamic_pruning_daat (5010.983 ms)
dynamic_pruning_daat (7213.560 ms)
dynamic_pruning_daat (1398.828 ms)
dynamic_pruning_daat (12639.566 ms)
dynamic_pruning_daat (7500.100 ms)
dynamic_pruning_daat (8397.099 ms)
dynamic_pruning_daat (3908.665 ms)
dynamic_pruning_daat (32191.107 ms)
dynamic_pruning_daat (2100.518 ms)
dynamic_pruning_daat (598.621 ms)
dynamic_pruning_daat (1369.017 ms)
dynamic_pruning_daat (5667.355 ms)
dynamic_pruning_daat (468.459 ms)
dynamic_pruning_daat (10859.470 ms)
dynamic_pruning_daat (25408.711 ms)
dynamic_pruning_daat (2462.128 ms)
dynamic_pruning_daat (7362.287 ms)
dynamic_pruning_daat (27075.357 ms)
dynamic_pruning_daat 

#### Evaluating Time Efficiency

The results show that this approach is statistically superior to the baseline model; however, it does not achieve the same order-of-magnitude speedup observed with the precomputed weight version.

This code compares the execution times of two retrieval methods. It begins by reading the recorded execution times (in milliseconds) from two text files, where each line represents the duration of a single query run. Using a paired t-test from the scipy.stats library, it tests whether the difference in execution times between the two methods is significant. The script then prints the t-statistic and p-value from the test, along with the average execution time for each method, providing a clear indication of both the magnitude and statistical reliability of the optimization's performance gain.

In [ ]:
from scipy.stats import ttest_rel

# Paths to files containing execution times (in ms)
daat_times_file = "daat_bm25_time.txt"        # baseline DAAT BM25 times
opt_daat_times_file = "dp_daat_bm25_time.txt"  # optimized DAAT BM25 times

# Each file contains one time measurement per line (integer milliseconds)
with open(daat_times_file, "r") as f:
    daat_times = [int(line.strip()) for line in f]

with open(opt_daat_times_file, "r") as f:
    opt_daat_times = [int(line.strip()) for line in f]

# Perform a paired t-test to determine if the optimization led to a significant difference
t_stat, p_val = ttest_rel(daat_times, opt_daat_times)

print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.6e}")

# Compute and print average execution times for both methods
avg_daat = sum(daat_times) / len(daat_times)
avg_opt_daat = sum(opt_daat_times) / len(opt_daat_times)

print(f"Average DAAT BM25 time: {avg_daat:.2f} ms")
print(f"Average DP DAAT BM25 time: {avg_opt_daat:.2f} ms")

T-statistic: 9.0811, P-value: 5.028490e-18
Average DAAT BM25 time: 3901.10 ms
Average DP DAAT BM25 time: 3297.26 ms


The combination of dynamic pruning and precomputed weights did not yield a performance improvement. This was primarily because retrieving the upper bound scores was not significantly faster than accessing the actual precomputed scores. Moreover, the additional logic required to manage upper bound lookups introduced overhead that ultimately offset any potential gains from pruning.

### Compressed Posting Lists

I plan to implement in-memory posting list compression to evaluate the tradeoff between memory usage and execution speed. While I consider runtime performance to be critical and have already applied several optimizations to minimize execution time I'm unlikely to adopt compression in the final implementation. Nonetheless, I will explore and analyze a range of in-memory compression techniques to better understand their impact and potential applicability.

#### VB Encoding

I am implementing in-memory posting list compression using Variable-Byte (VB) encoding, and I have modified the Inverted Index so that posting lists are decoded on-the-fly before creating the iterator. This ensures that the iterators operate on standard integer arrays while benefiting from the reduced memory footprint of VB encoding during storage.

In [ ]:
vb_encode_inv_f(inv_f)
delta_vb_encode_inv_d(inv_d)

delta_vb_encode_inv_d (227062.805 ms)


In [ ]:
class VBInvertedIndex(InvertedIndex):
    """
    Description:
        Inverted Index variant using VB encoded posting lists
    """

    def get_posting(self, termid: int, term: str) -> InvertedIndex.PostingListIterator: #NOTE: cambiato tipo di ritorno che era sbagliato
        """
        Description:
            Decode the posting list and Returns a PostingListIterator
            for a given term ID.
        Arguments:
            termid (int): The term ID to retrieve the posting list for.
        Returns:
            PostingListIterator: Iterator over the posting list.
        """
        return InvertedIndex.PostingListIterator(
            delta_decode(vb_decode(self.inv_d[termid])),
            vb_decode(self.inv_f[termid]),
            termid,
            term,
            self.lexicon,
            self.doc,
            self.stats,
            self.scorer
        )

I am building the index and generating a run file to measure the time overhead caused by decoding the Variable-Byte encoded posting lists during query processing.

In [ ]:
# Using BM25 Scoring
scorer = BM25Scorer()

# Initialize an inverted index that uses Variable-Byte (VB) encoded posting lists.
# This index will decode the postings on-the-fly when iterators are created.
inv_ind = VBInvertedIndex(
  lexicon,      # dictionary mapping terms to (termid, document frequency, etc.)
  inv_d,        # list of VB-encoded document ID arrays for each term
  inv_f,        # list of VB-encoded term frequency arrays for each term
  doc_index,    # document statistics (e.g., lengths, IDs)
  stats,        # global index statistics (e.g., total number of documents)
  scorer,       # scoring function or object used to compute document scores
)

In [ ]:
# Path to the MS MARCO 2019 test queries file (TSV format: query_id<TAB>query_text)
queries_file = "msmarco-test2019-queries.tsv"

# Name of this retrieval run (appears in the TREC run file)
run_name = "vb_bm25_run"

# Output filename for the TREC-formatted run
run_file = run_name + ".txt"

# Read queries from the TSV file and store as a list of tuples: (query_id, query_text)
queries = extract_queries(queries_file)

# Using the PyTerrier BM25 retriever (`br`) to process each query,
# compute scores for retrieved documents, and write results to a run file
write_run_file(
    queries,    # list of (query_id, query_text) tuples
    inv_ind,    # modified inverted index
    run_file,   # output filename for the run file
    run_name    # run identifier to appear in the last column of the run file
)

print(f"Run file '{run_file}' successfully created.")

daat (5163.551 ms)
daat (3685.802 ms)
daat (0.323 ms)
daat (391.442 ms)
daat (7271.406 ms)
daat (4768.119 ms)
daat (3755.634 ms)
daat (12063.397 ms)
daat (2379.610 ms)
daat (1403.468 ms)
daat (6303.511 ms)
daat (7622.447 ms)
daat (920.003 ms)
daat (18329.643 ms)
daat (8329.398 ms)
daat (9257.011 ms)
daat (4901.444 ms)
daat (44958.728 ms)
daat (3051.523 ms)
daat (1198.196 ms)
daat (2778.988 ms)
daat (7828.075 ms)
daat (568.110 ms)
daat (12685.814 ms)
daat (42667.037 ms)
daat (2871.069 ms)
daat (7405.569 ms)
daat (35970.658 ms)
daat (1450.215 ms)
daat (22114.109 ms)
daat (3348.764 ms)
daat (2517.113 ms)
daat (454.056 ms)
daat (12643.918 ms)
daat (3323.396 ms)
daat (3239.571 ms)
daat (13157.421 ms)
daat (14487.130 ms)
daat (4307.601 ms)
daat (6331.963 ms)
daat (20367.587 ms)
daat (20713.626 ms)
daat (33574.625 ms)
daat (15669.034 ms)
daat (16945.583 ms)
daat (47630.548 ms)
daat (2815.055 ms)
daat (7235.986 ms)
daat (35942.822 ms)
daat (4672.345 ms)
daat (2999.860 ms)
daat (387.389 ms)
daa

Variable-Byte compression introduces a noticeable time overhead, as evidenced by the higher mean execution times. This increase in processing time is also statistically significant, as confirmed by the paired t-test results.

In [ ]:
from scipy.stats import ttest_rel

# Paths to files containing execution times (in ms)
daat_times_file = "daat_bm25_time.txt"        # baseline DAAT BM25 times
opt_daat_times_file = "vb_bm25_time.txt"  # optimized DAAT BM25 times

# Each file contains one time measurement per line (integer milliseconds)
with open(daat_times_file, "r") as f:
    daat_times = [int(line.strip()) for line in f]

with open(opt_daat_times_file, "r") as f:
    opt_daat_times = [int(line.strip()) for line in f]

# Perform a paired t-test to determine if the optimization led to a significant difference
t_stat, p_val = ttest_rel(daat_times, opt_daat_times)

print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.6e}")

# Compute and print average execution times for both methods
avg_daat = sum(daat_times) / len(daat_times)
avg_opt_daat = sum(opt_daat_times) / len(opt_daat_times)

print(f"Average DAAT BM25 time: {avg_daat:.2f} ms")
print(f"Average VB DAAT BM25 time: {avg_opt_daat:.2f} ms")


T-statistic: -6.3671, P-value: 5.302682e-10
Average DAAT BM25 time: 3901.10 ms
Average VB DAAT BM25 time: 4997.68 ms


The compressed posting lists show a dramatic reduction in memory usage: the frequency and docID lists occupy approximately 300 MB and 400 MB, respectively, compared to roughly 2 GB for their uncompressed counterparts. This represents a substantial memory savings.

In [ ]:
import sys
from typing import Dict


def get_size(
    inv: Dict[int, Tuple[Any, ...]]
) -> int:
    """
    Computes the approximate memory size of an inverted index stored as a dictionary.

    Each entry in the dictionary maps a term ID (int) to a posting list stored as a tuple.

    Arguments:
        inv (Dict[int, Tuple[Any, ...]]): Inverted index mapping term IDs to posting lists.

    Returns:
        int: Total memory size in bytes occupied by all posting lists in the index.
    """
    total_bytes: int = 0

    # Iterate over each term in the inverted index
    for _term_id, posting_list in inv.items():

        # Add the size of the tuple object itself
        total_bytes += sys.getsizeof(posting_list)

    return total_bytes

In [ ]:
# Printing Sizes in Bytes of frequency and docid
# compressed posting lists
print(f"inv_f size in bytes: {get_size(inv_f)}")
print(f"inv_d size in bytes: {get_size(inv_d)}")

inv_f size in bytes: 349814331
inv_d size in bytes: 446766333


In [ ]:
# Loading uncompressed frequency posting lists into
# memory and printing size in bytes
inv_f = load_object("inv_f")
print(f"uncompressed inv_f size in bytes: {get_size(inv_f)}")

uncompressed inv_f size in bytes: 2110350208


In [ ]:
# Loading uncompressed docids posting lists into
# memory and printing size in bytes
inv_d = load_object("inv_d")
print(f"uncompressed inv_d size in bytes: {get_size(inv_d)}")

uncompressed inv_d size in bytes: 2110350208


However, given that a four-second delay is significant in this type of application and that 2 GB of memory is still manageable within RAM without requiring disk offloading, we conclude that the benefits of variable-byte compression do not outweigh its costs in this context.

#### PForDelta

FastPFor is a C++ high-performance integer compression library designed to efficiently compress large arrays of integers—such as posting lists in search engines—while maintaining fast decompression speeds. It uses PForDelta (Patched Frame of Reference) techniques combined with SIMD (Single Instruction, Multiple Data) optimizations to compress integer sequences by exploiting their small value ranges and patterns, such as gaps between sorted document IDs. The result is a highly space-efficient representation that can be decoded extremely quickly, making FastPFor ideal for large-scale information retrieval systems, inverted indexes, and analytics applications where both memory efficiency and query speed are critical.
PyFastPFor is simply a python wrapper to the C++ library.

In [ ]:
!pip install pyfastpfor

  Using cached pyfastpfor-1.4.0-cp312-cp312-linux_x86_64.whl
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)


This code performs PForDelta compression on the document ID posting lists in an inverted index using the PyFastPFor library. Each posting list is first converted into a NumPy array of 32-bit unsigned integers, then delta-encoded to store the differences between consecutive document IDs—reducing the magnitude of stored numbers and improving compressibility. The SIMD FastPFor codec is then applied to efficiently encode these arrays using vectorized operations. For each posting list, both its original and compressed lengths are recorded to analyze compression performance. By the end of the process, the inv_d structure contains compact, PForDelta-encoded posting lists that use significantly less memory while preserving the ability to be decompressed quickly for retrieval operations.

In [ ]:
import numpy as np
import pyfastpfor as pfor

# -------------------------------------------------------------
# Compressing Document ID Posting Lists Using PyFastPFor (PForDelta)
# -------------------------------------------------------------
# This code applies PForDelta compression to each posting list of document IDs
# in an inverted index. It uses differential encoding (delta4) to store
# differences between consecutive docIDs and compresses the result
# using SIMD FastPFor codec.
# -------------------------------------------------------------

# Load uncompressed posting lists (term_id -> list of docIDs)
inv_d = load_object("inv_d")

# Lists to store statistics for analysis
inv_d_len = []         # original posting list lengths
inv_d_comp_len = []    # compressed lengths (after PForDelta)

# Convert each posting list to a contiguous NumPy array of 32-bit unsigned ints
for term_id in inv_d:
    inv_d[term_id] = np.array(inv_d[term_id], dtype=np.uint32, order="C")  # TODO: che vuol dire order=C?
    inv_d_len.append(len(inv_d[term_id]))

# Initialize the SIMD FastPFor codec (fast, vectorized compression)
codec = pfor.getCodec("simdfastpfor128")

# Apply PForDelta compression to each posting list
for term_id in inv_d:

    # Allocate a temporary buffer to hold the compressed data.
    # Adding +64 ensures enough space to accommodate encoded data.
    tmp = np.zeros(inv_d_len[term_id] + 64, dtype=np.uint32, order="C")

    # Step 1: Apply delta encoding (store gaps between consecutive docIDs)
    # This improves compression efficiency since gaps are smaller than raw IDs.
    pfor.delta4(inv_d[term_id], inv_d_len[term_id])

    # Step 2: Encode the delta-encoded array with SIMD FastPFor
    n_len = codec.encodeArray(
        inv_d[term_id],               # input array
        inv_d_len[term_id],           # number of elements to encode
        tmp,                          # output buffer
        inv_d_len[term_id] + 64       # buffer size
    )

    # Save the compressed size for statistics
    inv_d_comp_len.append(n_len)

    # Replace original posting list with the compressed representation
    inv_d[term_id] = tmp[:n_len]     # truncate to actual compressed size

# -------------------------------------------------------------
# After this step:
#   - inv_d contains compressed posting lists (PForDelta-encoded)
#   - inv_d_len holds original list sizes
#   - inv_d_comp_len holds compressed sizes (useful for compression ratio)
# -------------------------------------------------------------

This code block calculates and reports how effective the PForDelta compression was on the posting lists. It sums up the total size of all posting lists before and after compression and computes the compression ratio, defined as the compressed size divided by the original size. A ratio below 1.0 indicates successful compression, with lower values reflecting greater space savings. Finally, it prints the result in a readable format, allowing for quick evaluation of how much memory storage was reduced through compression.
We achieved a compression ratio similar to VB, now we need to control decoding time overhead.

In [ ]:
# -------------------------------------------------------------
# Compute Compression Statistics for Posting Lists
# -------------------------------------------------------------
# This block computes how effective the compression was by comparing
# the total number of integers before and after compression.
# A ratio < 1.0 indicates that compression reduced storage requirements.
# -------------------------------------------------------------

uncompressed_space = 0   # total number of integers before compression
compressed_space = 0     # total number of integers after compression

# Iterate through all posting lists in the inverted index
for term_id in inv_d:
    # Add up the original and compressed sizes for this term
    uncompressed_space += inv_d_len[term_id]
    compressed_space += inv_d_comp_len[term_id]

# Compute the compression ratio
# Ratio = (compressed size) / (uncompressed size)
ratio = compressed_space / uncompressed_space

# Display results with two decimal precision
print(f"Compression Ratio: {ratio:.2f}")

# -------------------------------------------------------------
# Example interpretation:
#   Compression Ratio = 0.35  →  the compressed index uses ~35% of the space
#   Compression Ratio = 1.00  →  no compression benefit
# -------------------------------------------------------------

Compression Ratio: 0.38


This function, decode_pfor, decompresses a posting list that was previously encoded using the PForDelta (SIMD FastPFor) algorithm. It takes a term ID and retrieves its compressed data from the inverted index (inv_d), along with the corresponding original and compressed lengths. The function then allocates a NumPy array to hold the decoded integers and calls the codec's decodeArray method to reconstruct the original posting list. Finally, it replaces the compressed array in inv_d with the fully decompressed version, restoring the posting list to its original state in memory.

In [ ]:
import numpy as np
import pyfastpfor as pfor
from typing import Dict

@profile
def decode_pfor(
    term_id: int,
    inv_d: Dict[int, np.ndarray],
    inv_d_len: Dict[int, int],
    inv_d_comp_len: Dict[int, int],
    codec: 'pfor.Codec'
) -> None:
    """
    Description:
        Decodes a posting list that was previously compressed using
        PForDelta (SIMD FastPFor). The function replaces the compressed
        posting list with its decompressed version in-place.

    Arguments:
        term_id (int): The term ID whose posting list will be decoded.
        inv_d (Dict[int, np.ndarray]): Dictionary mapping each term ID
            to its compressed posting list.
        inv_d_len (Dict[int, int]): Dictionary containing the original
            (uncompressed) length of each posting list.
        inv_d_comp_len (Dict[int, int]): Dictionary containing the
            compressed length of each posting list.
        codec (pfor.Codec): The PForDelta codec instance used to
            encode/decode posting lists.

    Returns:
        None
        (The function modifies `inv_d` in-place by replacing the compressed
        array with its decompressed version.)
    """

    # Allocate a temporary NumPy array to store the decompressed integers
    tmp = np.zeros(
        inv_d_len[term_id],
        dtype=np.uint32,
        order="C"
    )  # TODO: scrivere order=C meaning

    # Perform the actual PForDelta decoding
    # decodeArray(src, src_len, dest, dest_len)
    codec.decodeArray(
        inv_d[term_id],           # compressed data
        inv_d_comp_len[term_id],  # number of compressed integers
        tmp,                      # destination buffer
        inv_d_len[term_id]        # expected number of decoded integers
    )

    # Replace the compressed posting list with the decompressed one
    return tmp

The total time to decode all posting lists is approximately 9 seconds, which means the decoding time for each individual posting list is well under one millisecond. So the overhead is basicaly non-existant

In [ ]:
# ------------------------------------------------------------- #
# Decode all posting lists using PForDelta (SIMD FastPFor codec)
# ------------------------------------------------------------- #

# Setting a limit to the number of posting lists decoded because otherwise
# the output console will explode, (simply handled with a counter)
max_postings = 1000
cnt = 0

# Iterate over each term ID in the inverted index dictionary
for term_id in inv_d:

    # Updating Counter and evaluating Exit Conditions
    cnt += 1
    if cnt >= max_postings:
        break

    # Decode the posting list for this term ID
    # This function replaces the compressed NumPy array stored in inv_d[term_id]
    # with its decompressed version, restoring the original document IDs.
    decode_pfor(
        term_id,           # term identifier
        inv_d,             # dictionary containing compressed posting lists
        inv_d_len,         # dictionary of original posting list lengths
        inv_d_comp_len,    # dictionary of compressed posting list lengths
        codec              # PForDelta codec used for decoding
    )

# At the end of this loop, all posting lists in inv_d are fully decoded
# and can be directly used for scoring or query evaluation.

decode_pfor (0.081 ms)
decode_pfor (0.415 ms)
decode_pfor (0.059 ms)
decode_pfor (0.122 ms)
decode_pfor (0.594 ms)
decode_pfor (0.085 ms)
decode_pfor (0.161 ms)
decode_pfor (0.210 ms)
decode_pfor (0.082 ms)
decode_pfor (0.040 ms)
decode_pfor (0.015 ms)
decode_pfor (0.027 ms)
decode_pfor (1.566 ms)
decode_pfor (0.114 ms)
decode_pfor (0.101 ms)
decode_pfor (0.042 ms)
decode_pfor (0.103 ms)
decode_pfor (0.056 ms)
decode_pfor (0.019 ms)
decode_pfor (0.306 ms)
decode_pfor (0.450 ms)
decode_pfor (0.113 ms)
decode_pfor (0.075 ms)
decode_pfor (0.209 ms)
decode_pfor (0.018 ms)
decode_pfor (0.198 ms)
decode_pfor (0.208 ms)
decode_pfor (0.113 ms)
decode_pfor (0.147 ms)
decode_pfor (0.012 ms)
decode_pfor (0.286 ms)
decode_pfor (0.041 ms)
decode_pfor (0.167 ms)
decode_pfor (0.493 ms)
decode_pfor (0.163 ms)
decode_pfor (0.118 ms)
decode_pfor (0.128 ms)
decode_pfor (0.015 ms)
decode_pfor (0.088 ms)
decode_pfor (1.475 ms)
decode_pfor (0.031 ms)
decode_pfor (0.153 ms)
decode_pfor (0.376 ms)
decode_pfor

The mean value is 0.0096 ms, that applied to the above mentioned method is negligible with respect to the execution time which is 4 orders of magnitude above.
The limitation with this approach is that it relies on NumPy arrays, which are incompatible with the previously optimized method that depends on native Python lists.

#### Testing DecodePFor + Precomputed Weights

I modified the inverted index class so that it not only decodes the posting list with document IDs but also converts it directly into a Python list. Afterward, I ran the usual procedure to extract execution times and printed summary statistics. Remarkably, we achieved a compression ratio of about one third of the original inv_d size without incurring any noticeable time overhead.

This efficiency stems from two factors: the use of an optimized algorithm and the fact that decompression is handled internally by C++ compiled code, which is far faster than equivalent Python operations. If the entire search engine were implemented in C++, we would likely observe a non‑negligible overhead, but in this hybrid setup the performance remains virtually unaffected.

In [ ]:
# TODO: fare sta cosa è meglio rispetto a runnare con np.array?

In [ ]:
import math
import numpy as np
from typing import List, Dict, Tuple, Callable, Optional


class InvertedIndex:

    class PostingListIterator:
        """
        Description:
            Iterator over a posting list for a single term in the inverted index.
        Arguments:
            docids (np.ndarray): Array of document IDs containing the term.
            freqs (np.ndarray): Array of term frequencies corresponding to docids.
            doc (Dict[int, Tuple]): Dictionary storing document statistics (e.g., length).
        """

        def __init__(
            self,
            docids: np.ndarray,
            freqs: np.ndarray,
            termid: int,
            token: str,
            lexicon: Dict[str, Tuple],
            doc: Dict[int, Tuple],
            stats: Dict[str, int],
            scorer: Callable[['InvertedIndex.PostingListIterator'], float]
        ):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.termid = termid
            self.token = token
            self.lexicon = lexicon
            self.doc = doc
            self.stats = stats
            self._scorer = scorer

        def docid(self) -> int | float:
            """
            Description:
                Returns the current document ID.
            Returns:
                int: Current document ID, or math.inf if iterator is at the end.
            """
            return math.inf if self._is_end_list() else self.docids[self.pos]

        def score(self) -> float:
            """
            Description:
                Computes the score for the current document, which is constant
                for every document in the same posting
            Returns:
                float: Score for the current document, or math.inf if iterator is at the end.
            """
            return self._scorer(self)

        def next(self, target: Optional[int] = None):
            """
            Description:
                Advances the iterator. If target is provided, jump to the first document
                with ID >= target.
            Arguments:
                target (Optional[int]): Target document ID to skip to. Default is None.
            """
            if target is None:
                if not self._is_end_list():
                    self.pos += 1
            elif target > self.docid():
                self.pos = np.searchsorted(self.docids, target, side='left')

        def len(self) -> int:
            """
            Description:
                Returns the number of documents in the posting list.
            Returns:
                int: Length of the posting list.
            """
            return len(self.docids)

        def _is_end_list(self) -> bool:
            """
            Description:
                Checks whether the iterator has reached the end of the posting list.
            Returns:
                bool: True if at end, False otherwise.
            """
            return self.pos >= len(self.docids)

    # ----------------- InvertedIndex main ----------------- #

    """
    Description:
        Inverted index storing the mapping from terms to posting lists, along with
        document statistics.
    Arguments:
        lexicon (Dict[str, Tuple[int, ...]]): Maps token to (termid, ...).
        inv_d (List[np.ndarray]): List of arrays of document IDs per term.
        inv_f (List[np.ndarray]): List of arrays of term frequencies per term.
        doc (Dict[int, Tuple]): Document statistics dictionary.
        stats (Dict[str, int]): Global statistics like number of documents.
    """

    def __init__(
        self,
        lexicon: Dict[str, Tuple[int, ...]],
        inv_d: List[np.ndarray],
        inv_f: List[np.ndarray],
        doc: Dict[int, Tuple],
        stats: Dict[str, int],
        scorer: Callable[['InvertedIndex.PostingListIterator'], float]
    ):
        self.lexicon = lexicon
        self.inv_d = inv_d
        self.inv_f = inv_f
        self.doc = doc
        self.stats = stats
        self.scorer = scorer

    def num_docs(self) -> int:
        """
        Description:
            Returns the total number of documents in the index.
        Returns:
            int: Number of documents.
        """
        return self.stats['num_docs']

    def get_posting(self, termid: int, term: str) -> 'PostingListIterator':
        """
        Description:
            Returns a PostingListIterator for a given term ID.
        Arguments:
            termid (int): The term ID to retrieve the posting list for.
        Returns:
            PostingListIterator: Iterator over the posting list.
        """

        # Decoding inv_d
        decoded_inv_d = decode_pfor(
            termid,           # term identifier
            inv_d,             # dictionary containing compressed posting lists
            inv_d_len,         # dictionary of original posting list lengths
            inv_d_comp_len,    # dictionary of compressed posting list lengths
            codec              # PForDelta codec used for decoding
        )

        # Converting inv_d to python list
        decoded_inv_d = decoded_inv_d.tolist()

        return InvertedIndex.PostingListIterator(
            decoded_inv_d,
            self.inv_f[termid],
            termid,
            term,
            self.lexicon,
            self.doc,
            self.stats,
            self.scorer
        )

    def get_termids(self, tokens: List[str]) -> List[int]:
        """
        Description:
            Returns a list of term IDs for the tokens that exist in the lexicon.
        Arguments:
            tokens (List[str]): List of tokens to look up.
        Returns:
            List[int]: List of corresponding term IDs.
        """
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(
        self,
        termids: List[int],
        terms: List[str]
    ) -> List['PostingListIterator']:
        """
        Description:
            Returns a list of PostingListIterators for a list of term IDs.
        Arguments:
            termids (List[int]): List of term IDs.
        Returns:
            List[PostingListIterator]: List of posting list iterators.
        """
        return [self.get_posting(termid, term) for termid, term in zip(termids, terms)]


In [ ]:

# File where the test queries are stored
queries_file = "msmarco-test2020-queries.tsv"

# Name of the run with the precomputed bm25 weights
run_name = "precomputed_pfor_bm25_run"

# Name of the file where the run will be stored
run_file = run_name + ".txt"

# Scorer Leveraging PreComputed Weights
scorer = PreComputedScorer()
# TODO non bisogna ricomputarli prima? forse mi sbalio e non sono al posto delle frequenze, in caso contrario vanno riocmputati però

# Extracting queries from test queries file
queries = extract_queries(queries_file)


# Initialize the inverted index with the current scoring function
# This allows different retrieval models (e.g., TF, TF-IDF, BM25)
inv_ind = InvertedIndex(
    lexicon,       # term -> (termid, ...) mapping
    inv_d,         # list of numpy arrays: doc IDs per term
    inv_f,         # list of numpy arrays: term frequencies per term
    doc_index,     # document statistics
    stats,         # global statistics (e.g., total # of documents)
    scorer=scorer  # scoring function to use for this run
)

# Generate and write a TREC-formatted run file for the current query set
# The filename combines the run name and the run suffix
write_run_file(
    queries,              # queries to process
    inv_ind,              # inverted index with current scoring function
    run_file,             # output filename for this run
    run_name              # run identifier (appears in run file)
)

decode_pfor (0.045 ms)
decode_pfor (0.009 ms)
daat (0.580 ms)
decode_pfor (0.023 ms)
decode_pfor (0.010 ms)
daat (9.568 ms)
decode_pfor (0.030 ms)
decode_pfor (0.026 ms)
decode_pfor (0.010 ms)
decode_pfor (0.024 ms)
daat (117.372 ms)
decode_pfor (0.028 ms)
decode_pfor (0.008 ms)
decode_pfor (0.018 ms)
daat (42.174 ms)
decode_pfor (0.032 ms)
decode_pfor (0.050 ms)
decode_pfor (0.065 ms)
decode_pfor (0.535 ms)
decode_pfor (0.256 ms)
daat (2300.110 ms)
decode_pfor (0.047 ms)
decode_pfor (0.044 ms)
decode_pfor (0.012 ms)
decode_pfor (0.006 ms)
daat (100.725 ms)
decode_pfor (0.054 ms)
decode_pfor (0.109 ms)
decode_pfor (0.010 ms)
decode_pfor (0.024 ms)
decode_pfor (0.013 ms)
daat (478.214 ms)
decode_pfor (0.031 ms)
decode_pfor (0.020 ms)
decode_pfor (0.007 ms)
daat (47.374 ms)
decode_pfor (0.030 ms)
decode_pfor (0.053 ms)
decode_pfor (0.014 ms)
decode_pfor (0.129 ms)
daat (475.394 ms)
decode_pfor (0.037 ms)
decode_pfor (0.010 ms)
decode_pfor (0.005 ms)
decode_pfor (0.011 ms)
decode_pfor (0.

In [ ]:
from scipy.stats import gmean


# Declaring path to the file containing execution times (one per line)
# and the list that will hold execution times
time_filename = "opt_daat_pfor_bm25_time.txt"
times = []

# Open the file, read all lines and convert them into a list
with open(time_filename, "r") as f:
    lines = f.readlines()                   # Reading Lines
    times = [float(line) for line in lines] # Converting using list comprehension

# Arithmetic mean
mean = sum(times) / len(times)

# Geometric mean is useful for averaging ratios or times when values
# vary multiplicatively, it's computed with scipy package to avoid overflow
geo_mean = gmean(times)

# Sort the list of times to prepare for median calculation and extract
# length as it will avoid lengthy expression
sorted_times = sorted(times)
l = len(sorted_times)

# If the number of elements is odd, take the middle element
if l % 2 == 1:
    median = sorted_times[l // 2]
# If even, take the average of the two middle elements
else:
    median = (sorted_times[l // 2] + sorted_times[(l // 2) - 1]) / 2

# Print results
print("Mean:", mean)
print("Geometric Mean:", geo_mean)
print("Median:", median)

Mean: 439.85722000000004
Geometric Mean: 196.31518052253884
Median: 284.131


| Statistic       | Value              |
|-----------------|--------------------|
| Mean            | 439.857 |
| Geometric Mean  | 196.315 |
| Median          | 284.131            |


In [ ]:
# TODO: qui mancherebbe un t-test con i risultati precedenti per mostrare che non è peggio, no? in ogni caso non lo faremo hahah
# magari si può mettere la tabella con i precomputed ma senza compressione in parte